In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import astropy.units as u
from scipy import ndimage, optimize, stats, interpolate
from skimage import morphology, filters, exposure, measure
from astropy.io import fits
from datetime import datetime
from matplotlib import cm, colormaps, ticker, colors
from matplotlib.ticker import ScalarFormatter

import sunpy.map
from sunpy.coordinates import frames
from astropy.coordinates import SkyCoord
from sunpy.map.maputils import all_coordinates_from_map, coordinate_is_on_solar_disk


import prepare_data
import detect
import plot_detection
from settings import *

%matplotlib inline

# Data Inspection

## Prepare Data

Extract Data from File System

In [ ]:
# Extract He I observation datetimes from FITS files
HE_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, HE_DIR
)

# Extract magnetogram datetimes from 6302l FITS files
MAG_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, MAG_DIR
)

# Extract EUV datetimes from FITS files
EUV_DATE_LIST = prepare_data.get_fits_date_list(
    DATE_RANGE, EUV_DIR
)

date_strs = [HE_DATE_LIST[0], HE_DATE_LIST[-1]]
file_date_str = f'{date_strs[0]}_{date_strs[-1]}'

num_maps = len(HE_DATE_LIST)
datetimes = [datetime.strptime(date_str, DICT_DATE_STR_FORMAT)
             for date_str in date_strs]
title_date_strs = [datetime.strftime(d, '%m/%d/%Y') for d in datetimes]
DATE_RANGE_SUPTITLE = (f'{num_maps} Maps Evaluated from '
                       + f'{title_date_strs[0]} to {title_date_strs[-1]}')

## Available Data

In [ ]:
print('Available Datetimes for He I Observations:')
prepare_data.display_dates(HE_DATE_LIST)

In [ ]:
print('Available Datetimes for Magnetograms:')
prepare_data.display_dates(MAG_DATE_LIST)

In [ ]:
print('Available Datetimes for EUV Observations:')
prepare_data.display_dates(EUV_DATE_LIST)

### Instrumental Darkening Effect

In [ ]:
# Extract data into list
he_data_list = []

for he_date_str in HE_DATE_LIST:
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
    
    he_data_list.append(np.flipud(he_map.data))

Average of maps

In [ ]:
avg_he_data = np.mean(np.array(he_data_list), axis=0)

plt.figure(figsize=(9,7))
plt.title(DATE_RANGE_SUPTITLE)
plt.imshow(avg_he_data,
         #   vmin=-10, vmax=10,
        #    vmin=-50, vmax=50,
           vmin=-100, vmax=100,
           cmap='RdBu')
plt.colorbar()

SVD

In [ ]:
# Create data matrix to compute SVD
all_he_data = np.array(he_data_list)
num_maps = all_he_data.shape[0]
image_shape = all_he_data.shape[1:]

data_matrix = all_he_data.reshape(
    (num_maps, image_shape[0]*image_shape[1])
)
data_matrix = data_matrix.T

# # Retrieve and reshape an image from data matrix for verification
# img_idx = 0
# image = data_matrix[:,img_idx].reshape(image_shape)

In [ ]:
U, S, Vh = np.linalg.svd(data_matrix, full_matrices=False)

In [ ]:
# Exclude last in range
mode_range = [0,12]
num_modes = mode_range[-1] - mode_range[0]

num_cols = 2
image_size = 7
num_rows = int(np.ceil(num_modes/num_cols))
fig = plt.figure(figsize=(image_size*num_cols, image_size*num_rows))
axes = {}

for i in range(0,num_modes):
    axes[i] = fig.add_subplot(num_rows, num_cols, i + 1)
    
    mode = U[:,i].reshape(image_shape)
    bound = np.min([np.abs(np.min(mode)), np.max(mode)])/10
    
    axes[i].imshow(-mode, interpolation='none',
                   cmap=plt.cm.RdBu, vmin=-bound, vmax=bound)
    axes[i].set_title(f'Eigen-Sun {i}', fontsize=24)

In [ ]:
fig = plt.figure(figsize=(6,4))
plt.semilogy(np.arange(S.size), S)
plt.ylabel('Singular Values')
plt.xlabel('Rank')

Rotate

In [ ]:
out_dir = 'output/Rotated_He/'

for he_date_str in HE_DATE_LIST[:1]:
    he_date_str = '2015_06_06__16_08'
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
    
    # rotated_img_file = f'{out_dir}He{he_date_str}.jpg'
    # if os.path.isfile(rotated_img_file):
    #     print((f'He {he_date_str} rotation already exists.'))
    #     continue
    
    datetime = datetime.strptime(he_date_str, DICT_DATE_STR_FORMAT)
    P_angle = sunpy.coordinates.sun.P(time=datetime)

    he_map_rotated = he_map.rotate(angle=P_angle)
    fig = plt.figure(figsize=(5, 5))
    ax = plot_detection.plot_he_map(fig, (1,1,1), he_map_rotated, he_date_str)
# ax = plot_detection.plot_he_map(fig, (1, 1, 1), he_map, he_date_str)
#     fig = plt.figure(figsize=(4,4))
    
    # fig = plt.figure(dpi=300)
    # ax = fig.add_subplot()
    # im = ax.imshow(np.flipud(he_map_rotated.data), vmin=-50, vmax=50, cmap='RdBu',
    #                extent=[0,2700,2700,0])
    # ax.set_title(he_date_str)
    # fig.colorbar(im)

    # plt.savefig(rotated_img_file)
    # plt.close()
    # print(f'{he_date_str} rotated map saved')

In [ ]:
detect.write_video(out_dir, fps=15)

### KPVT Data

In [ ]:
# https://nispdata.nso.edu/ftp/kpvt/daily/medres/ 
fits_path = TEST_HE_DIR + '011219mag.fits'
kpvt_mag_disk_med_res_img = plot_detection.plot_raw_fits_content(
    fits_path, header_list=['UTDATE'], cmaps=[plt.cm.gray, plt.cm.gray],
    # print_header=True
)[0]
# https://nispdata.nso.edu/ftp/kpvt/daily/raw/
kpvt_he_fits_path = TEST_HE_DIR + '01dec19h.fits'
kpvt_he_disk_img = plot_detection.plot_raw_fits_content(
    kpvt_he_fits_path, header_list=['UTDATE'], cmaps=[plt.cm.gray, plt.cm.gray],
    # print_header=True
)[0]
# https://nispdata.nso.edu/ftp/kpvt/daily/raw/
kpvt_mag_fits_path = TEST_HE_DIR + '01dec19m.fits'
kpvt_mag_disk_img = plot_detection.plot_raw_fits_content(
    kpvt_mag_fits_path, header_list=['UTDATE'], cmaps=[plt.cm.gray],
    # print_header=True
)[0]
# https://nispdata.nso.edu/ftp/kpvt/synoptic/hel.hires/
fits_path = TEST_HE_DIR + 'hB1984.fits'
kpvt_he_synoptic_img = plot_detection.plot_raw_fits_content(
    fits_path, header_list=['DATE9'], cmaps=[plt.cm.gray],
    # print_header=True
)[0]
# /ftp/kpvt/synoptic/helium
fits_path = TEST_HE_DIR + 'h1984.fits'
kpvt_he_synoptic_low_res_img = plot_detection.plot_raw_fits_content(
    fits_path, header_list=['DATE'], cmaps=[plt.cm.gray],
    # print_header=True
)[0]

Visualize

In [ ]:
titles = ['2001_12_19', 'CR1984']
plot_detection.plot_hists(
    [kpvt_he_disk_img, kpvt_he_synoptic_img], titles, semilogy=True
)

In [ ]:
plt.imshow(kpvt_he_disk_img, vmin=-200, vmax=100, cmap='gray')

In [ ]:
plt.imshow(kpvt_mag_disk_img, vmin=-50, vmax=50, cmap='gray')

In [ ]:
plt.imshow(kpvt_he_synoptic_low_res_img, vmin=-300, vmax=100, cmap='gray')

### VSM Data

In [ ]:
fits_path = HE_DIR + '2009_10_21__17_50.fts'
rockwell_he_disk_img = plot_detection.plot_raw_fits_content(
    fits_path, header_list=['DATE-OBS'], cmaps=[plt.cm.gray, plt.cm.afmhot],
    # print_header=True
)[0]
fits_path = HE_DIR + '2011_03_28__17_35.fts'
sarnoff_2011_he_disk_img = plot_detection.plot_raw_fits_content(
    fits_path, header_list=['DATE-OBS'], cmaps=[plt.cm.gray, plt.cm.afmhot],
    # print_header=True
)[0]
fits_path = HE_DIR + '2015_03_31__18_13.fts'
sarnoff_2015_he_disk_img = plot_detection.plot_raw_fits_content(
    fits_path, header_list=['DATE-OBS'], cmaps=[plt.cm.gray, plt.cm.afmhot],
    # print_header=True
)[0]
fits_path = TEST_HE_DIR + 'kbv2g150410t1408c2162_000_int-mas_dim-900.fits'
sarnoff_2015_he_synoptic_img = plot_detection.plot_raw_fits_content(
    fits_path, header_list=['DATE-OBS'], cmaps=[plt.cm.gray],
    # print_header=True
)[0]

Visualize

In [ ]:
titles = ['2015_03_31__18_13', 'CR2162']
plot_detection.plot_hists(
    [sarnoff_2015_he_disk_img, sarnoff_2015_he_synoptic_img],
    titles, semilogy=True
)

In [ ]:
plot_detection.plot_images(
    image_list=[rockwell_he_disk_img, sarnoff_2011_he_disk_img],
    title_list=['2009_10_21__17_50', '2011_03_28__17_35']
)

In [ ]:
plt.imshow(rockwell_he_disk_img, vmin=-100, vmax=100, cmap='gray')

# Single Map Data

## Extract Data

Observations and pre-computed segmentations

### Observations

In [ ]:
# KPVT: H&H
he_date_str = '2003_07_14__18_07'

# # Rockwell: Null detection
# he_date_str = '2004_11_20__17_07'

# Rockwell: Smiley
# he_date_str = '2004_12_03__16_36'

# Rockwell: CH boundary difference
# he_date_str = '2004_11_13__16_31'

# Rockwell: Solar min case
# he_date_str = '2009_10_22__18_38'

# Sarnoff: Pre-updated FITS
# he_date_str = '2012_04_01__17_03'

# Sarnoff: Start of Updated FITS in May
# he_date_str = '2012_05_01__18_08'

# Sarnoff: Poster case
# he_date_str = '2012_06_11__18_01'

# Sarnoff: East limb hammer
# he_date_str = '2012_06_28__16_44'

# Failed limb detection
# he_date_str = '2012_07_08__19_37'

# Sarnoff: Greatest area in 06/2012
# he_date_str = '2012_06_09__19_20'

# # COSPAR: East axe
# he_date_str = '2015_01_04__20_30'

# # COSPAR: N-S polar
# he_date_str = '2015_01_20__20_25'

# # COSPAR: Center Hook
# he_date_str = '2015_02_10__18_45'

# # COSPAR: QS blotch
# he_date_str = '2015_04_18__17_22'

# # COSPAR: Trianglular CH
# he_date_str = '2015_06_06__16_08'

# he_date_str = HE_DATE_LIST[0]

he_fits_file = DATA_FITS_FORMAT.format(
    data_dir=HE_DIR, date_str=he_date_str
)
he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
he_map_data = np.flipud(he_map.data)

fig = plt.figure(figsize=(4, 4))
ax = plot_detection.plot_he_map(fig, (1, 1, 1), he_map, he_date_str)

In [ ]:
mag_date_str = prepare_data.get_nearest_date_str(
    MAG_DATE_LIST, selected_date_str=he_date_str
)

# Extract magnetogram
mag_fits_file = DATA_FITS_FORMAT.format(
    data_dir=MAG_DIR, date_str=mag_date_str
)
mag_map = prepare_data.get_nso_sunpy_map(mag_fits_file)

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=mag_map)
mag_map.plot(axes=ax, vmin=-50, vmax=50)

In [ ]:
euv_date_str = prepare_data.get_nearest_date_str(
    EUV_DATE_LIST, selected_date_str=he_date_str
)

# Extract euv map
euv_fits_file = DATA_FITS_FORMAT.format(
    data_dir=EUV_DIR, date_str=euv_date_str
)
euv_map = sunpy.map.Map(euv_fits_file)

fig = plt.figure(figsize=(4, 4))
ax = plot_detection.plot_euv_map(fig, (1, 1, 1), euv_map, euv_date_str)

### Pre-Processed Products

v0.5.1

In [ ]:
# Extract FITS file pre-processed map
pre_process_fits_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
pre_processed_map = sunpy.map.Map(pre_process_fits_file)
pre_processed_map_data = np.flipud(pre_processed_map.data)

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111, projection=pre_processed_map)
pre_processed_map.plot(axes=ax)

In [ ]:
# Extract differentially rotated magnetogram map
reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                         f'Mag{mag_date_str}_He{he_date_str}.fits')
reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

# Extract saved processed magnetogram
reprojected_smooth_file = (f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}'
                           f'_He{he_date_str}_smooth.fits')
reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_file)

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=reprojected_mag_map)
reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50)

v0.1-v0.5

In [ ]:
#  Extract saved pre-processed image array
pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                    + '_pre_processed_map.npy')
pre_processed_map_data = np.load(pre_process_file, allow_pickle=True)[-1]
pre_processed_map = sunpy.map.Map(np.flipud(pre_processed_map_data), he_map.meta)

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=pre_processed_map)
pre_processed_map.plot(axes=ax, title='')

In [ ]:
# Extract Heliographic coordinate reprojected magnetogram map
hg_mag_fits_file = (f'{HELIOGRAPH_MAG_SAVE_DIR}'
                    f'Mag{mag_date_str}_He{he_date_str}.fits')
hg_mag_map = sunpy.map.Map(hg_mag_fits_file)

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111, projection=hg_mag_map)
hg_mag_map.plot(axes=ax, title='', vmin=-50, vmax=50)

### Ensemble Maps

v0.5.1

In [ ]:
# Extract saved ensemble map
ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
ensemble_map = sunpy.map.Map(ensemble_file)
ensemble_map_data = np.flipud(ensemble_map.data)

# Extract saved processed magnetogram
reprojected_smooth_file = (f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}'
                           f'_He{he_date_str}_smooth.fits')
reprojected_smooth_map = sunpy.map.Map(reprojected_smooth_file)

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=ensemble_map)
ensemble_map.plot(axes=ax, title='', cmap='magma')
plot_detection.plot_map_contours(ax, reprojected_smooth_map)

v0.2-0.5

In [ ]:
# Extract saved ensemble map array and convert to Sunpy map
ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
ensemble_map.plot_settings['cmap'] = colormaps['magma']

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=ensemble_map)
ensemble_map.plot(axes=ax, title='')

### Single Mask

In [ ]:
# Create testing ensemble map from a single segmentation
# percent_of_peak = 70
# morph_radius_dist = 15

percent_of_peak = 80
morph_radius_dist = 13

ch_mask_data = detect.get_ch_mask_list_v0_5_1(
    pre_processed_map, [percent_of_peak], [morph_radius_dist]
)[0]
ensemble_map_data = np.where(~np.isnan(np.flipud(pre_processed_map.data)), 0, np.nan)
ensemble_map_data = np.where(ch_mask_data, 100, ensemble_map_data)

ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), pre_processed_map.meta)
ensemble_map.plot_settings['cmap'] = colormaps['magma']

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111, projection=ensemble_map)
ensemble_map.plot(axes=ax, title='')

In [ ]:
# v0.5: Extract saved single mask array and convert to Sunpy map
mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
mask_data = np.load(mask_file, allow_pickle=True)[-1]
mask_map = sunpy.map.Map(np.flipud(mask_data), he_map.meta)
mask_map.plot_settings['cmap'] = colormaps['gray']

he_base_data = np.where(he_map.data == he_map.data[0,0], np.nan, he_map.data)
he_base_map = sunpy.map.Map(he_base_data, he_map.meta)

fig = plt.figure(figsize=(24, 5))

plot_detection.plot_he_map(fig, (1, 4, 1), he_map, he_date_str)

# Plot He I observation with overlayed detection contours
ax = fig.add_subplot(142, projection=he_map)
he_base_map.plot(axes=ax, vmin=-100, vmax=100, title=he_date_str,
                    cmap='afmhot')
for contour in mask_map.contour(0):
    ax.plot_coord(contour, color='black', linewidth=1)

plot_detection.plot_euv_map(fig, (1, 4, 3), euv_map, euv_date_str)

ax = fig.add_subplot(144, projection=he_map)
reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50, title=mag_date_str)
plot_detection.plot_map_contours(ax, reprojected_smooth_map)

## Inspect Observations

In [ ]:
# Crop EUV map to similar zoom level to other observations 
he_submap = he_map.submap(
    bottom_left=SkyCoord(
        # Tx=-750*u.arcsec, Ty=-600*u.arcsec,
        Tx=-150*u.arcsec, Ty=-600*u.arcsec,
        frame=he_map.coordinate_frame
    ),
    top_right=SkyCoord(
        # Tx=0*u.arcsec, Ty=0*u.arcsec,
        Tx=600*u.arcsec, Ty=0*u.arcsec,
        frame=he_map.coordinate_frame
    )
)
he_submap.plot(vmin=-100, vmax=100)

In [ ]:
hist_data = np.where(he_submap.data.flatten() == 0, np.nan, he_submap.data.flatten())
edges = np.arange(-100, 101, 2)

fig = plt.figure(figsize=(7,5), dpi=150)
ax = fig.add_axes(111)

data = ax.hist(
    hist_data, edges, histtype='step',
    color='white', edgecolor='black', linewidth=3,
)
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

ax.set_ylabel('Counts')
ax.set_xlabel('Equivalent Width (m\u00C5)')

## Publication Plots

In [ ]:
# He I Image

# # Remove off limb pixels
# all_hp_coords = sunpy.map.maputils.all_coordinates_from_map(he_map)
# on_disk_mask = sunpy.map.maputils.coordinate_is_on_solar_disk(all_hp_coords)

# he_plot_map = sunpy.map.Map(
#     np.where(on_disk_mask, he_map.data, -100), he_map.meta
# )
he_plot_map = he_map

fig = plt.figure(figsize=(10, 10))
ax = plot_detection.plot_he_map(fig, (1, 1, 1), he_plot_map, he_date_str)

ax.set_title('')
ax.coords.grid(False)
for coord in ax.coords:
    coord.set_ticks_visible(False)
    coord.set_ticklabel_visible(False)

In [ ]:
# EUV Image
fig = plt.figure(figsize=(10, 10))
ax = plot_detection.plot_euv_map(fig, (1, 1, 1), euv_map, euv_date_str)

ax.set_title('')
ax.coords.grid(False)
for coord in ax.coords:
    coord.set_ticks_visible(False)
    coord.set_ticklabel_visible(False)

In [ ]:
# Magnetogram
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=mag_map)

display_mag_map = sunpy.map.Map(
    np.where(mag_map.data == 0, -50, mag_map.data), mag_map.meta
)
display_mag_map.plot(axes=ax, vmin=-50, vmax=50)

ax.set_title('')
ax.coords.grid(False)
for coord in ax.coords:
    coord.set_ticks_visible(False)
    coord.set_ticklabel_visible(False)

In [ ]:
# Ensemble Plot
custom_font = {'fontname':'Times New Roman'}

dark = True
background_val = -20
# background_val = 0


if dark:
    cmap = 'magma'
    ensemble_plot_map = sunpy.map.Map(
        np.where(ensemble_map.data == 0, background_val, ensemble_map.data), he_map.meta
    )
else:
    cmap = 'viridis'
    ensemble_plot_map = sunpy.map.Map(
        np.where(ensemble_map.data == 0, np.nan, ensemble_map.data), he_map.meta
    )

# fig = plt.figure(figsize=(8, 6), dpi=200)
fig = plt.figure(figsize=(4, 3), dpi=400)
ax = fig.add_subplot(111, projection=ensemble_plot_map)
ensemble_plot_map.plot(axes=ax, cmap=cmap, annotate=False, vmin=0, vmax=100)
plot_detection.plot_map_contours(ax, reprojected_smooth_map)


if dark:
    ensemble_plot_map.draw_grid(axes=ax)
else:
    ensemble_plot_map.draw_grid(axes=ax, color='black')
    ensemble_plot_map.draw_limb(axes=ax, color='black')

ax.coords.grid(False)
for coord in ax.coords:
    coord.set_ticks_visible(False)
    coord.set_ticklabel_visible(False)

color_range = np.linspace(np.interp(0, [background_val, 100], [0, 1]), 1, 256)
cmap_object = colormaps[cmap]
reduced_cmap = colors.ListedColormap(cmap_object(color_range))

norm = colors.Normalize(vmin=0, vmax=1)
cb = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=reduced_cmap), ax=ax)
cb.set_label('Confidence', labelpad=11, rotation=-90)#, **custom_font)
cb.ax.tick_params(labelsize=8)

Methods

In [ ]:
# He I Histogram

hist_data = np.where(he_map_data.flatten() == 0, np.nan, he_map_data.flatten())
edges = np.arange(-100, 101, 10)

fig = plt.figure(figsize=(7*0.6,5*0.6), dpi=300)
ax = fig.add_axes(111)

data = ax.hist(
    hist_data, edges, histtype='step',
    color='white', edgecolor='black', linewidth=2,
)
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax.set_xticks([-100, -50, 0, 50, 100])

ax.set_ylabel('Counts')
ax.set_xlabel('He I Pixel Intensity')

In [ ]:
# Pre-Processed He I Histogram

hist_data = pre_processed_map_data.flatten()
edges = np.arange(-1, 1.1, 0.1)

fig = plt.figure(figsize=(7*0.6,5*0.6), dpi=300)
ax = fig.add_axes(111)

data = ax.hist(
    hist_data, edges, histtype='step',
    color='white', edgecolor='black', linewidth=2,
)
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax.set_xticks([-1, -0.5, 0, 0.5, 1])

ax.set_ylabel('Counts')
ax.set_xlabel('Rescaled He I Pixel Intensity')

In [ ]:
# Threshold Mask

# percent_of_peak = 90
percent_of_peak = 70

thresh_bound = detect.get_thresh_bound(pre_processed_map_data, percent_of_peak)

thresh_mask = np.where(pre_processed_map_data > thresh_bound, 1, 0)
empty_disk = np.where(~np.isnan(pre_processed_map_data), 0, np.nan)
thresh_disk = np.where(thresh_mask, 1, empty_disk)
thresh_map = sunpy.map.Map(np.flipud(thresh_disk), he_map.meta)

# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(1, 1, 1)
# ax.imshow(thresh_disk, cmap='magma')

# ax.tick_params(left=False, right=False, labelleft=False,
#                labelbottom=False, bottom=False)

fig = plt.figure(figsize=(3, 3), dpi=400)
ax = fig.add_subplot(111, projection=thresh_map)
thresh_map.plot(axes=ax, cmap='magma', annotate=False)
thresh_map.draw_grid(axes=ax)


ax.coords.grid(False)
for coord in ax.coords:
    coord.set_ticks_visible(False)
    coord.set_ticklabel_visible(False)

In [ ]:
# Preliminary Segmentation Mask

percent_of_peak = 90
morph_radius = 13

# percent_of_peak = 70
# morph_radius = 15

ch_mask = detect.get_ch_mask_list_v0_5_1(
    pre_processed_map, [percent_of_peak], [morph_radius]
)[0]

empty_disk = np.where(~np.isnan(pre_processed_map_data), 0, np.nan)
ch_disk = np.where(ch_mask, 1, empty_disk)
ch_map = sunpy.map.Map(np.flipud(ch_disk), he_map.meta)

# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(1, 1, 1)
# ax.imshow(ch_disk, cmap='magma')

# ax.tick_params(left=False, right=False, labelleft=False,
#                labelbottom=False, bottom=False)

fig = plt.figure(figsize=(3, 3), dpi=400)
ax = fig.add_subplot(111, projection=ch_map)
ch_map.plot(axes=ax, cmap='magma', annotate=False)
ch_map.draw_grid(axes=ax)


ax.coords.grid(False)
for coord in ax.coords:
    coord.set_ticks_visible(False)
    coord.set_ticklabel_visible(False)

# Single Segmentation

Requires single map extraction from single map data section

## Pre-Process

### Versions

v0.1

In [ ]:
pre_process_v0_1_he, he_high_cut, he_nan = detect.pre_process_v0_1(
    he_map_data, peak_count_cutoff_percent=0.1
)

arrays = [he_map_data, he_nan, he_high_cut, pre_process_v0_1_he]
titles = ['he', 'he NaN', 'he High Cut', 'he Band Cut']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

In [ ]:
# Compare dates
date_idx = 0
for he_date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    compare_he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    raw_he = prepare_data.get_image_from_fits(compare_he_fits_file)
    pre_process_v0_1_he = detect.pre_process_v0_1(raw_he)[0]
    arrays = [raw_he, pre_process_v0_1_he]
    titles = [he_date_str, 'Pre-Processed']
    plot_detection.plot_hists(arrays, titles, semilogy=True)

v0.4

In [ ]:
pre_process_v0_4_he = detect.pre_process_v0_4(he_map_data)

arrays = [he_map_data, pre_process_v0_4_he]
titles = ['L2 Observation', 'Pre-Processed Observation']

plot_detection.plot_hists(arrays, titles, semilogy=True)

In [ ]:
# Compare dates
date_idx = 0
for he_date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    compare_he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    raw_he = prepare_data.get_image_from_fits(compare_he_fits_file)
    pre_process_v0_4_he = detect.pre_process_v0_4(raw_he)
    arrays = [raw_he, pre_process_v0_4_he]
    titles = [he_date_str, 'Pre-Processed']
    plot_detection.plot_hists(arrays, titles, semilogy=True)

v0.5.1

In [ ]:
pre_processed_map_v0_5_1_map = detect.pre_process_v0_5_1(he_map)

arrays = [he_map_data, np.flipud(pre_processed_map_v0_5_1_map.data)]
titles = ['L2 Observation', 'Pre-Processed Observation']

plot_detection.plot_hists(arrays, titles, semilogy=True)

vY

In [ ]:
pre_processed_vY_map = detect.pre_process_vY(he_map)

fig = plt.figure(figsize=(11, 10))

ax = fig.add_subplot(2, 2, 1, projection=he_map)
he_map.plot(axes=ax, title=he_map.date)

ax = fig.add_subplot(2, 2, 2, projection=he_map)
he_map.plot(axes=ax, vmin=-100, vmax=100, title='+/-100 mAngstrom Saturation')

ax = fig.add_subplot(2, 2, (3,4), projection=pre_processed_vY_map)
pre_processed_vY_map.plot(axes=ax, title='')

### Save Outputs

v0.1-v0.4

In [ ]:
overwrite = False

if not os.path.isdir(PREPROCESS_SAVE_DIR):
    os.makedirs(PREPROCESS_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    if os.path.isfile(pre_process_file) and not overwrite:
        print((f'He {he_date_str} pre-processed map already exists.'))
        continue
    
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    he_map_data = prepare_data.get_image_from_fits(he_fits_file)
    
    # pre_processed_map_data = detect.pre_process_v0_1(he_map_data)[0]
    pre_processed_map_data = detect.pre_process_v0_4(he_map_data)
    
    save_list = [he_date_str, pre_processed_map_data]
    np.save(pre_process_file, np.array(save_list, dtype=object), 
            allow_pickle=True)
    print(f'{he_date_str} Pre-Processed Map Saved')

vY

In [ ]:
overwrite = False

if not os.path.isdir(PREPROCESS_MAP_SAVE_DIR):
    os.makedirs(PREPROCESS_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    if os.path.isfile(pre_process_file) and not overwrite:
        print((f'He {he_date_str} pre-processed map already exists.'))
        continue
    
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
    
    pre_processed_map = detect.pre_process_vY(he_map)
    
    pre_processed_map.save(pre_process_file, overwrite=overwrite)
    print(f'{he_date_str} Pre-Processed Map Saved')

HE I/EUV Ratio

In [ ]:
overwrite = False


if not os.path.isdir(RATIO_SAVE_DIR):
    os.makedirs(RATIO_SAVE_DIR)

for he_date_str in HE_DATE_LIST:

    euv_date_str = prepare_data.get_nearest_date_str(
        EUV_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Optionally overwrite existing files
    ratio_fits_file = f'{RATIO_SAVE_DIR}He{he_date_str}_EUV{euv_date_str}.fits'
    if os.path.isfile(ratio_fits_file):
        if overwrite:
            os.remove(ratio_fits_file)
        else:
            print((f'{he_date_str} to {euv_date_str} ratio already exists.'))
            continue
    
    # Extract He I observation
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    # Remove error causing keywords which have invalid ascii content
    he_map.meta.pop('history')
    he_map.meta.pop('comment')
    
    # Extract and reproject EUV observation
    euv_fits_file = DATA_FITS_FORMAT.format(
        data_dir=EUV_DIR, date_str=euv_date_str
    )
    euv_map = sunpy.map.Map(euv_fits_file)
    reprojected_euv_map = prepare_data.diff_rotate(
        input_map=euv_map, target_map=he_map
    )
    
    # Pre-process He I data via background removal and upper cutoff
    # Satisfactory only for Sarnoff camera observations
    he_map_data = np.where(he_map.data == he_map.data[0,0],
                           np.nan, he_map.data)
    he_map_data = np.where(he_map_data >= np.percentile(he_map_data, 99.9),
                           np.nan, he_map_data)
    
    ratio_data = np.divide(he_map_data, (reprojected_euv_map.data)**0.5)
    ratio_map = sunpy.map.Map(ratio_data, he_map.meta)
    
    # Save to FITS files
    ratio_map.save(ratio_fits_file)
    print(f'He{he_date_str} to EUV{euv_date_str} ratio saved.')

He I/EUV Ratio Pre-Process

In [ ]:
overwrite = False

if not os.path.isdir(PREPROCESS_SAVE_DIR):
    os.makedirs(PREPROCESS_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    euv_date_str = prepare_data.get_nearest_date_str(
        EUV_DATE_LIST, selected_date_str=he_date_str
    )
    
    # Optionally overwrite existing files
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    if os.path.isfile(pre_process_file) and not overwrite:
        print((f'He {he_date_str} pre-processed map already exists.'))
        continue

    ratio_fits_file = f'{RATIO_SAVE_DIR}He{he_date_str}_EUV{euv_date_str}.fits'

    ratio_map_data = prepare_data.get_image_from_fits(ratio_fits_file)
    
    # pre_processed_map_data = detect.pre_process_v0_1(ratio_map_data)[0]
    pre_processed_map_data = detect.pre_process_v0_4(ratio_map_data)
    
    save_list = [he_date_str, pre_processed_map_data]
    np.save(pre_process_file, np.array(save_list, dtype=object), 
            allow_pickle=True)
    print(f'{he_date_str} Pre-Processed Map Saved')

### Alternates

In [ ]:
def remove_peak_counts(array):
    """Retrieve an array with the value of peak counts replaced with NaN.
    """
    peak_counts_val = detect.get_peak_counts_loc(array, bins_as_percent=False)
    zero_vals = (array > peak_counts_val - 1e-2) & (array < peak_counts_val + 1e-2)
    
    return np.where(zero_vals, np.NaN, array)

def band_pass(raw_he):
    """Pre-process equivalent width array by setting background to NaN
    and a simple brightness band pass.
    """
    he_nan = np.where(raw_he == 0, np.NaN, raw_he)
    
    he_high_cut = np.where(he_nan > 100, np.NaN, he_nan)
    # he_band_cut = np.where(he_high_cut < -100, np.NaN, he_high_cut)
    he_band_cut = np.clip(he_high_cut, -100, 100)
    
    return he_band_cut, he_high_cut, he_nan


def equalize(raw_he):
    """Pre-process equivalent width array by setting background to NaN
    and a simple brightness band pass.
    """
    # Histogram equalization
    he1 = exposure.equalize_hist(raw_he)
    he1 = detect.remove_background(he1)
    
    # Shift nonzero values into positive range and equalize histogram
    he2 = np.where(raw_he == 0, 0, raw_he + np.abs(np.min(raw_he)))
    he3 = exposure.equalize_hist(he2)
    
    he3 = np.where(he3 == np.min(he3), np.NaN, he3)
    
    return he3, he2, he1


def rescale(raw_he):
    """Pre-process equivalent width array by applying linear rescaling
    to normalize the contrast and setting background to NaN. Linear
    rescaling between 2-98 percentiles produces a less harsh contrast
    enhancement than histogram equalization.
    """
    p2, p98 = np.percentile(raw_he[~np.isnan(raw_he)], (2, 98))
    
    # Shift nonzero values into positive range and normalize
    he1 = np.where(raw_he == 0, 0, raw_he + np.abs(np.min(raw_he)))
    he2 = exposure.rescale_intensity(he1, in_range=(p2, p98))
    
    # Normalize directly
    he3 = exposure.rescale_intensity(raw_he, in_range=(p2, p98))
    he3 = detect.remove_background(he3)
        
    return he3, he2, he1


def rescale_center(raw_he):
    """Pre-process equivalent width array by applying linear rescaling
    to normalize the contrast, set background to NaN, and centering mode
    to zero.
    """
    p2, p98 = np.percentile(raw_he, (2, 98))
    
    # Linearly rescale
    he1 = exposure.rescale_intensity(raw_he, in_range=(p2, p98))    
    he2 = detect.remove_background(he1)
    
    # Center mode to zero
    peak_counts_val = detect.get_peak_counts_loc(he2, bins_as_percent=False)
    he3 = he2 - peak_counts_val + 1

    return he3, he2, he1

Band Pass

In [ ]:
he_band_cut, he_high_cut, he_nan = band_pass(he_map_data)

arrays = [he_map_data, he_nan, he_high_cut, he_band_cut]
titles = ['he', 'he NaN', 'he High Cut', 'he Band Cut']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

Equalize

In [ ]:
he3, he2, he1 = equalize(he_map_data)

arrays = [he_map_data, he1, he2, he3]
titles = ['he', 'Equalized', 'Shifted', 'Shifted & Equalized']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

In [ ]:
hist, edges = detect.get_hist(he3, bins_as_percent=True, n=1000)
plt.semilogy(edges[0:-1], hist)
detect.get_peak_counts_loc(he3)

Rescaled

In [ ]:
he3, he2, he1 = rescale(he_map_data)

arrays = [he_map_data, he3, he1, he2]
titles = ['he', 'Stretched', 'Shifted', 'Shifted & Stretched']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

In [ ]:
he3, he2, he1 = rescale_center(he_map_data)

arrays = [he_map_data, he1, he2, he3]
titles = ['he', 'Stretched', 'Shifted', 'Removed Background']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

Pre-Processed Ratio

In [ ]:
ratio_fits_file = f'{RATIO_SAVE_DIR}He{he_date_str}_EUV{euv_date_str}.fits'
raw_ratio = prepare_data.get_image_from_fits(ratio_fits_file)
ratio = detect.pre_process_v0_4(raw_ratio)

arrays = [raw_ratio, ratio]
titles = ['Raw Ratio', 'Pre-Processed Ratio']

plot_detection.plot_hists(arrays, titles, semilogy=True)

Off-Limb Masking

In [ ]:
all_hp_coords = all_coordinates_from_map(he_map)
mask = coordinate_is_on_solar_disk(all_hp_coords)
limb_removed_he_map = sunpy.map.Map(
    np.where(mask, he_map.data, np.nan), he_map.meta
)

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=limb_removed_he_map)
limb_removed_he_map.plot(axes=ax, title='', vmin=-100, vmax=100)

In [ ]:
all_hp_coords = all_coordinates_from_map(mag_map)
mask = coordinate_is_on_solar_disk(all_hp_coords)
limb_removed_mag_map = sunpy.map.Map(mag_map.data, mag_map.meta, mask=~mask)

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=limb_removed_mag_map)
limb_removed_mag_map.plot(axes=ax, title='', vmin=-50, vmax=50)

## Reprojection

Save Heliographic Reprojection Outputs

In [ ]:
overwrite = True
smooth_size_percent = 10

if not os.path.isdir(HELIOGRAPH_MAG_SAVE_DIR):
    os.makedirs(HELIOGRAPH_MAG_SAVE_DIR)

for he_date_str in HE_DATE_LIST:

    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    
    fits_file_name = f'{HELIOGRAPH_MAG_SAVE_DIR}Mag{mag_date_str}_He{he_date_str}'
    reprojected_fits_file = f'{fits_file_name}.fits'
    reprojected_smooth_fits_file = f'{fits_file_name}_smooth.fits'
    
    # Optionally overwrite existing files
    if (os.path.isfile(reprojected_fits_file) or \
        os.path.isfile(reprojected_smooth_fits_file)) and not overwrite:
        print((f'{mag_date_str} magnetogram reprojected '
                + f'to {he_date_str} already exists.'))
        continue
    
    # Extract observations
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)
    
    mag_fits_file = DATA_FITS_FORMAT.format(
        data_dir=MAG_DIR, date_str=mag_date_str
    )
    mag_map = prepare_data.get_nso_sunpy_map(mag_fits_file)

    # Process magnetogram
    hg_mag_map = detect.reproject_to_cea(mag_map)
    reprojected_mag_map = hg_mag_map.reproject_to(
        pre_processed_map.wcs, algorithm='adaptive'
    )
    
    smoothed_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent)
    hg_smoothed_map = detect.reproject_to_cea(smoothed_map)
    reprojected_smooth_map = hg_smoothed_map.reproject_to(
        pre_processed_map.wcs, algorithm='adaptive'
    )
    
    # Save to FITS files
    reprojected_mag_map.save(reprojected_fits_file, overwrite=overwrite)
    reprojected_smooth_map.save(reprojected_smooth_fits_file, overwrite=overwrite)
    print(f'{mag_date_str} magnetogram reprojected to {he_date_str} map saved.')

Differential rotation

In [ ]:
he_date_str = HE_DATE_LIST[3]
mag_date_str = MAG_DATE_LIST[4]

he_fits_file = DATA_FITS_FORMAT.format(
   data_dir=HE_DIR, date_str=he_date_str
)
mag_fits_file = DATA_FITS_FORMAT.format(
    data_dir=MAG_DIR, date_str=mag_date_str
)
he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
mag_map = prepare_data.get_nso_sunpy_map(mag_fits_file)

fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection=mag_map)
mag_map.plot(axes=ax1, vmin=-50, vmax=50,
               title=f'Original: {mag_map.date}')

smoothed_map = prepare_data.get_smoothed_map(mag_map, smooth_size_percent=10)
plot_detection.plot_map_contours(ax1, smoothed_map)

reprojected_map = prepare_data.diff_rotate(
   input_map=mag_map, target_map=he_map
)

ax2 = fig.add_subplot(122, projection=reprojected_map)
reprojected_map.plot(axes=ax2, vmin=-50, vmax=50,
                     title=f'Reprojection: {reprojected_map.date}')

reprojected_smooth_map = prepare_data.diff_rotate(
   input_map=smoothed_map, target_map=he_map
)
plot_detection.plot_map_contours(ax2, reprojected_smooth_map)

Helioprojective Scale

In [ ]:
# (HP Tx, Ty arcsec)/pix > (HP Tx, Ty arcsec)
Tx_scale = he_map.scale.axis1.to(u.arcsec/u.pix) * u.pix
Ty_scale = he_map.scale.axis2.to(u.arcsec/u.pix) * u.pix

f'Tx: {Tx_scale.value:.5f} Ty: {Ty_scale.value:.5f} arcsec'

In [ ]:
# (HP Tx, Ty arcsec) > (HC Mm)
hp_delta_coords = frames.Helioprojective(
    he_map.scale.axis1*u.pix,
    he_map.scale.axis2*u.pix,
    observer='earth', obstime=he_map.date
)
hc_delta_coords = hp_delta_coords.transform_to(
    frames.Heliocentric(observer='earth', obstime=he_map.date)
)
f'x: {hc_delta_coords.x.to(u.Mm).value:.5f} y: {hc_delta_coords.x.to(u.Mm).value:.5f} Mm'

#### Carrington Non-CEA

In [ ]:
# Reprojection map shape scaling factors
# Increase to increase map resolution and reduce distance scale / pixel
# Aim to match Helioprojective scale to preserve resolution
NON_CEA_LON_FACTOR = 1.55
NON_CEA_LAT_FACTOR = 1.55

# Obtain dimension  in image pixel number of the solar radius
Rs_hp_coord = SkyCoord(
    he_map.rsun_obs, 0*u.arcsec, frame='helioprojective',
    observer='earth', obstime=he_map.date
)
Rs_pixel_pair = he_map.world_to_pixel(Rs_hp_coord)
ref_pixel_pair = he_map.world_to_pixel(he_map.reference_coordinate)
Rs_dim = int((Rs_pixel_pair.x - ref_pixel_pair.x).value)

new_rows = int(2*Rs_dim*NON_CEA_LAT_FACTOR)
new_cols = int(4*Rs_dim*NON_CEA_LON_FACTOR)

hg_header = sunpy.map.header_helper.make_heliographic_header(
    he_map.date, he_map.observer_coordinate,
    shape=(new_rows, new_cols), frame='stonyhurst'
)

# Convert to 180 deg center longitude for Carrington map visualization
hg_header['crval1'] = 180
non_cea_map = he_map.reproject_to(
    hg_header#, algorithm='adaptive'
)

fig = plt.figure(figsize=(16, 5))

ax = fig.add_subplot(1, 3, 1, projection=he_map)
he_map.plot(axes=ax, vmin=-100, vmax=100, title=he_map.date)

ax = fig.add_subplot(1, 3, (2,3), projection=non_cea_map)
non_cea_map.plot(axes=ax, vmin=-100, vmax=100, title='')

(new_rows, new_cols)

Non-CEA Scale

In [ ]:
# (HG lon, lat deg)/pix > (HG lon, lat deg)
lon_scale = non_cea_map.scale.axis1.to(u.deg/u.pix) * u.pix
lat_scale = non_cea_map.scale.axis2.to(u.deg/u.pix) * u.pix

f'lon: {lon_scale.value:.5f} lat: {lat_scale.value:.5f} deg'

In [ ]:
# (HG lon, lat deg) > (HC Mm)
x_scale = (non_cea_map.rsun_meters * lon_scale.to(u.rad)/u.rad).to(u.Mm)
y_scale = (non_cea_map.rsun_meters * lat_scale.to(u.rad)/u.rad).to(u.Mm)

f'x: {x_scale.value:.5f} y: {y_scale.value:.5f} Mm'

In [ ]:
# WRONG
# (HG lon, lat deg) > (HC Mm)
hg_delta_coords = frames.HeliographicStonyhurst(lon_scale, lat_scale)
hc_delta_coords = hg_delta_coords.transform_to(
    frames.Heliocentric(observer='earth', obstime=non_cea_map.date)
)

x_scale = hc_delta_coords.x.to(u.Mm)
y_scale = hc_delta_coords.y.to(u.Mm)
f'x: {x_scale.value:.5f} y: {y_scale.value:.5f} Mm'

# WRONG
# (HP??? lon, lat deg) > (HC Mm)
hp_delta_coords = frames.Helioprojective(
    lon_scale, lat_scale,
    observer='earth', obstime=he_map.date
)
hc_delta_coords = hp_delta_coords.transform_to(
    frames.Heliocentric(observer='earth', obstime=non_cea_map.date)
)

x_scale = hc_delta_coords.x.to(u.Mm)
y_scale = hc_delta_coords.y.to(u.Mm)
f'x: {x_scale.value:.5f} y: {y_scale.value:.5f} Mm'

#### Stonyhurst CEA

In [ ]:
# Reprojection map shape scaling factors
# Increase to increase map resolution and reduce distance scale / pixel
# Aim to match Helioprojective scale within 0.01 tolerance to preserve resolution
CEA_X_SCALE_FACTOR = np.pi/2
CEA_Y_SCALE_FACTOR = 1

# Obtain dimension  in image pixel number of the solar radius
Rs_hp_coord = SkyCoord(
    he_map.rsun_obs, 0*u.arcsec, frame='helioprojective',
    observer='earth', obstime=he_map.date
)
Rs_pixel_pair = he_map.world_to_pixel(Rs_hp_coord)
ref_pixel_pair = he_map.world_to_pixel(he_map.reference_coordinate)
Rs_dim = int((Rs_pixel_pair.x - ref_pixel_pair.x).value)

new_row_num = int(2*Rs_dim*CEA_Y_SCALE_FACTOR)
new_col_num = int(4*Rs_dim*CEA_X_SCALE_FACTOR)

hg_header = sunpy.map.header_helper.make_heliographic_header(
    he_map.date,
    he_map.observer_coordinate,
    # observer,
    shape=(new_row_num, new_col_num), frame='stonyhurst',
    projection_code='CEA'
)

# Specify Earth-based observer for solar radius, distance to Sun,
# and Heliographic coordinates to avoid warning messages due to
# missing keywords
earth_hp_coords = frames.Helioprojective(
    Tx=0*u.arcsec, Ty=0*u.arcsec,
    observer='earth', obstime=he_map.date,
)
earth_header = sunpy.map.make_fitswcs_header((1,1), earth_hp_coords)
for earth_coord_key in ['RSUN_REF', 'DSUN_OBS', 'HGLN_OBS', 'HGLT_OBS']:
    hg_header[earth_coord_key] = earth_header[earth_coord_key]

cea_he_map = he_map.reproject_to(
    hg_header, #algorithm='adaptive'
)

# Crop map to within 90 degrees of the central meridian
top_right = SkyCoord(
    lon=90*u.deg, lat=90*u.deg, frame=cea_he_map.coordinate_frame
)
bottom_left = SkyCoord(
    lon=-90*u.deg, lat=-90*u.deg, frame=cea_he_map.coordinate_frame
)
cea_he_map = cea_he_map.submap(bottom_left, top_right=top_right)

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111, projection=cea_he_map)
cea_he_map.plot(axes=ax, vmin=-100, vmax=100)

(new_row_num, new_col_num)

Raw Scale

In [ ]:
# (HP Tx, Ty arcsec)/pix > (HP Tx, Ty arcsec)
Tx_scale = he_map.scale.axis1.to(u.arcsec/u.pix) * u.pix
Ty_scale = he_map.scale.axis2.to(u.arcsec/u.pix) * u.pix

f'Tx: {Tx_scale.value:.5f} Ty: {Ty_scale.value:.5f} arcsec'

In [ ]:
# (HP Tx, Ty arcsec) > (HC Mm)
hp_delta_coords = frames.Helioprojective(
    he_map.scale.axis1*u.pix,
    he_map.scale.axis2*u.pix,
    observer='earth', obstime=he_map.date
)
hc_delta_coords = hp_delta_coords.transform_to(
    frames.Heliocentric(observer='earth', obstime=he_map.date)
)
f'x: {hc_delta_coords.x.to(u.Mm).value:.5f} y: {hc_delta_coords.x.to(u.Mm).value:.5f} Mm'

In [ ]:
cea_mag_map = detect.reproject_to_cea(mag_map)

reprojected_mag_map = cea_mag_map.reproject_to(
    cea_he_map.wcs, #algorithm='adaptive'
)

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111, projection=reprojected_mag_map)
reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50)

CEA Scale

In [ ]:
# Scale Available: (HG lon, lat deg)/pix > (HG lon, lat deg)
lon_scale = cea_he_map.scale.axis1.to(u.deg/u.pix) * u.pix
lat_scale = cea_he_map.scale.axis2.to(u.deg/u.pix) * u.pix

f'lon: {lon_scale.value:.5f} lat: {lat_scale.value:.5f} deg'

In [ ]:
# (HG lon, lat deg) > (HC Mm)
x_scale = (cea_he_map.rsun_meters * lon_scale.to(u.rad)/u.rad).to(u.Mm)
y_scale = (cea_he_map.rsun_meters * lat_scale.to(u.rad)/u.rad).to(u.Mm)

f'x: {x_scale.value:.5f} y: {y_scale.value:.5f} Mm'

In [ ]:
# WRONG
# (HG lon, lat deg) > (HC Mm)
hg_delta_coords = frames.HeliographicStonyhurst(
    pre_processed_map.scale.axis1*u.pix,
    pre_processed_map.scale.axis2*u.pix,
)
hc_delta_coords = hg_delta_coords.transform_to(
    frames.Heliocentric(observer='earth', obstime=pre_processed_map.date)
)
x_scale = hc_delta_coords.x.to(u.Mm)
y_scale = hc_delta_coords.y.to(u.Mm)
f'x: {x_scale.value:.5f} y: {y_scale.value:.5f} Mm'

# WRONG
# (HP??? lon, lat deg) > (HC Mm)
hp_delta_coords = frames.Helioprojective(
    pre_processed_map.scale.axis1*u.pix,
    pre_processed_map.scale.axis2*u.pix,
    observer='earth', obstime=he_map.date
)
hc_delta_coords = hp_delta_coords.transform_to(
    frames.Heliocentric(observer='earth', obstime=pre_processed_map.date)
)
x_scale = hc_delta_coords.x.to(u.Mm)
y_scale = hc_delta_coords.y.to(u.Mm)
f'x: {x_scale.value:.5f} y: {y_scale.value:.5f} Mm'

## Preliminary Segmentations

In [ ]:
GREEN = '#6ece58'
BLUE = '#3e4989'
ORANGE = '#fd9668'
PURPLE = '#721f81'


def get_thresh_px_percent_list(array, percent_of_peak_list):
    """Retrieve the area percentage of pixels accepted by varied thresholds.
    """
    thresh_bound_list = [
        detect.get_thresh_bound(array, percent_of_peak)
        for percent_of_peak in percent_of_peak_list
    ]
    px_percent_list = [
        np.count_nonzero(array > thresh_bound)*100/array.size
        for thresh_bound in thresh_bound_list
    ]
    return px_percent_list


def get_parameter_stats(outcome_list):
    """Retrieve maximum difference between segmentations in area percentage
    detected, the average area percentage at the max difference for a cutoff,
    the number selected below this cutoff, and differences in area percentage.
    """    
    outcome_diffs = np.abs(np.diff(outcome_list))

    max_diff_i = np.argmax(outcome_diffs)
    max_diff = np.max(outcome_diffs)*100/outcome_list[max_diff_i]
    
    cutoff = np.mean([outcome_list[max_diff_i], 
                      outcome_list[max_diff_i + 1]])

    selected_parameter_num = np.count_nonzero(outcome_list > cutoff)
    
    return max_diff, cutoff, selected_parameter_num, outcome_diffs


def plot_pixel_percent_bars(ax, parameter_list, pixel_percent_list,
                            max_diff, cutoff, selected_parameter_num,
                            step, title, unit, xlabel, thresh=True):
    bar_width = 0.8*step
    selected_parameters = parameter_list[selected_parameter_num:]
    
    ax.set_title(f'{title}\n Cutoff: {selected_parameters[0]}{unit} | ' +
                 f'Max Difference: {max_diff:.1f}%' , fontsize=28)
    ax.set_xlabel(xlabel, fontsize=24)
    
    ax.set_ylabel('Pixel Percentage (%)', fontsize=24)
    
    ax.plot([parameter_list[0] - step/2, parameter_list[-1] + step/2], [cutoff, cutoff], 
               linestyle='--', color='k', linewidth=3)
    
    if thresh:
        ax.bar(parameter_list, pixel_percent_list, 
               width=bar_width, color=BLUE)
        ax.bar(selected_parameters, 
               pixel_percent_list[selected_parameter_num:], 
               width=bar_width, color=GREEN)
    else:
        ax.bar(parameter_list, pixel_percent_list,
               width=bar_width, color=PURPLE)
        ax.bar(selected_parameters,
               pixel_percent_list[selected_parameter_num:], 
               width=bar_width, color=ORANGE)

### Threshold

#### Versions

v0.1 Pre-Process

In [ ]:
pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]

plot_detection.plot_thresholds(
    pre_process_v0_1_he, bounds=[75, 90, 105], bounds_as_percent=True
)

In [ ]:
# Compare dates
date_idx = 0
for he_date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    compare_he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    raw_he = prepare_data.get_image_from_fits(compare_he_fits_file)
    he = detect.pre_process_v0_1(raw_he)[0]

    plot_detection.plot_thresholds(he, bounds=[75, 85, 100], bounds_as_percent=True)

In [ ]:
# Parameter sweep
step = 5
percent_of_peak_lists = [
    list(np.arange(0,200,step)), list(np.arange(80,130,step))
]

for percent_of_peak_list in percent_of_peak_lists:
    px_percent_list = get_thresh_px_percent_list(pre_process_v0_1_he, percent_of_peak_list)
    
    parameter_stats = get_parameter_stats(px_percent_list)
    max_diff, cutoff, selected_parameter_num, pixel_percent_diffs = parameter_stats
    
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot()

    plot_pixel_percent_bars(
        ax, percent_of_peak_list, px_percent_list, max_diff, cutoff, selected_parameter_num,
        step, title='Threshold', unit='%', xlabel='Percent of Peak Pixel Count (%)', thresh=True)

v0.4 Pre-Process

In [ ]:
pre_process_v0_4_he = detect.pre_process_v0_4(he_map_data)

plot_detection.plot_thresholds(
    pre_process_v0_4_he, bounds=[75, 90, 105], bounds_as_percent=True
)

v0.5.1 Pre-Process

In [ ]:
pre_processed_v0_5_1_map = detect.pre_process_v0_5_1(he_map)

plot_detection.plot_thresholds(
    np.flipud(pre_processed_v0_5_1_map.data), bounds=[75, 90, 105],
    bounds_as_percent=True
)

vY Pre-Process

In [ ]:
pre_processed_vY_map = detect.pre_process_vY(he_map)

plot_detection.plot_thresholds(
    np.flipud(pre_processed_vY_map.data), bounds=[75, 90, 105],
    bounds_as_percent=True
)

### Structuring Element Radius

In [ ]:
def plot_varied_morph_radius(pre_processed_map_data, percent_of_peak_list,
                             morph_radius_list, ch_mask_list, px=False):
    plot_detection.plot_images([pre_processed_map_data], image_size=4)

    image_list = [pre_processed_map_data for _ in range(len(ch_mask_list))]
    axes = plot_detection.plot_image_grid(
        image_list, num_cols=3, cmap='afmhot', image_size=7
    )
    zipped_items = zip(axes.values(), percent_of_peak_list,
                       morph_radius_list, ch_mask_list)

    for ax, percent_of_peak, radius, ch_mask in zipped_items:
        if px:
            ax.set_title(f'{percent_of_peak:d}% of Peak | {radius:d}px Radius')
        else:
            ax.set_title((f'{percent_of_peak:d}% of Mode Threshold | '
                          f'{radius:d}Mm SE Disk Radius'))
        
        ax.tick_params(left=False, right=False, labelleft=False,
                       labelbottom=False, bottom=False)
            
        ax.contour(ch_mask, cmap='gray')

#### Versions

v0.1 Pre-Process

In [ ]:
morph_radius_list = [12,16,20]
percent_of_peak_list = [90 for _ in range(len(morph_radius_list))]

pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]

ch_mask_list = [
    detect.get_ch_mask(pre_process_v0_1_he, percent_of_peak, morph_radius)
    for percent_of_peak, morph_radius
    in zip(percent_of_peak_list, morph_radius_list)
]
plot_varied_morph_radius(pre_process_v0_1_he, percent_of_peak_list,
                         morph_radius_list, ch_mask_list, px=True)

In [ ]:
# Parameter sweep
step = 2
morph_radius_lists = [
    list(np.arange(1,21,step)), list(np.arange(8,15,step))
]

for morph_radius_list in morph_radius_lists:
    percent_of_peak_list = [90 for _ in range(len(morph_radius_list))]
    ch_mask_list = [
        detect.get_ch_mask(pre_process_v0_1_he, percent_of_peak, morph_radius)
        for percent_of_peak, morph_radius
        in zip(percent_of_peak_list, morph_radius_list)
    ]
    
    px_percent_list = detect.get_px_percent_list(ch_mask_list)
    
    parameter_stats = get_parameter_stats(px_percent_list)
    max_diff, cutoff, selected_parameter_num, pixel_percent_diffs = parameter_stats
    
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot()

    plot_pixel_percent_bars(
        ax, morph_radius_list, px_percent_list, max_diff, cutoff, selected_parameter_num,
        step, title='SE Disk Radius', unit='px', xlabel='SE Disk Radius (px)', thresh=False
    )

v0.4 Pre-Process

In [ ]:
morph_radius_list = [12,16,20]
percent_of_peak_list = [90 for _ in range(len(morph_radius_list))]

pre_process_v0_4_he = detect.pre_process_v0_4(he_map_data)

ch_mask_list = [
    detect.get_ch_mask(pre_process_v0_4_he, percent_of_peak, morph_radius)
    for percent_of_peak, morph_radius
    in zip(percent_of_peak_list, morph_radius_list)
]
plot_varied_morph_radius(pre_process_v0_4_he, percent_of_peak_list,
                         morph_radius_list, ch_mask_list, px=True)

v0.5.1 Pre-Process

In [ ]:
morph_radius_list = [8, 12, 16]
percent_of_peak_list = [80 for _ in range(len(morph_radius_list))]


pre_processed_v0_5_1_map = detect.pre_process_v0_5_1(he_map)

ch_mask_list = detect.get_ch_mask_list_v0_5_1(
    pre_processed_v0_5_1_map, percent_of_peak_list, morph_radius_list
)
plot_varied_morph_radius(
    np.flipud(pre_processed_v0_5_1_map.data), percent_of_peak_list,
    morph_radius_list, ch_mask_list
)

vY Pre-Process

In [ ]:
he_date_str = HE_DATE_LIST[1]
morph_radius_list = [8, 12, 16]
percent_of_peak_list = [90 for _ in range(len(morph_radius_list))]


pre_processed_vY_map = detect.pre_process_vY(he_map)

ch_mask_list = detect.get_ch_mask_list_vY(
    pre_processed_vY_map, percent_of_peak_list, morph_radius_list
)
plot_varied_morph_radius(
    np.flipud(pre_processed_vY_map.data), percent_of_peak_list,
    morph_radius_list, ch_mask_list
)

#### Alternates

In [ ]:
# Initial distance definition of SE disk radius ideas

# Area square: (HP Tx, Ty arcsec) > (HC Mm)
# Get HG coords: (pixel) > (HP Tx, Ty arcsec) > (HG lon, lat deg)
# Reproject CEA: (HP Tx, Ty arcsec) > (pixel)

# SE Disk Radius: (HC Mm) > (pixel)
# HC to HP/HG to pixel
# To pixel step fails
empty_dim_list = [0*u.km for _ in morph_radius_list]
morph_radius_hc_coords = SkyCoord(
    x=[morph_radius_dist*u.Mm for morph_radius_dist in morph_radius_list],
    y=empty_dim_list, z=empty_dim_list, frame='heliocentric',
    observer='earth', obstime=pre_processed_map.date
)
morph_radius_hp_coords = morph_radius_hc_coords.transform_to(
    frames.Helioprojective(observer='earth', obstime=pre_processed_map.date)
)
morph_radius_pixel_coord = pre_processed_map.world_to_pixel(
    morph_radius_hc_coords
).x
ref_pixel_coord = pre_processed_map.world_to_pixel(
    pre_processed_map.reference_coordinate
).x

### Save Single Mask Outputs

v0.2-v0.4

In [ ]:
overwrite = False

percent_of_peak = 80
morph_radius = 18


if not os.path.isdir(DETECTION_SAVE_DIR):
    os.makedirs(DETECTION_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    mask_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(mask_file) and not overwrite:
        print((f'He {he_date_str} single mask already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]

    ch_mask = detect.get_ch_mask(
        pre_processed_map, percent_of_peak, morph_radius
    )
    
    save_list = [he_date_str, ch_mask]
    np.save(mask_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Single Mask Saved')

### Design Variable Grid

Includes fill and remove steps

In [ ]:
def plot_design_var_grid(pre_processed_map_data, percent_of_peak_list,
                         morph_radius_list, ch_mask_list, num_cols):
    image_list = [pre_processed_map_data for _ in range(len(ch_mask_list))]
    axes = plot_detection.plot_image_grid(
        image_list, num_cols, cmap='afmhot', image_size=7
    )
    zipped_items = zip(axes.values(), percent_of_peak_list,
                    morph_radius_list, ch_mask_list)

    for ax, percent_of_peak, radius, ch_mask in zipped_items:
        ax.set_title(f'{percent_of_peak:d}% of Peak | {radius:d}Mm Radius')
        
        ax.tick_params(left=False, right=False, labelleft=False,
                        labelbottom=False, bottom=False)
            
        ax.contour(ch_mask, cmap='gray')

In [ ]:
def plot_varied_masks(pre_processed_map_data, percent_of_peak_list,
                      morph_radius_list, ch_mask_list, title=True):
    """UNUSED: Plot each mask as an individual image with not axis ticks"""
    plot_detection.plot_images([pre_processed_map_data], image_size=4)
    
    zipped_items = zip(percent_of_peak_list,
                       morph_radius_list, ch_mask_list)
    
    for percent_of_peak, radius, ch_mask in zipped_items:
        
        empty_disk = np.where(~np.isnan(pre_processed_map_data), 0, np.nan)
        ch_disk = np.where(ch_mask, 1, empty_disk)
        
        axes = plot_detection.plot_image_grid(
            [ch_disk], num_cols=3, cmap='magma', image_size=7
        )
        
        if title:
            axes[0].set_title((f'{percent_of_peak:d}% of Mode Threshold | '
                               f'{radius:d}Mm SE Disk Radius'))
        
        axes[0].tick_params(left=False, right=False, labelleft=False,
                       labelbottom=False, bottom=False)

percent_of_peak_list = [90, 70]
morph_radius_list = [13, 15]

ch_mask_list = detect.get_ch_mask_list_v0_5_1(
    pre_processed_map, percent_of_peak_list, morph_radius_list
)
plot_varied_masks(
    np.flipud(pre_processed_map.data), percent_of_peak_list,
    morph_radius_list, ch_mask_list, title=False
)

#### Versions

##### v0.5.1 Pre-Process

Design Grid

In [ ]:
# v0.5.1 SOLIS Design
percent_of_peak_list = [70, 70, 80, 90]
morph_radius_list = [   15, 17, 13, 13] # Mm

# # v0.5.1 KPVT Design
# percent_of_peak_list = [85, 105, 85, 95]
# morph_radius_list = [   17, 13, 15, 13] # Mm

ch_mask_list = detect.get_ch_mask_list_v0_5_1(
    pre_processed_map, percent_of_peak_list, morph_radius_list
)

plot_design_var_grid(
    pre_processed_map_data, percent_of_peak_list, morph_radius_list,
    ch_mask_list, num_cols=2
)

Exploratory Grid

In [ ]:
# # Breaks up QS merger for COSPAR triangular CH
# percent_of_peak_list = [80, 70, 80, 90]
# morph_radius_list = [   12, 17, 13, 13] # Mm

percent_of_peaks = [80, 90, 100]
morph_radii = [      9, 13, 17] # Mm
percent_of_peak_list = [percent_of_peak 
                        for _ in morph_radii
                        for percent_of_peak in percent_of_peaks]
morph_radius_list = [morph_radius 
                     for morph_radius in reversed(morph_radii)
                     for _ in percent_of_peaks]

ch_mask_list = detect.get_ch_mask_list_v0_5_1(
    pre_processed_map, percent_of_peak_list, morph_radius_list
)

plot_design_var_grid(
    pre_processed_map_data, percent_of_peak_list, morph_radius_list,
    ch_mask_list, num_cols=len(percent_of_peaks)
)

##### vY Pre-Process

In [ ]:
percent_of_peak_list = [90, 90]
morph_radius_list = [10, 15]

pre_processed_vY_map = detect.pre_process_vY(he_map)

ch_mask_list = detect.get_ch_mask_list_vY(
    pre_processed_vY_map, percent_of_peak_list, morph_radius_list
)
plot_varied_morph_radius(
    np.flipud(pre_processed_vY_map.data), percent_of_peak_list,
    morph_radius_list, ch_mask_list
)

## Ensemble

### Versions

Appropriate pre-processed products must be extracted

v0.2: Detected Pixel Percentage Sort

In [ ]:
percent_of_peak_list = [80,80,90,100,100]
morph_radius_list = [13,17,15,13,17] # px


out = detect.get_ensemble_v0_2(
    pre_processed_map_data, percent_of_peak_list, morph_radius_list
)
ensemble_map_data, ch_mask_list, confidence_list, px_percent_list = out
plot_detection.plot_ensemble(
    pre_processed_map_data, ensemble_map_data, ch_mask_list,
    confidence_list, px_percent_list, mask_contour=True
)

v0.3: Evenly Assigned Smoothness

In [ ]:
percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [   13,17, 15, 13, 17] # px


out = detect.get_ensemble_v0_3(
    pre_processed_map_data, percent_of_peak_list, morph_radius_list
)
ensemble_map, map_data_by_ch, confidence_list, gradient_medians = out
plot_detection.plot_ensemble(
    pre_processed_map_data, ensemble_map, map_data_by_ch,
    confidence_list, gradient_medians
)

v0.5: Directly Assigned Unipolarity

In [ ]:
percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [   18,20, 16, 16, 20] # px
unipolarity_threshold = 0.5


out = detect.get_ensemble_v0_5(
    pre_processed_map, reprojected_mag_map,
    percent_of_peak_list, morph_radius_list, unipolarity_threshold
)
ensemble_map_data, map_data_by_ch, confidence_list, unipolarity_by_ch = out

plot_detection.plot_ensemble(
    pre_processed_map_data, ensemble_map_data, map_data_by_ch,
    confidence_list, unipolarity_by_ch
)

v0.5.1

In [ ]:
# Conservative Design
# percent_of_peak_list = [80, 80, 90, 100]
# morph_radius_list = [   15, 17, 13, 13] # Mm

# Aggressive Design
percent_of_peak_list = [70, 70, 80, 90]
morph_radius_list = [   15, 17, 13, 13] # Mm

unipolarity_threshold = 0.5


out = detect.get_ensemble_v0_5_1(
    pre_processed_map, reprojected_mag_map,
    percent_of_peak_list, morph_radius_list,
    unipolarity_threshold
)
ensemble_map_data, masks_by_ch, confidence_list, unipolarity_by_ch = out

plot_detection.plot_ensemble(
    pre_processed_map_data, ensemble_map_data, masks_by_ch,
    confidence_list, unipolarity_by_ch
)

vY

In [ ]:
percent_of_peak_list = [85, 73, 95, 85]
morph_radius_list = [   10, 14, 10, 14]
unipolarity_threshold = 0.5

out = detect.get_ensemble_vY(
    pre_processed_map, hg_mag_map,
    percent_of_peak_list, morph_radius_list,
    unipolarity_threshold
)
ensemble_map_data, masks_by_ch, confidence_list, unipolarity_by_ch = out

plot_detection.plot_ensemble(
    pre_processed_map_data, ensemble_map_data, masks_by_ch,
    confidence_list, unipolarity_by_ch
)

### Save Outputs

v0.2-v0.4

In [ ]:
overwrite = True

# percent_of_peak_list = [80,80, 90, 100,100]
# morph_radius_list = [18,20, 16, 16,20]
percent_of_peak_list = [100,100, 110, 120,120]
morph_radius_list = [18,20, 16, 16,20]


if not os.path.isdir(DETECTION_SAVE_DIR):
    os.makedirs(DETECTION_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]

    # ensemble_map_data = detect.get_ensemble_v0_2(
    #     pre_processed_map, percent_of_peak_list, morph_radius_list
    # )[0]
    ensemble_map_data = detect.get_ensemble_v0_3(
        pre_processed_map, percent_of_peak_list, morph_radius_list
    )[0]
    
    save_list = [he_date_str, ensemble_map_data]
    np.save(ensemble_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Ensemble Map Saved')

v0.5

In [ ]:
overwrite = False

percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [18,20, 16, 16,20]
unipolarity_threshold = 0.5


if not os.path.isdir(DETECTION_SAVE_DIR):
    os.makedirs(DETECTION_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract He I observation
    he_fits_file = DATA_FITS_FORMAT.format(
        data_dir=HE_DIR, date_str=he_date_str
    )
    he_map = prepare_data.get_nso_sunpy_map(he_fits_file)

    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map_data = np.load(pre_process_file, allow_pickle=True)[-1]
    pre_processed_map = sunpy.map.Map(
        np.flipud(pre_processed_map_data), he_map.meta
    )

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                            f'Mag{mag_date_str}_He{he_date_str}.fits')
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    ensemble_map_data = detect.get_ensemble_v0_5(
        pre_processed_map, reprojected_mag_map,
        percent_of_peak_list, morph_radius_list,
        unipolarity_threshold
    )[0]
    
    save_list = [he_date_str, ensemble_map_data]
    np.save(ensemble_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Ensemble Map Saved')

v0.5.1

In [ ]:
overwrite = False

# v0.5.1 SOLIS Design
percent_of_peak_list = [70, 70, 80, 90]
morph_radius_list = [   15, 17, 13, 13] # Mm
# unipolarity_threshold = 0.5
unipolarity_threshold = 0

# # v0.5.1 KPVT Design
# percent_of_peak_list = [85, 105, 85, 95]
# morph_radius_list = [   17, 13, 15, 13] # Mm
# unipolarity_threshold = 0


if not os.path.isdir(DETECTION_MAP_SAVE_DIR):
    os.makedirs(DETECTION_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract pre-processed map
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                             f'Mag{mag_date_str}_He{he_date_str}.fits')
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    ensemble_map_data = detect.get_ensemble_v0_5_1(
        pre_processed_map, reprojected_mag_map,
        percent_of_peak_list, morph_radius_list,
        unipolarity_threshold
    )[0]
    ensemble_map = sunpy.map.Map(
        np.flipud(ensemble_map_data), pre_processed_map.meta
    )
    
    ensemble_map.save(ensemble_file, overwrite=overwrite)
    print(f'{he_date_str} Ensemble Map Saved')

vY

In [ ]:
overwrite = False

# percent_of_peak_list = [  62, 68, 73, 80]
# morph_radius_list = [11, 13,  8, 10]
# unipolarity_threshold = 0.01
percent_of_peak_list = [85, 73, 95, 85]
morph_radius_list = [   10, 14, 10, 14]
unipolarity_threshold = 0.5



if not os.path.isdir(DETECTION_MAP_SAVE_DIR):
    os.makedirs(DETECTION_MAP_SAVE_DIR)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
    if os.path.isfile(ensemble_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    # Extract pre-processed map
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    reprojected_fits_file = (f'{HELIOGRAPH_MAG_SAVE_DIR}'
                            f'Mag{mag_date_str}_He{he_date_str}.fits')
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    ensemble_map_data = detect.get_ensemble_vY(
        pre_processed_map, reprojected_mag_map,
        percent_of_peak_list, morph_radius_list,
        unipolarity_threshold
    )[0]
    ensemble_map = sunpy.map.Map(
        np.flipud(ensemble_map_data), pre_processed_map.meta
    )
    
    ensemble_map.save(ensemble_file, overwrite=overwrite)
    print(f'{he_date_str} Ensemble Map Saved')

### Alternates

v0.3b: Percentile Assigned Smoothness

In [ ]:
def get_smooth_ensemble(pre_processed_map, percent_of_peak_list,
                        morph_radius_list, even_confidence=True):
    """Retrieve an ensemble of segmentations sorted by CH smoothness.
    
    Args
        array: image to process
        percent_of_peak_list: list of float percentage values
            at which to take threshold
        morph_radius_list: list of int pixel number for radius of disk 
            structuring element in morphological operations
        even_confidence: boolean to specify confidence assignment
            True to assign confidence by even ranking in (0,100]%
            False to assign confidence by percentile of gradient
                median among values from other candidate CHs in [0,100]%
    Returns
        Ensemble greyscale coronal holes mask sorted by mean gradient.
        List of binary coronal holes masks.
        List of confidence levels in mask layers.
    """
    # Create global segmentations for varied design variable combinations
    ch_masks = [
        detect.get_ch_mask(pre_processed_map, percent_of_peak, morph_radius)
        for percent_of_peak, morph_radius
        in zip(percent_of_peak_list, morph_radius_list)
    ]
    
    # Lists to hold pre processed map and gradient data respectively
    # for distinct CHs from all segmentations
    map_data_by_ch = []
    grad_data_by_ch = []
    
    for ch_mask in ch_masks:
        # Masked array of candidate CHs
        masked_candidates = detect.get_masked_candidates(pre_processed_map, ch_mask)
        
        # Compute spatial gradient
        gradient_candidates = filters.sobel(masked_candidates)
        
        map_data_by_ch.extend(
            detect.get_map_data_by_ch(pre_processed_map, ch_mask)
        )
        grad_data_by_ch.extend(
            detect.get_map_data_by_ch(gradient_candidates, ch_mask)
        )
    
     # Obtain sorting indixes from greatest to least gradient median
    gradient_medians = [np.median(grad_data[~np.isnan(grad_data)])
                        for grad_data in grad_data_by_ch]
    sorted_idxs = np.flip(np.argsort(gradient_medians))
    
    # Sort candidate CHs from greatest to least gradient median
    map_data_by_ch = [map_data_by_ch[i] for i in sorted_idxs]
    gradient_medians = [gradient_medians[i] for i in sorted_idxs]
    
    # Assign confidence by percentile or direct ranking
    num_ch = len(map_data_by_ch)
    if even_confidence:
        confidence_list = [(c + 1)*100/num_ch
                           for c in range(num_ch)]
    else:
        percent_conversion = 100 / (np.max(gradient_medians)
                                    - np.min(gradient_medians))
        confidence_list = [
            100 - (grad_median - np.min(gradient_medians)) *percent_conversion
            for grad_median in gradient_medians
        ]

    # Construct ensemble map by adding distinct CHs with assigned
    # confidence level values to an empty base disk
    ensemble_map = np.where(~np.isnan(pre_processed_map), 0, np.nan)
    
    for distinct_ch, confidence in zip(map_data_by_ch, confidence_list):
        ensemble_map = np.where(
            ~np.isnan(distinct_ch), confidence, ensemble_map
        )
    return ensemble_map, map_data_by_ch, confidence_list, gradient_medians

In [ ]:
percent_of_peak_list = [80,80,90,100,100]
morph_radius_list = [13,17,15,13,17]


pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]
out = get_smooth_ensemble(
    pre_process_v0_1_he, percent_of_peak_list, morph_radius_list,
    even_confidence=False
)
ensemble_map, map_data_by_ch, confidence_list, gradient_medians = out
plot_detection.plot_ensemble(
    pre_process_v0_1_he, ensemble_map, map_data_by_ch,
    confidence_list, gradient_medians
)

v0.5b: Evenly Assigned Unipolarity, No threshold

In [ ]:
def get_unipol_ensemble(pre_processed_map, reprojected_mag_map,
                        percent_of_peak_list, morph_radius_list,
                        even_confidence=True):
    """Retrieve an ensemble of segmentations sorted by CH unipolarity.
    
    Args
        pre_processed_map: Sunpy map object to segment
        reprojected_mag_map: Sunpy map object of magnetogram reprojected
            to align with the ensemble map
        percent_of_peak_list: list of float percentage values
            at which to take threshold
        morph_radius_list: list of int pixel number for radius of disk 
            structuring element in morphological operations
        even_confidence: boolean to specify confidence assignment
            True to assign confidence by even ranking in (0,100]%
            False to assign confidence as 100% for unipolarity of 0
                and 0% for unipolarity of 1
    Returns
        Ensemble greyscale coronal holes mask sorted by unipolarity.
        List of coronal holes masks.
        List of confidence levels in mask layers.
    """
    pre_processed_map_data = np.flipud(pre_processed_map.data)
    
    # Create global segmentations for varied design variable combinations
    ch_masks = [
        detect.get_ch_mask(pre_processed_map_data, percent_of_peak, morph_radius)
        for percent_of_peak, morph_radius
        in zip(percent_of_peak_list, morph_radius_list)
    ]
    
    # List to be extended by masks for distinct CHs from all segmentations
    masks_by_ch = []
    
    ones_array = np.ones_like(pre_processed_map_data)
    
    for ch_mask in ch_masks:
        masks_by_ch.extend(
            detect.get_map_data_by_ch(ones_array, ch_mask)
        )
    
    num_ch = len(masks_by_ch)
    
    # Compute constant area per square pixel once for all CHs
    A_per_square_px = detect.get_A_per_square_px(pre_processed_map)
    
    # List to hold unipolarity for distinct CHs from all segmentations
    unipolarity_by_ch = []
    
    for ch_label in range(num_ch):
        distinct_ch_mask = masks_by_ch[ch_label]
        
        # Not flipping works right
        distinct_ch_map = sunpy.map.Map(
            distinct_ch_mask, pre_processed_map.meta
        )
        # ax = fig.add_subplot(num_rows, num_cols, ch_label + 1, projection=pre_processed_map)
        # distinct_ch_map.plot(cmap='magma')
        
        # fake_mag_map_data = np.where(~np.isnan(np.flipud(distinct_ch_mask)), 25,
        #                              reprojected_mag_map.data)
        # fake_mag_map = sunpy.map.Map(fake_mag_map_data, reprojected_mag_map.meta)
        # outcomes = get_outcomes(
        #     distinct_ch_map, fake_mag_map, A_per_square_px
        # )
        
        outcome_dict = detect.get_outcomes(
            distinct_ch_map, pre_processed_map_data, reprojected_mag_map,
            A_per_square_px
        )
        unipolarity_by_ch.append(outcome_dict['unipolarity'])
    
    # Sort unipolarities from greatest to least
    sorted_idxs = np.argsort(unipolarity_by_ch)
    
    # Sort candidate CHs from greatest to least gradient median
    masks_by_ch = [masks_by_ch[i] for i in sorted_idxs]
    unipolarity_by_ch = [unipolarity_by_ch[i] for i in sorted_idxs]
    
    # Assign confidence by direct ranking or by unipolarity value
    if even_confidence:
        confidence_list = [(c + 1)*100/num_ch
                           for c in range(num_ch)]
    else:
        confidence_list = [100 - unipolarity*100
                           for unipolarity in unipolarity_by_ch]

    # Construct ensemble map by adding distinct CHs with assigned
    # confidence level values to an empty base disk    
    ensemble_map_data = np.where(
        ~np.isnan(pre_processed_map_data), 0, np.nan
    )
    for distinct_ch, confidence in zip(masks_by_ch, confidence_list):
        ensemble_map_data = np.where(
            ~np.isnan(distinct_ch), confidence, ensemble_map_data
        )
    return ensemble_map_data, masks_by_ch, confidence_list, unipolarity_by_ch

In [ ]:
percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [18,20, 16, 16,20]


pre_process_v0_4_he_map_data = detect.pre_process_v0_4(he_map_data)
pre_process_v0_4_he_map = sunpy.map.Map(
    np.flipud(pre_process_v0_4_he), he_map.meta
)

out = get_unipol_ensemble(
    pre_process_v0_4_he_map, reprojected_mag_map,
    percent_of_peak_list, morph_radius_list,
    even_confidence=True
)
ensemble_map_data, map_data_by_ch, confidence_list, unipolarity_by_ch = out

plot_detection.plot_ensemble(
    pre_process_v0_4_he_map_data, ensemble_map_data, map_data_by_ch,
    confidence_list, unipolarity_by_ch
)

v0.5c: EUV Ratio, v0.4

In [ ]:
percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [18,20, 16, 16,20]


ratio_fits_file = f'{RATIO_SAVE_DIR}He{he_date_str}_EUV{euv_date_str}.fits'
raw_ratio = prepare_data.get_image_from_fits(ratio_fits_file)

pre_process_v0_4_ratio_map_data = detect.pre_process_v0_4(raw_ratio)

out = detect.get_ensemble_v0_3(
    pre_process_v0_4_ratio_map_data, percent_of_peak_list, morph_radius_list
)
ensemble_map, map_data_by_ch, confidence_list, gradient_medians = out
plot_detection.plot_ensemble(
    pre_process_v0_4_ratio_map_data, ensemble_map, map_data_by_ch,
    confidence_list, gradient_medians
)

# Single Map Outcomes

Requires single map extraction from single map data section

## Calc Verification

#### He I Smoothness

In [ ]:
def get_ch_band_widths(map_data_by_ch):
    """Retrieve a list of 5th to 95th percentile band widths for each
    detected CH.
    
    Args
        map_data_by_ch: list of isolated CH images from a segmentation
    """
    percentiles = [5, 95]
    bound_list = [np.percentile(map_data[~np.isnan(map_data)], percentiles)
                  for map_data in map_data_by_ch]
    
    hole_band_widths = [bounds[1] - bounds[0]
                        for bounds in bound_list]
    return hole_band_widths


def get_ch_lower_tail_widths(map_data_by_ch):
    """Retrieve a list of lower tail widths for each detected CH.
    
    Args
        map_data_by_ch: list of isolated CH images from a segmentation
    """
    filt_map_data_by_ch = [map_data[~np.isnan(map_data)]
                         for map_data in map_data_by_ch]
    
    # List of the 1st percentile brightness value of each CH
    first_percentile_list = [np.percentile(map_data, 1)
                             for map_data in filt_map_data_by_ch]
        
    # List of peak count of each CH
    peak_counts_value_list = [
        detect.get_peak_counts_loc(map_data, bins_as_percent=False)
        for map_data in filt_map_data_by_ch
    ]

    # List of lower tail widths of each CH
    ch_lower_tail_width_list = [
        peak_count - first_percentile
        for peak_count, first_percentile 
        in zip(peak_counts_value_list, first_percentile_list)]
    
    return ch_lower_tail_width_list


def plot_sorted_ch_hists(array, ch_mask, apply_gradient, hist_stat,
                         descend_sort=False):
    """Plot segmented CH histograms sorted by histogram statistics.
    
    Args
        array: image to process
        ch_mask: binary coronal holes mask
        apply_gradient: boolean to specify taking spatial gradient of image
        hist_stat: str to specify histogram sorting statistic
            'median', 'width', 'tail_width'
        descend_sort: boolean to specify sorting CHs from greatest to least
            statistic
    """
    # Masked array of candidate CHs
    masked_candidates = detect.get_masked_candidates(array, ch_mask)
    if apply_gradient:
        masked_candidates = filters.sobel(masked_candidates)
    
    # Isolated images of detected CHs
    map_data_by_ch = detect.get_map_data_by_ch(
        masked_candidates, ch_mask
    )
    num_ch = len(map_data_by_ch)
    
    # Compute statistics for each CH
    medians = [np.nanmedian(map_data) for map_data in map_data_by_ch]
    ch_band_widths = get_ch_band_widths(map_data_by_ch)
    
    # Histogram x limit bounds
    hist_xlim_min = np.mean(medians)
    if not apply_gradient:
        hist_xlim_min = hist_xlim_min - 2*np.max(ch_band_widths)
    hist_xlim_max = np.mean(medians) + 2*np.max(ch_band_widths)
    
    # Obtain indices of candidates sorted by specifed mode
    if hist_stat == 'median':
        sorted_candidate_idxs = np.argsort(medians)
        titles = [f'Median: {median:.2f}'
                  for median in medians]
    elif hist_stat == 'width':
        sorted_candidate_idxs = np.argsort(ch_band_widths)
        titles = [f'90% Band Width: {ch_band_width:.1f}'
                  for ch_band_width in ch_band_widths]
    elif hist_stat == 'tail_width':
        ch_lower_tail_width_list = get_ch_lower_tail_widths(
            map_data_by_ch
        )
        sorted_candidate_idxs = np.argsort(ch_lower_tail_width_list)
        titles = [f'1% to Peak Width: {ch_lower_tail_width:.1f}'
                  for ch_lower_tail_width in ch_lower_tail_width_list]
    
    if descend_sort:
        sorted_candidate_idxs = np.flip(sorted_candidate_idxs)

    for r in range(int(np.ceil(num_ch/2))):
        fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(60, 10))
        ax = axes.ravel()
        
        for c in range(2):
            i = 2*r + c
            ax_i = 3*c
            if i + 1 > num_ch:
                return
            
            # Retrieve isolated CH image and contour
            ch_num = sorted_candidate_idxs[i]
            ch_im = map_data_by_ch[ch_num]
            ch_contour = np.where(~np.isnan(ch_im), 1, 0)

            # Zoom in on an isolated CH
            y, x = np.where(~np.isnan(ch_im))
            ch_zoom = ch_im[np.min(y) - 10:np.max(y) + 10,
                             np.min(x) - 10:np.max(x) + 10]
                
            hist, edges = detect.get_hist(
                ch_zoom[~np.isnan(ch_zoom)], bins_as_percent=False, n=200
            )
            
            ax[ax_i].set_title(f'Hole {ch_num + 1}', fontsize=32)
            ax[ax_i].imshow(array, cmap='gray', vmin=-100, vmax=100)
            ax[ax_i].contour(ch_contour, cmap='plasma')
            
            if apply_gradient:
                cmap = plt.cm.viridis
            else:
                cmap = plt.cm.magma

            ax[ax_i + 1].imshow(ch_zoom, cmap)

            ax[ax_i + 2].set_title(titles[ch_num], fontsize=32)
            ax[ax_i + 2].bar(edges[0:-1], hist)
            ax[ax_i + 2].set_xlim([hist_xlim_min, hist_xlim_max])

v0.3

In [ ]:
# Requires single mask ensemble map
ch_mask = np.where(ensemble_map_data > 0, 1, 0)
plot_sorted_ch_hists(
    he_map_data, ch_mask, apply_gradient=True,
    hist_stat='median'
)

#### Alternate Statistics

In [ ]:
# Plot all CHs and histograms for a single date.
# Display ranked maps for all dates.
def get_ranked_map(array, ch_mask, apply_gradient, hist_stat,
                   ascend_sort=True):
    """Retrieve a map of CHs from a single segmentation as ranked by a
    histogram statistic.
    
    Args
        array: image to process
        ch_mask: binary coronal holes mask
        apply_gradient: boolean to specify taking spatial gradient of image
        hist_stat: str to specify histogram sorting statistic
            'median', 'width', 'tail_width'
        ascend_sort: boolean to specify sorting CHs from least to greatest
            statistic
    Returns
        List of isolated CH images from a segmentation.
    """
    # Masked array of candidate CHs
    masked_candidates = detect.get_masked_candidates(array, ch_mask)
    if apply_gradient:
        masked_candidates = filters.sobel(masked_candidates)
    
    # Isolated images of detected CHs
    map_data_by_ch = detect.get_map_data_by_ch(
        masked_candidates, ch_mask
    )
    num_ch = len(map_data_by_ch)
    
    # Rank candidates by histogram statistic
    if hist_stat == 'median':
        medians = [np.nanmedian(map_data) for map_data in map_data_by_ch]
        sorted_candidate_idxs = np.argsort(medians)
    elif hist_stat == 'width':
        ch_band_widths = get_ch_band_widths(map_data_by_ch)
        sorted_candidate_idxs = np.argsort(ch_band_widths)
    elif hist_stat == 'tail_width':
        ch_lower_tail_width_list = get_ch_lower_tail_widths(
            map_data_by_ch
        )
        sorted_candidate_idxs = np.argsort(ch_lower_tail_width_list)
    
    if ascend_sort:
        sorted_candidate_idxs = np.flip(sorted_candidate_idxs)
    
    map_data_by_ch = np.array(map_data_by_ch)
    ranked_ch_ims = map_data_by_ch[sorted_candidate_idxs]
    
    ranked_map = np.where(
        ~np.isnan(array), 0, np.nan
    )
    for isolated_ch_im, ch_num in zip(ranked_ch_ims, range(num_ch)):
        ranked_map = np.where(
            ~np.isnan(isolated_ch_im), (ch_num + 1)*100/num_ch, ranked_map
        )
    return ranked_map


def plot_ensemble_comparison(he_map, ensemble_map, euv_map):
    fig = plt.figure(figsize=(18, 5))
    
    # Plot He observation
    ax = fig.add_subplot(1, 3, 1, projection=he_map)
    he_map.plot(axes=ax, vmin=-100, vmax=100)
    
    # Plot ensemble map with overlayed neutral lines
    ax = fig.add_subplot(1, 3, 2, projection=he_map)
    ensemble_map.plot(axes=ax, title='', cmap='magma')
    
    # Plot EUV observation
    ax = fig.add_subplot(1, 3, 3, projection=euv_map)
    euv_map.plot(axes=ax)

Brightness Width

In [ ]:
percent_of_peak = 90
morph_radius = 15

pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]
ch_mask = detect.get_ch_mask(pre_process_v0_1_he, percent_of_peak, morph_radius)

plot_sorted_ch_hists(pre_process_v0_1_he, ch_mask,
                     apply_gradient=False, hist_stat='width')

Brightness Tail Width

In [ ]:
percent_of_peak = 90
morph_radius = 15

pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]
ch_mask = detect.get_ch_mask(pre_process_v0_1_he, percent_of_peak, morph_radius)

plot_sorted_ch_hists(pre_process_v0_1_he, ch_mask,
                     apply_gradient=False, hist_stat='tail_width')

Gradient Median

In [ ]:
percent_of_peak = 90
morph_radius = 15

pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]
ch_mask = detect.get_ch_mask(pre_process_v0_1_he, percent_of_peak, morph_radius)

plot_sorted_ch_hists(pre_process_v0_1_he, ch_mask,
                     apply_gradient=True, hist_stat='median')

Gradient Width

In [ ]:
percent_of_peak = 90
morph_radius = 15

pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]
ch_mask = detect.get_ch_mask(pre_process_v0_1_he, percent_of_peak, morph_radius)

plot_sorted_ch_hists(pre_process_v0_1_he, ch_mask,
                     apply_gradient=True, hist_stat='width')

Statistic Comparison

In [ ]:
# Compare ranking by different smoothness statistics for a single date
percent_of_peak = 90
morph_radius = 15
apply_gradient_list = [False, False, True, True]
hist_stat_list = ['width', 'tail_width', 'median', 'width']


pre_process_v0_1_he = detect.pre_process_v0_1(he_map_data)[0]

ch_mask = detect.get_ch_mask(pre_process_v0_1_he, percent_of_peak, morph_radius)
    
for apply_gradient, hist_stat in zip(apply_gradient_list, hist_stat_list):
    ranked_map_data = get_ranked_map(
        pre_process_v0_1_he, ch_mask, apply_gradient, hist_stat
    )
    
    ranked_map = sunpy.map.Map(np.flipud(ranked_map_data), he_map.meta)
    plot_ensemble_comparison(he_map, ranked_map, euv_map)

### Coordinates: Missing Helioprojective Keywords

In [ ]:
# Test without modifying when reading FITS file
with fits.open(he_fits_file) as hdu_list:
    header = hdu_list[-1].header
    num_data_arrays = header.get('NAXIS3')
    
    if not num_data_arrays:
        data = hdu_list[-1].data
    else:
        data = hdu_list[-1].data[0]

he_map = sunpy.map.Map(data, header)

Backup Header Heliocentric delt/pix To Helioprojective delt/pix

In [ ]:
# Cartesian distance change per pixel
hc_delta_coords = frames.Heliocentric(
    header['CDELT1A']*u.Unit(header['CUNIT1A']),
    header['CDELT2A']*u.Unit(header['CUNIT2A']),
    z=0*u.m, observer='earth', obstime=header['DATE-OBS']
)

In [ ]:
# Sunpy coordinate transform
hp_delta_coords = hc_delta_coords.transform_to(
    frames.Helioprojective(
        header['CRVAL1']*u.arcsec, header['CRVAL2']*u.arcsec,
        observer='earth', obstime=header['DATE-OBS']
    )
)
print(f'HP Scale: {hp_delta_coords.Tx.value:.11f}, {hp_delta_coords.Ty.value:.11f} arcsec/pix')

In [ ]:
# Thompson 2005 method section 4.1
earth_hp_coords = frames.Helioprojective(
    header['CRVAL1']*u.arcsec, header['CRVAL2']*u.arcsec,
    observer='earth', obstime=header['DATE-OBS'],
)
earth_header = sunpy.map.make_fitswcs_header(data, earth_hp_coords)

# Sun-Earth distance
Ds = earth_header['dsun_obs']*u.m

hp_delta_Tx = (hc_delta_coords.x/Ds).to(u.dimensionless_unscaled)
hp_delta_Tx = (hp_delta_Tx*u.rad).to(u.arcsec)
hp_delta_Ty = (hc_delta_coords.y/Ds).to(u.dimensionless_unscaled)
hp_delta_Ty = (hp_delta_Ty*u.rad).to(u.arcsec)

print(f'HP Scale: {hp_delta_Tx.value:.11f}, {hp_delta_Ty.value:.11f} arcsec/pix')

### Area

Ensemble Map Masking

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(121, projection=ensemble_map)
ensemble_map.plot(axes=ax)
ensemble_map.draw_grid(axes=ax)
for contour in ensemble_map.contour(35):
    ax.plot_coord(contour, color='white')
    
ax = fig.add_subplot(122, projection=ensemble_map)
ensemble_map.plot(axes=ax)
ensemble_map.draw_grid(axes=ax)
for contour in ensemble_map.contour(65):
    ax.plot_coord(contour, color='white')

In [ ]:
# Mask out detections below a CL threshold
fig = plt.figure(figsize=(12, 5))

ensemble_map.mask = (ensemble_map.data > 50)
ax = fig.add_subplot(121)
ax.imshow(np.flipud(ensemble_map.mask), cmap='magma')

ensemble_map.mask = (ensemble_map.data < 50)
ax = fig.add_subplot(122, projection=ensemble_map)
ensemble_map.plot(axes=ax)

ensemble_map.mask = None

Threshold Map by Confidence

In [ ]:
confidence_level = 20


confidence_map = np.where(ensemble_map_data >= confidence_level, ensemble_map_data, 0)
labeled_map, num_ch = ndimage.label(confidence_map)

confidence_map = np.where(np.isnan(ensemble_map_data), np.nan, confidence_map)
labeled_map = np.where(np.isnan(ensemble_map_data), np.nan, labeled_map)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 10))

ax[0].set_title(he_date_str)
ax[0].imshow(he, cmap=plt.cm.gray)

ax[1].imshow(ensemble_map_data, cmap=plt.cm.magma)
ax[2].imshow(labeled_map, cmap=plt.cm.bone)
print(num_ch)

#### Verification

In [ ]:
fake_ensemble_data = np.where(~np.isnan(np.flipud(ensemble_map.data)), 1, np.nan)
ensemble_map = sunpy.map.Map(fake_ensemble_data, pre_processed_map.meta)

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111, projection=ensemble_map)
ensemble_map.plot(axes=ax, title='')

Open Area: Compare official function with decomposed to investigate failed retrievals

In [ ]:
SOLAR_AREA = 4*np.pi *(1*u.solRad).to(u.Mm)**2

confidence_level = 0

if confidence_level <= 0:
    confidence_level = 1e-3

# ----------------------------------------------------------------------------
if ensemble_map.coordinate_frame.name == 'helioprojective':
    hp_delta_coord = frames.Helioprojective(
        ensemble_map.scale.axis1*u.pix,
        ensemble_map.scale.axis2*u.pix,
        observer='earth', obstime=ensemble_map.date
    )
    hc_delta_coord = hp_delta_coord.transform_to(
        frames.Heliocentric(observer='earth', obstime=ensemble_map.date)
    )
    A_per_square_px = np.abs(
        hc_delta_coord.x.to(u.Mm)*hc_delta_coord.y.to(u.Mm)
    )
elif ensemble_map.coordinate_frame.name == 'heliographic_stonyhurst':

    x_scale, y_scale = detect.get_hg_map_dist_scales(pre_processed_map)
    A_per_square_px = x_scale*y_scale
else:
    raise Exception(('Coordinate frame not recognized for obtaining '
                     'area per square pixel.'))

# ----------------------------------------------------------------------------
# Detected pixels at a confidence level
# Flip upside down to align Sunpy coordinates and Numpy indices
detected_px_coords = np.where(
    np.flipud(ensemble_map.data) >= confidence_level
)

if ensemble_map.coordinate_frame.name == 'helioprojective':
    
    # Convert detected pixels to Helioprojective Tx, Ty
    detected_hp_coords = ensemble_map.pixel_to_world(
        detected_px_coords[1]*u.pix, detected_px_coords[0]*u.pix
    )

    # Convert detected Helioprojective Tx, Ty to Heliographic lon, lat
    raw_detected_hg_coords = detected_hp_coords.transform_to(
        frames.HeliographicStonyhurst(obstime=ensemble_map.date)
    )

    # Remove pixels with failed conversion and longitudes outside (-90,90)
    failed_coord_idxs = np.where(
        np.isnan(raw_detected_hg_coords.lon) 
        | (np.abs(raw_detected_hg_coords.lon.to(u.deg).value) >= 90)
    )
    detected_hg_coords = np.delete(
        raw_detected_hg_coords, failed_coord_idxs
    )
    
elif ensemble_map.coordinate_frame.name == 'heliographic_stonyhurst':
    
    # Convert detected pixels to Heliographic lon, lat
    detected_hg_coords = ensemble_map.pixel_to_world(
        detected_px_coords[1]*u.pix, detected_px_coords[0]*u.pix
    )
    failed_coord_idxs = np.array([], dtype=np.int8)

# ----------------------------------------------------------------------------
if ensemble_map.coordinate_frame.name == 'helioprojective':
    
    # B-angle to subtract from latitude
    B0 = ensemble_map.observer_coordinate.lat

    pixel_lons = detected_hg_coords.lon.to(u.rad).value
    pixel_lats = detected_hg_coords.lat.to(u.rad).value - B0.to(u.rad).value
    pixel_areas = A_per_square_px/(np.cos(pixel_lons)*np.cos(pixel_lats))

elif ensemble_map.coordinate_frame.name == 'heliographic_stonyhurst':
    pixel_areas = np.ones(detected_hg_coords.shape)*A_per_square_px

# Sum area detected in all pixels
area = np.sum(pixel_areas)
area_percent = area/SOLAR_AREA*100
area_percent

In [ ]:
detect.get_open_area(ensemble_map, confidence_level=0)

In [ ]:
np.any(detected_hg_coords.lat > 0)

In [ ]:
from astropy.coordinates import CylindricalRepresentation

# WRONG BC RHO IS CYLINDRICAL AS OPPOSED TO SPHERICAL
# # Convert detected Helioprojective Sky Coords to Heliocentric radial rho, psi
# raw_pixel_hc_coords = pixel_hp_coords.transform_to(
#     frames.Heliocentric(observer='earth', obstime=obstime)
# )
# # raw_pixel_hc_coords = raw_pixel_hc_coords.represent_as(CylindricalRepresentation)
# pixel_hc_coords = raw_pixel_hc_coords[np.where(~np.isnan(raw_pixel_hc_coords.x))]

# # Compute area per pixel while accounting for foreshortening
# rho = np.sqrt(pixel_hc_coords.x**2 + pixel_hc_coords.y**2 + pixel_hc_coords.z**2)
# pixel_sol_rad_ratios = (pixel_hc_coords.rho/u.solRad).to(u.dimensionless_unscaled)
# pixel_angles_to_limb = pixel_sol_rad_ratios*np.pi/2 *u.rad
# pixel_areas = A_per_square_px/np.cos(pixel_angles_to_limb.value)

Failed Coordinate Conversion Points

In [ ]:
if np.any(failed_coord_idxs):
    failed_hp_coords = detected_hp_coords[failed_coord_idxs]
    failed_pixel_pairs = ensemble_map.world_to_pixel(failed_hp_coords)
else:
    print('No points matched condition')

ensemble_map.mask = None

fig = plt.figure(figsize=(12, 5))
failed_point_color = '#1ed950'

ax = fig.add_subplot(121)
ax.imshow(ensemble_map.data, cmap='magma')
if np.any(failed_coord_idxs):
    ax.scatter(failed_pixel_pairs.x.value, failed_pixel_pairs.y.value,
               color=failed_point_color)

ax.invert_yaxis()

ax = fig.add_subplot(122, projection=ensemble_map)
ensemble_map.plot(axes=ax)
ensemble_map.draw_grid(axes=ax)
for contour in ensemble_map.contour(confidence_level):
    ax.plot_coord(contour, color='white')

if np.any(failed_coord_idxs):
    ax.plot_coord(failed_hp_coords, 'o', color=failed_point_color)

In [ ]:
# Heliographic pixels with too large longitude
large_lon_pixel_hg_coords = raw_detected_hg_coords[
    np.where(~np.isnan(raw_detected_hg_coords.lon)
             & (raw_detected_hg_coords.lon.to(u.deg).value >= 90))
]
large_lon_pixel_hg_coords
# large_lon_pixel_hg_coords[np.argsort(large_lon_pixel_hg_coords.lon.to(u.deg).value)]

In [ ]:
# All failed conversion pixels
pixel_hg_coords = raw_detected_hg_coords[failed_detect_idxs]
pixel_hg_coords

# HG
# nan_idx = np.argmax(pixel_hg_coords.lon.to(u.deg).value)

#### Errors

Correct B-Angle

In [ ]:
# EPH_B0 keyword: [deg] Disk center solar latitude at DATE-AVG
# Yields a lat offset in HG Stonyhurst coordinates
he_map.center
he_map.reference_coordinate
he_map.center.observer.lat

In [ ]:
# Earth observer HGLT_OBS keyword
earth_hp_coords = frames.Helioprojective(
    header['CRVAL1']*u.arcsec, header['CRVAL2']*u.arcsec,
    observer='earth', obstime=header['DATE-OBS'],
)
earth_header = sunpy.map.make_fitswcs_header(data, earth_hp_coords)
earth_header['HGLT_OBS']

Limb Size Correction

In [ ]:
stonyhurst_frame = frames.HeliographicStonyhurst(obstime=he_map.date)

num_points = 100
lon_value = -50 * u.deg
lat_value = 0 * u.deg
constant_lon = SkyCoord(lon_value, np.linspace(-90, 90, num_points) * u.deg,
                        frame=stonyhurst_frame)
constant_lat = SkyCoord(np.linspace(-90, 90, num_points) * u.deg, lat_value,
                        frame=stonyhurst_frame)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection=he_map)

# north = SkyCoord(0 * u.deg, 10 * u.deg, frame="heliographic_stonyhurst")
# offset_frame = NorthOffsetFrame(north=north)
# overlay = ax.get_coords_overlay(offset_frame)
# overlay[0].set_ticks(spacing=30. * u.deg)
# overlay.grid(ls='--', color='blue')

ax.plot_coord(constant_lon, color="lightblue")
ax.plot_coord(constant_lat, color="tomato")
# he_map.draw_grid(axes=ax, grid_spacing=10*u.deg)
he_map.draw_limb(axes=ax)
he_map.plot(axes=ax, vmin=-100, vmax=100)

In [ ]:
pixel_lons = detected_hg_coords.lon.to(u.rad).value
pixel_lons = detected_hg_coords.lon.to(u.rad).value
pixel_lats = detected_hg_coords.lat.to(u.rad).value - B0.to(u.rad).value
pixel_areas = A_per_square_px/(np.cos(pixel_lons)*np.cos(pixel_lats))
pixel_areas

In [ ]:
# Retrieve Off Limb Coords: 
# https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.utils.get_limb_coordinates.html

In [ ]:
LIMB_FACTOR = 1.5

# def get_nso_sunpy_map(fits_file):
#     """Retrieve a Sunpy map with a Helioprojective Cartesian
#     coordinate system and the first data array in a SOLIS VSM FITS file.
    
#     Args
#         fits_file: path to FITS file
#     Returns
#         Sunpy map object.
#     """
with fits.open(fits_file) as hdu_list:
    header = hdu_list[-1].header
    num_data_arrays = header.get('NAXIS3')
    
    if not num_data_arrays:
        data = hdu_list[-1].data
    else:
        data = hdu_list[-1].data[0]

# # Apply absolute value of coordinate change per pixel such that
# # Solar-X is positive
# header['CDELT1'] = abs(header['CDELT1'])

# Helioprojective Cartesian coordinates must have
# arcsec units for further processing. Warning messages
# will appear but the map will be produced successfully.
if (header['WCSNAME'] == 'Helioprojective-cartesian'
    and header['CUNIT1'] != 'arcsec'):
    pass
    
# Heliocentric Cartesian coordinates must have zero
# centered coordinates
if (header['WCSNAME'] == 'Heliocentric-cartesian (approximate)'
    and (header['CRVAL1'] != 0 or header['CRVAL2'] != 0)):
    print((f'Failed to convert {fits_file} into a Sunpy map.')
            + ('Coordinates were Heliocentric but were not ')
            + ('zero centered.'))
    pass
    
# Specify Earth-based observer for solar radius, distance to Sun,
# and Heliographic coordinates to avoid warning messages due to
# missing keywords
earth_hp_coords = frames.Helioprojective(
    header['CRVAL1']*u.arcsec, header['CRVAL2']*u.arcsec,
    observer='earth', obstime=header['DATE-OBS'],
)
earth_header = sunpy.map.make_fitswcs_header(data, earth_hp_coords)
for earth_coord_key in ['DSUN_OBS', 'HGLN_OBS', 'HGLT_OBS']:
    header[earth_coord_key] = earth_header[earth_coord_key]

# Enlarge solar radius by a factor to account for larger apparent solar
# limb in He I observations
header['RSUN_REF'] = (100 + LIMB_FACTOR)/100 * earth_header['RSUN_REF']

# Change primary World Coordinate System from Heliocentric Cartesian
# to Helioprojective Cartesian for Sunpy to create map
if header['WCSNAME'] == 'Heliocentric-cartesian (approximate)':
    
    # Cartesian coordinate units
    coord_u1 = u.Unit(header['CUNIT1'])
    coord_u2 = u.Unit(header['CUNIT2'])
    
    # Convert center pixel coordinates from distance to angle
    hc_coords = frames.Heliocentric(
        header['CRVAL1']*coord_u1,
        header['CRVAL2']*coord_u2, z=0*u.m,
        observer='earth', obstime=header['DATE-OBS']
    )
    hp_coords = hc_coords.transform_to(earth_hp_coords)
    header['CRVAL1'] = hp_coords.Tx.value
    header['CRVAL2'] = hp_coords.Ty.value
    
    # Convert change per pixel from distance to angle
    hc_delta_coords = frames.Heliocentric(
        header['CDELT1']*coord_u1,
        header['CDELT2']*coord_u2, z=0*u.m,
        observer='earth', obstime=header['DATE-OBS']
    )
    hp_delta_coords = hc_delta_coords.transform_to(earth_hp_coords)
    header['CDELT1'] = hp_delta_coords.Tx.value
    header['CDELT2'] = hp_delta_coords.Ty.value
    
    # Modify keywords
    header['WCSNAME'] = 'Helioprojective-cartesian'
    header['CTYPE1'] = 'HPLN-TAN'
    header['CTYPE2'] = 'HPLT-TAN'
    header['CUNIT1'] = 'arcsec'
    header['CUNIT2'] = 'arcsec'

    # Remove error causing keywords indicate presence of
    # coordinate transformation
    header.pop('PC1_1')
    header.pop('PC2_2')
        
he_map = sunpy.map.Map(data, header)

In [ ]:
(he_map.rsun_meters/u.solRad).to(u.dimensionless_unscaled)

### Magnetic Data

Magnetogram Masking

In [ ]:
fig = plt.figure(figsize=(12, 5))

ax = fig.add_subplot(121, projection=reprojected_mag_map)
reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50)
for contour in ensemble_map.contour(0):
    ax.plot_coord(contour, color='yellow')

# Mask out data
reprojected_mag_map.mask = (ensemble_map.data < 1)
ax = fig.add_subplot(122, projection=reprojected_mag_map)
reprojected_mag_map.plot(axes=ax, vmin=-0.1, vmax=0.1, cmap='coolwarm')

reprojected_mag_map.mask = None
# Red: positive

#### Verification

Open Flux: Compare official function with decomposed to investigate failed retrievals

In [ ]:
open_flux = detect.get_unsigned_open_flux(
    ensemble_map, reprojected_mag_map, confidence_level=0
)
f'{open_flux:.7e} Wb'

In [ ]:
confidence_level = 0

if confidence_level <= 0:
    confidence_level = 1e-3
    
A_per_square_px = detect.get_A_per_square_px(ensemble_map)
        
detected_hg_coords, failed_coord_idxs = detect.get_detected_hg_coords(
    ensemble_map, confidence_level
)

pixel_areas = detect.get_pixel_areas(
    ensemble_map, A_per_square_px, detected_hg_coords
)

# Magnetic field strength per detected pixel
# Flip upside down to align Sunpy coordinates and Numpy indices
detected_idxs = np.where(np.flipud(ensemble_map.data) >= confidence_level)
pixel_B_LOS = reprojected_mag_map.data[detected_idxs]*u.G

# Remove pixels with failed coordinate conversion
pixel_B_LOS = np.delete(pixel_B_LOS, failed_coord_idxs)

# Remove pixels with failed magnetic data retrieval
failed_mag_idxs = np.where(np.isnan(pixel_B_LOS))
pixel_B_LOS = np.delete(pixel_B_LOS, failed_mag_idxs)


pixel_areas = np.delete(pixel_areas, failed_mag_idxs)
    
unsigned_open_flux = np.sum(np.abs(pixel_B_LOS)*pixel_areas).to(u.Wb)

unsigned_open_flux

Failed Magnetic Retrieval Points

In [ ]:
if np.any(failed_mag_idxs):
    failed_hp_coords = detected_hp_coords[failed_mag_idxs]
    failed_pixel_pairs = ensemble_map.world_to_pixel(failed_hp_coords)
else:
    print('No points matched condition')

fig = plt.figure(figsize=(12, 5))
failed_point_color = '#1ed950'

ax = fig.add_subplot(121)
ax.imshow(reprojected_mag_map.data, vmin=-50, vmax=50, cmap='gray')
if np.any(failed_mag_idxs):
    ax.scatter(failed_pixel_pairs.x.value, failed_pixel_pairs.y.value,
               color=failed_point_color)

ax.invert_yaxis()

ax = fig.add_subplot(122, projection=he_map)
reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50)
reprojected_mag_map.draw_grid(axes=ax)
for contour in ensemble_map.contour(confidence_level):
    ax.plot_coord(contour, color='yellow')

if np.any(failed_mag_idxs):
    ax.plot_coord(failed_hp_coords, 'o', color=failed_point_color)

### Properties Per CH

#### Test Magnetic

In [ ]:
# Set uniform value to detected regions to verify perfect unipolarity observed
fake_mag_map_data = np.where(ensemble_map.data > 1, 25, reprojected_mag_map.data)
fake_mag_map = sunpy.map.Map(fake_mag_map_data, reprojected_mag_map.meta)
fake_mag_map.plot(vmin=-50, vmax=50)

In [ ]:
confidence_level = 0

outcome_by_ch_dict = detect.get_outcomes_by_ch(
    ensemble_map, he_map_data, fake_mag_map, confidence_level
)

# Mask of detected CHs at the given confidence level
confidence_ch_mask = np.where(
    ensemble_map.data >= confidence_level, ensemble_map.data, 0
)

# List of ensemble map data for distinct CHs
ensemble_map_data_by_ch = detect.get_map_data_by_ch(
    ensemble_map.data, confidence_ch_mask
)

[print(f'{unipolarity:.4f}', end='\t')
 for unipolarity in outcome_by_ch_dict['unipolarity']]
print()

Verify vs Global Properties

In [ ]:
SOLAR_AREA = 4*np.pi*(1*u.solRad).to(u.Mm)**2

summed_area = np.sum(outcome_by_ch_dict['area'])*u.Mm**2 /SOLAR_AREA*100
global_area = detect.get_open_area(ensemble_map, confidence_level=0)[0]

f'Summed: {summed_area:.6f} % | Global: {global_area:.6f} %'

In [ ]:
summed_flux = np.sum(outcome_by_ch_dict['unsigned_flux'])
global_flux = detect.get_unsigned_open_flux(
    ensemble_map, reprojected_mag_map, confidence_level=0
)
f'{summed_flux:.6e} Wb | Global: {global_flux:.6e} Wb'

#### Calculate on CHs in Confidence Range

In [ ]:
confidence_range = [0,100]
# confidence_range = [50,90]
thresh_ensemble_map_data = np.where(
    np.all([ensemble_map.data >= confidence_range[0],
            ensemble_map.data <= confidence_range[1]], axis=0),
    ensemble_map.data, 0
)

# Remove leftover edges
large_obj_mask = morphology.remove_small_objects(
    np.where(thresh_ensemble_map_data > 0, True, False), min_size=5000
)
thresh_ensemble_map_data[~large_obj_mask] = 0

# Remove off-disk
thresh_ensemble_map_data[np.isnan(ensemble_map.data)] = np.nan

thresh_ensemble_map = sunpy.map.Map(
    thresh_ensemble_map_data, ensemble_map.meta
)
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=thresh_ensemble_map)
thresh_ensemble_map.plot(axes=ax, title='', cmap='magma', vmin=0, vmax=100)

In [ ]:
confidence_level = 0

# Compute outcomes by CH and sort from greatest to least
outcome_by_ch_dict = detect.get_outcomes_by_ch(
    thresh_ensemble_map, he_map_data, reprojected_mag_map, confidence_level
)
sorted_idxs = np.flip(np.argsort(outcome_by_ch_dict['unipolarity']))

sorted_outcome_by_ch_dict = {}
for key, outcome_by_ch in zip(outcome_by_ch_dict, outcome_by_ch_dict.values()):
    sorted_outcome_by_ch_dict[key] = [outcome_by_ch[i] for i in sorted_idxs]

    
if confidence_level <= 0:
    confidence_level = 1e-3

# Mask of detected CHs at the given confidence level
confidence_ch_mask = np.where(
    thresh_ensemble_map.data >= confidence_level, 1, 0
)

# List of ensemble map data for distinct CHs
ensemble_map_data_by_ch = detect.get_map_data_by_ch(
    thresh_ensemble_map.data, confidence_ch_mask
)
ensemble_map_data_by_ch = [ensemble_map_data_by_ch[i] for i in sorted_idxs]


[print(f'{area:.1e} Mm^2', end='\t')
 for area in sorted_outcome_by_ch_dict['area']]
print()
[print(f'Lat: {cm_lat:.1f} deg', end='\t')
 for cm_lat in sorted_outcome_by_ch_dict['cm_lat']]
print()
[print(f'Lon: {cm_lon:.1f} deg', end='\t')
 for cm_lon in sorted_outcome_by_ch_dict['cm_lon']]
print()
[print(f'{signed_flux:.4e} Mx', end='\t')
 for signed_flux in sorted_outcome_by_ch_dict['signed_flux']]
print()
[print(f'Skew: {mag_skew:.4f}', end='\t')
 for mag_skew in sorted_outcome_by_ch_dict['mag_skew']]
print()
[print(f'U: {unipolarity:.4f}', end='\t')
 for unipolarity in sorted_outcome_by_ch_dict['unipolarity']]
print()
[print(f'Grad: {grad_median:.4f}', end='\t')
 for grad_median in sorted_outcome_by_ch_dict['grad_median']]
print()

In [ ]:
# ch_idx = np.argmax(sorted_outcome_by_ch_dict['unipolarity'])
# ch_idx = np.argmax(sorted_outcome_by_ch_dict['area'])
# ch_idx = np.argmax(sorted_outcome_by_ch_dict['grad_median'])
# ch_idx = np.argmax(np.abs(sorted_outcome_by_ch_dict['mag_skew']))
# ch_idx = np.argmin(np.abs(sorted_outcome_by_ch_dict['cm_lon']))
ch_idx = 3

# signed_flux = sorted_outcome_by_ch_dict['signed_flux'][ch_idx]
# mag_skew = sorted_outcome_by_ch_dict['mag_skew'][ch_idx]
unipolarity = sorted_outcome_by_ch_dict['unipolarity'][ch_idx]
area = sorted_outcome_by_ch_dict['area'][ch_idx]
cm_lon = sorted_outcome_by_ch_dict['cm_lon'][ch_idx]
cm_lat = sorted_outcome_by_ch_dict['cm_lat'][ch_idx]
# grad_median = sorted_outcome_by_ch_dict['grad_median'][ch_idx]

# Calculate foreshortening factor. 0: 90deg inc angle, 1: 0deg inc angle
B0 = ensemble_map.observer_coordinate.lat.value
foreshort_factor = np.cos(np.deg2rad(cm_lon))*np.cos(np.deg2rad(cm_lat - B0))

# title = f'{area:.2e} Mm^2 | {signed_flux:.2e} Mx | {mag_skew:.2f} Skew'
# title = f'{signed_flux:.2e} Mx | {unipolarity:.2f} Unipolarity | {mag_skew:.2f} Skew'
title = rf'{unipolarity:.2f} Unipolarity | {area:.1e} $Mm^2$ | {foreshort_factor:.2f} Foreshort Factor'
# title = f'{unipolarity:.2f} Unipolarity | {grad_median:.2f} Gradient Median'

A_per_square_px = detect.get_A_per_square_px(ensemble_map)        


selected_ch_map_data = ensemble_map_data_by_ch[ch_idx]
selected_ch_map_data = np.where(np.isnan(selected_ch_map_data), -100,
                                selected_ch_map_data)
selected_ch_map = sunpy.map.Map(selected_ch_map_data, he_map.meta)

fig = plt.figure(figsize=(6, 6))
fig.suptitle(he_date_str)

ax = fig.add_subplot(projection=ensemble_map)
ensemble_map.plot(axes=ax, title=title)
ensemble_map.draw_grid(axes=ax)
for contour in selected_ch_map.contour(0):
    ax.plot_coord(contour, color='g')

In [ ]:
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111, projection=reprojected_mag_map)
reprojected_mag_map.plot(axes=ax, vmin=-50, vmax=50)

Rank each CH

In [ ]:
def get_ranked_mag_map(ensemble_map, confidence_level, sorted_idxs):
    """Retrieve a map of CHs from a single segmentation as ranked by a
    histogram statistic.
    
    Args
        ensemble_map_data: map data which was segmented
        ch_mask: binary coronal holes mask
    Returns
        
    """    
    # Mask of detected CHs at the given confidence level
    confidence_ch_mask = np.where(
        ensemble_map.data >= confidence_level, ensemble_map.data, 0
    )

    # List of ensemble map data for distinct CHs
    ensemble_map_data_by_ch = detect.get_map_data_by_ch(
        ensemble_map.data, confidence_ch_mask
    )
    num_ch = len(ensemble_map_data_by_ch)
    ensemble_map_data_by_ch = [ensemble_map_data_by_ch[i]
                               for i in np.flip(sorted_idxs)]
    
    ranked_map_data = np.where(
        ~np.isnan(ensemble_map.data), 0, np.nan
    )
    for map_data, ch_num in zip(ensemble_map_data_by_ch, range(num_ch)):
        ranked_map_data = np.where(
            ~np.isnan(map_data), (ch_num + 1)*100/num_ch, ranked_map_data
        )
    
    ranked_map = sunpy.map.Map(ranked_map_data, he_map.meta)
    return ranked_map

In [ ]:
num_ch = len(ensemble_map_data_by_ch)

# image_list = [pre_processed_map_data for _ in range(num_ch)]

mag_data = np.flipud(reprojected_mag_map.data)
smooth_size = 0.05 *mag_data.shape[0]
smoothed_mag_data = np.where(np.isnan(mag_data), 0, mag_data)
smoothed_mag_data = ndimage.uniform_filter(
    smoothed_mag_data, smooth_size
)
mag_data = np.clip(
    np.where(~np.isnan(mag_data), smoothed_mag_data, np.nan), -2, 2
)
image_list = [mag_data for _ in range(num_ch)]

axes = plot_detection.plot_image_grid(image_list, num_cols=3, cmap='gray')

for ax, i, ch_data in zip(axes.values(), range(num_ch), ensemble_map_data_by_ch):
    mask = np.where(np.isnan(ch_data), 0, 1)
    
    ax.set_title((f'{sorted_outcome_by_ch_dict["unipolarity"][i]:.2f} Unipolarity | '
                  f'{sorted_outcome_by_ch_dict["mag_skew"][i]:.2f} Skew'))
    ax.contour(np.flipud(mask), cmap=plt.cm.plasma)

Save maps thresholded above a confidence level and then sorted by that outcome

In [ ]:
# NEUTRAL LINE COMPARISON NEEDS UPDATE
overwrite = False
confidence_level = 0

out_dir = DETECTION_IMAGE_DIR + 'Unipolarity_Rank/'

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    comparison_img_file = f'{out_dir}He{he_date_str}.jpg'
    if os.path.isfile(comparison_img_file) and not overwrite:
        print((f'EUV {euv_date_str} comparison already exists.'))
        continue
    
    # Extract He I observation
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    if not he_map:
        print(f'{he_date_str} He I observation extraction failed.')
        continue
    
    # Extract saved ensemble map array and convert to Sunpy map
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
    ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
    ensemble_map.plot_settings['cmap'] = colormaps['magma']

    # Extract saved processed magnetograms
    mag_date_str = prepare_data.get_nearest_date_str(
        MAG_DATE_LIST, selected_date_str=he_date_str
    )
    mag_fits_name = f'{ROTATED_MAG_SAVE_DIR}Mag{mag_date_str}_He{he_date_str}'
    reprojected_fits_file = f'{mag_fits_name}.fits'
    reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)

    # Compute outcomes by CH and sort from greatest to least
    # TODO: Update get_outcomes_by_ch call to dict format
    outcomes_by_ch = detect.get_outcomes_by_ch(
        ensemble_map, pre_processed_map, reprojected_mag_map, confidence_level
    )
    unipolarity_by_ch = outcomes_by_ch[4]
    sorted_idxs = np.argsort(unipolarity_by_ch)

    # Obtain ranked map
    ranked_map = get_ranked_mag_map(
        ensemble_map, confidence_level, sorted_idxs
    )
    ranked_map.plot_settings['cmap'] = colormaps['magma']

    euv_date_str = prepare_data.get_nearest_date_str(
        EUV_DATE_LIST, selected_date_str=he_date_str
    )

    fig = plt.figure(figsize=(18, 5))
    plot_detection.plot_he_neutral_lines_euv_comparison(
        fig, he_date_str, mag_date_str, euv_date_str, ranked_map
    )
    
    # Save plot
    plt.savefig(comparison_img_file)
    plt.close(fig)
    print(f'{euv_date_str} map comparison saved.')

Skewness Histogram in Single CH

In [ ]:
def plot_sym_log_hist(data_list, num_bins):
    # Symmetric log bins
    neg_outcomes = np.where(data_list < 0, np.abs(data_list), 0)
    bin_min = np.ceil(np.log10(np.max(neg_outcomes)))
    bin_max = np.ceil(np.log10(np.max(np.abs(data_list))))
    bins = np.hstack((-np.logspace(bin_min, 0, num_bins//2),
                    np.logspace(0, bin_max, num_bins//2)))

    fig = plt.figure(figsize=(10,6))
    ax = plt.subplot()
    ax.set_xscale('symlog')
    ax.hist(data_list, bins)
    
    return fig, ax

In [ ]:
data_list = sorted_outcome_by_ch_dict['pixel_signed_fluxes'][ch_idx]
fig, ax = plot_sym_log_hist(data_list, num_bins=500)
fig.suptitle(f'CH Index: {ch_idx}')
ax.set_title('Pixel Magnetic Flux Histogram')
ax.set_xlabel('Magnetic Flux (Wb)')
ax.set_ylim([0,3500])
f'Summed: {np.sum(data_list):.3e} | Pre-Computed {sorted_outcome_by_ch_dict["signed_flux"][ch_idx]:.3e}'

Line of Sight B

In [ ]:
confidence_level = 0

if confidence_level <= 0:
    confidence_level = 1e-3
    
percent_unipolar_by_ch = []
signed_B_by_ch = []
unsigned_B_by_ch = []
unipolarity_by_ch = []

# Thresholded array at the given confidence level
confidence_map_data = np.where(
    ensemble_map.data >= confidence_level, ensemble_map.data, 0
)

# Array with number labels per distinct CH and number of labels
labeled_map_data, num_labels = ndimage.label(confidence_map_data)
num_ch = num_labels - 1

# List of magnetic field data images for distinct CHs.
# Equivalent to obtain Helioprojective coordinates of pixels per CH
# and then obtaining magnetic data at HP coordinates
mag_map_data_by_ch = detect.get_map_data_by_ch(
    reprojected_mag_map.data, confidence_map_data
)

# # Obtain masked magnetic data per CH > Compute on magnetic data per CH
# mask_idxs_by_ch = [np.where(labeled_map_data == label)
#                    for label in range(1, num_labels)]
# mag_data_by_ch = [reprojected_mag_map.data[mask_idxs]
#                   for mask_idxs in mask_idxs_by_ch]

# List of masks for distinct CHs
masks_by_ch = [np.where(labeled_map_data == label, 1, 0)
               for label in range(1, num_labels)]

for ch_label in range(num_ch):
    mag_map_data = mag_map_data_by_ch[ch_label]
    mag_data = mag_map_data[~np.isnan(mag_map_data)]
    
    # Pixel percent unipolarity
    num_positive = np.count_nonzero(mag_data > 0)
    num_negative = np.count_nonzero(mag_data < 0)
    num_px = np.count_nonzero(mag_data)
    percent_unipolarity = max(num_positive, num_negative)*100/num_px
    percent_unipolar_by_ch.append(percent_unipolarity)
    
    # Signed average magnetic field
    signed_B = np.abs(np.mean(mag_data))
    signed_B_by_ch.append(signed_B)

    # Unsigned average magnetic field
    unsigned_B = np.mean(np.abs(mag_data))
    unsigned_B_by_ch.append(unsigned_B)
    
    # Unipolarity
    unipolarity = (unsigned_B - signed_B)/unsigned_B
    unipolarity_by_ch.append(unipolarity)


# Sort outcomes by CH from greatest to least
sorted_idxs = np.flip(np.argsort(percent_unipolar_by_ch))
percent_unipolar_by_ch.sort(reverse=True)

# Sort candidate CHs from greatest to least outcome median
masks_by_ch = [masks_by_ch[i] for i in sorted_idxs]
signed_B_by_ch = [signed_B_by_ch[i] for i in sorted_idxs]
unsigned_B_by_ch = [unsigned_B_by_ch[i] for i in sorted_idxs]
unipolarity_by_ch = [unipolarity_by_ch[i] for i in sorted_idxs]

In [ ]:
image_list = [pre_processed_map for _ in range(num_ch)]
axes = plot_detection.plot_image_grid(image_list, num_cols=3, cmap='gray')

for ax, i, mask in zip(axes.values(), range(num_ch), masks_by_ch):
    ax.set_title((f'{percent_unipolar_by_ch[i]:.1f}% Unipolar '
                  + f'| {unipolarity_by_ch[i]:.2f} Unipolarity'))
    ax.contour(np.flipud(mask), cmap=plt.cm.plasma)

### Boundary Complexity

In [ ]:
from lib.CDM.fracstat import *

def get_fractal_D(img, scale_range):
    # Compute fractal dimension
    
    scales, n_filled = box_counting(img, scale_range, f=1.1)[:2]
    if np.any(n_filled == 0):
        D = np.nan
    else:
        fit = power_law(scales, n_filled, scale_range)[0]
        D = -fit[0]
    
    return D
    

def plot_fractal_D(img, scale_range, title_var, contours=False):
    fig = plt.figure(figsize=(8, 10))

    ax = fig.add_subplot(211)
    ax.imshow(img, cmap='gray')
    if contours:
        ax.contour(img, cmap='gray')
    
    ax.set_title(title_var)
    
    # Plot full range
    img_row_num = img.shape[0]
    full_range = [min([5, scale_range[0]]), max(img_row_num, scale_range[1])]
    full_scales, full_n_filled = box_counting(img, scale_range=full_range, f=1.1)[:2]
    
    ax = fig.add_subplot(212)
    full_X = full_scales/img_row_num
    ax.loglog(full_X, full_n_filled, c='k')
    
    # Compute fractal dimension in specified range and plot selected range
    scales, n_filled = box_counting(img, scale_range, f=1.1)[:2]
    X = scales/img_row_num
    fit, cov = power_law(X, n_filled)
    predict = np.poly1d(fit)
    D = -fit[0]
    D_err = np.sqrt(cov[0,0])
    
    X_range = np.array(scale_range)/img_row_num
    ax.loglog(X_range, 10**predict(np.log10(X_range)), c='C0', linewidth=2)
    ax.vlines(X_range, ymin=0, ymax=1e6, linestyles='--', colors='k')
    
    ax.set_xlabel(r'$\epsilon$')
    ax.set_ylabel('Box Number Containing Boundary')
    ax.set_title(f'Fractal Dimension: {D:.3f} +/- {D_err:.4f}')
    ax.set_ylim([1,1e5])

    return fig

In [ ]:
contours = measure.find_contours(ch_mask_data)
ch_boundary_data = np.zeros(ch_mask_data.shape)

for contour in contours:
    contour = contour.astype(int)
    ch_boundary_data[contour[:,0], contour[:,1]] = 1

fig = plot_fractal_D(
    ch_boundary_data,
    scale_range=[10, 500],
    title_var=f'SE Disk Radius: {morph_radius_dist} Mm',
    contours=True
)
# plt.savefig(f'{OUTPUT_DIR}Fractal/{percent_of_peak}_{morph_radius_dist}_Boundary')

In [ ]:
get_fractal_D(ch_boundary_data, scale_range=[10, 500])

In [ ]:
fig = plot_fractal_D(
    ch_mask_data,
    scale_range=[10, 500],
    # scale_range=[10, np.min(ch_mask_data.shape)],
    title_var=f'SE Disk Radius: {morph_radius_dist} Mm'
)
plt.savefig(f'{OUTPUT_DIR}Fractal/{morph_radius_dist}_Mask')

## Outcome Comparison

#### Pre-Process Outcomes vs Method

In [ ]:
percent_of_peak_list = [80, 90, 100, 110]
area_percent_df_by_method_list = []
mad_by_thresh_by_method_list = []

for pre_process_save_dir in ['v0_1/', 'v0_4/']:
    pre_process_save_dir = out_dir + PREPROCESS_DIR + pre_process_save_dir
    
    area_percent_df = detect.get_thresh_outcome_time_series_dfs(
        HE_DATE_LIST, percent_of_peak_list, HE_DIR, pre_process_save_dir
    )[1]
    area_percent_df_by_method_list.append(area_percent_df)
    mad_by_thresh_by_method_list.append(
        detect.get_mad_by_confidences(area_percent_df, percent_of_peak_list)
    )

In [ ]:
x_ticks = np.arange(len(percent_of_peak_list))
threshold_label_list = [
    f'{thresh_level}% of Peak Threshold'
    for thresh_level in percent_of_peak_list
]

plt.figure(1, figsize=(8,6))

plt.bar(x_ticks - 0.2, mad_by_thresh_by_method_list[0], width=0.2, label='v0.3')
plt.bar(x_ticks, mad_by_thresh_by_method_list[1], width=0.2, label='Band Pass')
plt.bar(x_ticks + 0.2, mad_by_thresh_by_method_list[2], width=0.2, label='Rescaling')
plt.xticks(x_ticks, threshold_label_list, rotation=10)
plt.ylabel(f'MAD of Detected Area Percentage (%)')
plt.legend()

### Ensemble Outcomes vs Method

Compare outcomes between confidence levels and/or methods

In [ ]:
out_dir = DETECT_DIR + '_Outcome_Comparison/' + DATE_DIR
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

confidence_level_list = [0, 35, 65, 95]
# confidence_level_list = list(range(0,96,5))

# version_dirs = ['v0_3/', 'Band_Pass/', 'Rescale/', 'Rescale_Center/']
# version_dirs = ['v0_3/', 'Rescale/']
# version_dirs = ['v0_3/', 'Rescale/', 'v0_4/']
# version_dirs = ['v0_3/', 'v0_4/']
# version_dirs = ['v0_4_Single/', 'v0_4/']
# version_dirs = ['v0_4_Unipolar']
version_dirs = ['v0_1', 'v0_2', 'v0_3', 'v0_4', 'v0_5']
descript_list = version_dirs + [f'cl{cl}' for cl in confidence_level_list]

Plot Formatting

In [ ]:
cl_dx_list = np.arange(-0.3,0.31,0.2)
method_list = ['Bright & Coherent Mask', 'Ensemble', 'Smoothness',
               'Consistency', 'Unipolarity']

# cl_dx_list = np.arange(-0.9,0.91,0.2)
# method_list = ['Unipolarity']

# cl_dx_list = np.arange(0,1,0.05)
# method_list = ['Unipolarity']

# cl_dx_list = np.arange(-0.3,0.31,0.2)
# # method_list = ['v0.3', 'v0.3 Design + Band Pass', 'v0.3 Design + Rescale',
# #               'v0.3 Design + Rescale & Center']
# method_list = ['v0.1', 'v0.2', 'v0.3', 'v0.4']

# cl_dx_list = [-0.1, 0.1]
# # method_list = ['v0.3', 'v0.3 Design + Rescale']
# # method_list = ['v0.3', 'v0.4']
# method_list = ['v0.4 Single', 'v0.4 Ensemble']

# cl_dx_list = [-0.2, 0, 0.2]
# method_list = ['v0.3', 'v0.3 Design + Rescale', 'v0.4']

cmap = colormaps['viridis']
color_list = cmap(np.linspace(0, 0.75, len(confidence_level_list)))
# cmap = colormaps['plasma_r']
# color_list = cmap(np.linspace(0.25, 1, len(confidence_level_list)))

v0.2-v0.5 Compute Outcomes

In [ ]:
area_percent_df_by_method_list = []
autocorr_by_conf_by_method_list = []
mad_by_conf_by_method_list = []
norm_mad_by_conf_by_method_list = []


for version_dir in version_dirs:
    detection_save_dir = os.path.join(DETECT_DIR, version_dir, 'Saved_npy_Files/')
    
    outcome_time_series_dict = detect.get_outcome_time_series_dict(
        HE_DATE_LIST, confidence_level_list, detection_save_dir
    )
    area_percent_df_by_method_list.append(
        outcome_time_series_dict['area_percent']
    )
    
    autocorr_by_confidences = [
        outcome_time_series_dict['area'][cl].autocorr()
        for cl in confidence_level_list
    ]
    autocorr_by_conf_by_method_list.append(autocorr_by_confidences)
    out = detect.get_mad_by_confidences(
        outcome_time_series_dict['area'], confidence_level_list
    )
    mad_by_confidences, norm_mad_by_confidences = out
    mad_by_conf_by_method_list.append(mad_by_confidences)
    norm_mad_by_conf_by_method_list.append(norm_mad_by_confidences)
    print(f'Outcomes computed for {version_dir}')

descript_list = version_dirs + [f'cl{cl}' for cl in confidence_level_list]
autocorr_file = f'{out_dir}Autocorr_comp_{"_".join(descript_list)}.npy'
np.save(autocorr_file, np.array(autocorr_by_conf_by_method_list),
        allow_pickle=True)

MAD

In [ ]:
x_ticks = np.arange(len(confidence_level_list))
confidence_label_list = [
    f'{confidence_level}% Confidence'
    for confidence_level in confidence_level_list
]

plt.figure(1, figsize=(9,6))
for mad_by_confidences, cl_dx, method, color in zip(
    mad_by_conf_by_method_list, cl_dx_list, method_list, color_list):
    plt.bar(x_ticks + cl_dx, mad_by_confidences, width=0.2,
            label=method, color=color)

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Method Comparison')
plt.xticks(x_ticks, confidence_label_list, rotation=10)
plt.ylabel(f'MAD of Detected Area (Mm^2)')
plt.legend()

Normalized MAD

In [ ]:
x_ticks = np.arange(len(confidence_level_list))
confidence_label_list = [
    f'{confidence_level}% Confidence'
    for confidence_level in confidence_level_list
]

plt.figure(1, figsize=(9,6))
for norm_mad_by_confidences, cl_dx, method, color in zip(
    norm_mad_by_conf_by_method_list, cl_dx_list, method_list, color_list):
    plt.bar(x_ticks + cl_dx, norm_mad_by_confidences, width=0.2,
            label=method, color=color)
    
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Method Comparison')
plt.xticks(x_ticks, confidence_label_list, rotation=10)
plt.ylim([0, 50])
plt.ylabel(f'Normalized MAD of Detected Area (%)')
plt.legend()

## Time Plots

Extract Outcomes

In [ ]:
confidence_level_list = [1, 50, 75, 95]
num_ch_df, area_percent_df, px_percent_df = detect.get_outcome_time_series_dfs(
    HE_DATE_LIST[:5], confidence_level_list, DETECTION_SAVE_DIR
)

In [ ]:
outcome_df = px_percent_df

fig = plt.figure(figsize=(18, 5))
ax = fig.add_subplot(111)
plot_detection.plot_outcome_df_vs_time(
    ax, outcome_df, he_date_str, cmap='plasma', ylabel='Detected Pixel Percentage (%)'
)

In [ ]:
outcome_df = area_percent_df

fig = plt.figure(figsize=(18, 5))
ax = fig.add_subplot(111)
plot_detection.plot_outcome_df_vs_time(
    ax, outcome_df, he_date_str, cmap='bone', ylabel='Detected Area Percentage (%)'
)

#### Pre-Process

Histogram Moments vs Time

In [ ]:
hist_stat_list = []

for he_date_str in HE_DATE_LIST:
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map_data = np.load(pre_process_file, allow_pickle=True)[-1]
    
    peak_counts_val = detect.get_peak_counts_loc(
        pre_processed_map_data, bins_as_percent=False
    )
    hist_stat_list.append(
        [peak_counts_val, np.nanstd(pre_processed_map_data)]
    )

# Convert to dataframes
datetime_list = [datetime.strptime(he_date_str, DICT_DATE_STR_FORMAT)
                 for he_date_str in HE_DATE_LIST]
hist_df = pd.DataFrame(
    hist_stat_list, columns=['Peak', 'StDev'],
    index=datetime_list
)

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'Histogram_Moments/'
cmap = 'plasma'
ylabel = 'Histogram Moments'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    img_file = f'{out_dir}{he_date_str}.jpg'
    if os.path.isfile(img_file) and not overwrite:
        print((f'He {he_date_str} map already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    hist, edges = detect.get_hist(pre_processed_map,
                                         bins_as_percent=False)
    
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = np.load(ensemble_file, allow_pickle=True)[-1]
    
    fig = plt.figure(figsize=(18, 10))
    
    ax = fig.add_subplot(231)
    ax.set_title(he_date_str)
    ax.imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax = fig.add_subplot(232)
    ax.set_title('Semilog Histogram')
    ax.semilogy(edges[1:], hist)
    if 'Rescale' in DETECTION_VERSION_DIR:
        ax.set_xlim([-1.3, 1.1])
        ax.set_ylim([1E2, 5E4])
    else:
        ax.set_xlim([-110, 110])
        ax.set_ylim([1E1, 5E4])
    
    ax = fig.add_subplot(233)
    ax.imshow(ensemble_map, cmap=plt.cm.magma)
    
    ax = fig.add_subplot(2, 3, (4, 6))
    datetimes = hist_df.index
    ax.plot(hist_df['StDev'], label='Standard Deviation', linewidth=3)
    ax.plot(hist_df['Peak'], label='Mode', linewidth=3)
    
    # Vertical line for datetime indicator
    vline_datetime = datetime.strptime(he_date_str, DICT_DATE_STR_FORMAT)
    min_moment = min(hist_df.min())
    max_moment = max(hist_df.max())
    ax.vlines(x=[vline_datetime, vline_datetime], ymax=2*max_moment, ymin=0,
              colors='k', linestyles='dashed')
    
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ax.set_ylabel(ylabel)
    
    ax.set_xlim([datetimes[0], datetimes[-1]])
    if 'Rescale' in DETECTION_VERSION_DIR:
        ax.set_ylim([0.4, 0.8])
    else:
        ax.set_ylim([0.9*min_moment, 1.1*max_moment])
    
    ax.legend(reverse=True)

    plt.savefig(img_file)
    plt.close(fig)
    print(f'{he_date_str} map saved.')

Pre-Process Outcomes vs Time

In [ ]:
percent_of_peak_list = [80, 90, 100, 110]
num_ch_df, area_percent_df, area_df, px_percent_df = detect.get_thresh_outcome_time_series_dfs(
    HE_DATE_LIST, percent_of_peak_list, HE_DIR, PREPROCESS_SAVE_DIR
)

In [ ]:
overwrite = True
out_dir = DETECTION_IMAGE_DIR + 'Thresh_Area_Percentage/'
outcome_df = area_percent_df
cmap = 'plasma'
ylabel = 'Detected Area Percentage (%)'


if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for he_date_str in HE_DATE_LIST[24:25]:
    
    # Optionally overwrite existing files
    img_file = f'{out_dir}{he_date_str}.jpg'
    if os.path.isfile(img_file) and not overwrite:
        print((f'He {he_date_str} map already exists.'))
        continue
    
    pre_process_file = (PREPROCESS_SAVE_DIR + he_date_str
                        + '_pre_processed_map.npy')
    pre_processed_map = np.load(pre_process_file, allow_pickle=True)[-1]
    
    hist, edges = detect.get_hist(pre_processed_map,
                                         bins_as_percent=False)
    
    ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = np.load(ensemble_file, allow_pickle=True)[-1]
    
    fig = plt.figure(figsize=(18, 10))
    
    ax = fig.add_subplot(231)
    ax.set_title(he_date_str)
    ax.imshow(pre_processed_map, cmap=plt.cm.gray)
    
    ax = fig.add_subplot(232)
    ax.set_title('Semilog Histogram')
    ax.semilogy(edges[1:], hist)
    if 'Rescale/' in DETECTION_VERSION_DIR:
        ax.set_ylim([1E2, 5E4])
    else:
        ax.set_xlim([-110, 110])
        ax.set_ylim([1E1, 5E4])
    
    ax = fig.add_subplot(233)
    ax.imshow(ensemble_map, cmap=plt.cm.magma)
    
    ax = fig.add_subplot(2, 3, (4, 6))    
    plot_detection.plot_thresh_outcome_vs_time(
        ax, outcome_df, he_date_str, cmap, ylabel)

    plt.savefig(img_file)
    plt.close(fig)
    print(f'{he_date_str} map saved.')

## Contour Plots

### Evaluate Outcomes on Grid

In [ ]:
overwrite = False
output_dir = DETECTION_IMAGE_DIR + 'Outcome_Maps/'
save_file = f'{output_dir}{file_date_str}.npy'

# v0.5.1
p_start, p_step, p_num = (70, 10, 5)
r_start, r_step, r_num = (8, 4, 4)

# # vY
# p_start, p_step, p_num = (60, 7.5, 7)
# r_start, r_step, r_num = (8, 2, 6)

PERCENTS_OF_PEAK = np.arange(p_start, p_start + p_num*p_step, p_step)
MORPH_RADII = np.arange(r_start, r_start + r_num*r_step, r_step)

# thresh_step = 5
# radius_step = 2
# PERCENTS_OF_PEAK = np.arange(50,86,thresh_step)
# # PERCENTS_OF_PEAK = np.array(range(65,101,thresh_step))
# # PERCENTS_OF_PEAK = np.array(range(45,101,thresh_step))
# MORPH_RADII = np.arange(14,25,radius_step)

print(PERCENTS_OF_PEAK)
print(MORPH_RADII)

v0.5-

In [ ]:
# def get_area_for_date_str_1D(percent_of_peak, he_date_str):
#     """Retrieve detected area percentages for the specified date.
#     """
#     he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
#     he = detect.pre_process_he_v0_4(he_map.data)
#     ch_mask_data = detect.get_ch_mask(
#         he, percent_of_peak, MORPH_RADIUS
#     )
#     ch_mask_map = sunpy.map.Map(np.flipud(ch_mask_data), he_map.meta)
#     return detect.get_area(ch_mask_map, 0)[0]


def get_area_for_date_str(percent_of_peak, morph_radius, he_date_str):
    """Retrieve detected area percentages for specified dates.
    """
    he_map = prepare_data.get_nso_sunpy_map(HE_DIR + he_date_str + '.fts')
    he = detect.pre_process_v0_4(he_map.data)
    ch_mask_data = detect.get_ch_mask(
        he, percent_of_peak, morph_radius
    )
    ch_mask_map = sunpy.map.Map(np.flipud(ch_mask_data), he_map.meta)
    return detect.get_open_area(ch_mask_map, 0)
    

def get_outcomes(percent_of_peak, morph_radius):
    """Retrieve segmentation map outcomes with specified design
    variables over time.
    
    Args
        percent_of_peak_list: list of float percentage values
            at which to take threshold
        morph_radius_list: list of int pixel number for radius of disk 
            structuring element in morphological operations
    Returns
        Array of median area percentage.
        Autocorrelation and normalized MAD of the detected area time series.
    """
    print((percent_of_peak, morph_radius), end='  ')
    area_tuple_list = [
        get_area_for_date_str(percent_of_peak, morph_radius, he_date_str)
        for he_date_str in HE_DATE_LIST
    ]
    area_percent_list = [
        area_tuple[0] for area_tuple in area_tuple_list
    ]
    area_list = [
        area_tuple[1] for area_tuple in area_tuple_list
    ]
    # percent_of_peak = design_vars
    # print((design_vars, MORPH_RADIUS))
    # area_percent_list = [
    #     get_area_for_date_str_1D(percent_of_peak, he_date_str)
    #     for he_date_str in HE_DATE_LIST
    # ]
    # area_percent_list = OPTIMIZER.get_area_list(design_vars)
    
    area_percent_median = np.median(area_percent_list)
    autocorr = pd.Series(area_list).autocorr()
    
    # Compute normalized MAD of detected area
    mad = np.median(np.abs(area_list - np.median(area_list)))
    if np.median(area_list) == 0:
        norm_mad = 0
    else:
        norm_mad = mad/np.median(area_list)*100
    
    return np.array([area_percent_median, autocorr, norm_mad])

get_vect_outcomes = np.vectorize(get_outcomes, signature='(),()->(3)')

In [ ]:
# (Expensive computation: 31 min for 1 month, 48 nodes)

if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

# Optionally overwrite existing files
if os.path.isfile(save_file) and not overwrite:
    sys.exit(f'{file_date_str} data already exists.')

num_nodes = len(PERCENTS_OF_PEAK)*len(PERCENTS_OF_PEAK)
print(f'Evaluating outcomes for {num_nodes} nodes')

# ~1min/step
X, Y = np.meshgrid(PERCENTS_OF_PEAK, MORPH_RADII)
outcome_array = get_vect_outcomes(X, Y)

np.save(save_file, outcome_array, allow_pickle=True)

In [ ]:
# Merge outcome maps
output_dir = DETECTION_IMAGE_DIR + 'Outcome_Maps_L/'
save_file = f'{output_dir}{file_date_str}.npy'
area_percent_median_array_L = np.load(save_file, allow_pickle=True)[:,:,0]

output_dir = DETECTION_IMAGE_DIR + 'Outcome_Maps_R/'
save_file = f'{output_dir}{file_date_str}.npy'
area_percent_median_array_R = np.load(save_file, allow_pickle=True)[:,:,0]

area_percent_median_array = np.hstack([
    area_percent_median_array_L[:,:4], area_percent_median_array_R
])

v0.5.1+

In [ ]:
def get_contour_outcomes_for_date_str(he_date_str, percent_of_peak, morph_radius_dist):
    """Retrieve detected area for specified dates.
    """
    pre_process_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
    pre_processed_map = sunpy.map.Map(pre_process_file)

    # Obtain segmentation mask, sunpy map, and boundaries --------------------
    ch_mask_data = detect.get_ch_mask_list_vY(
        pre_processed_map, [percent_of_peak], [morph_radius_dist]
    )[0]
    ch_mask_map = sunpy.map.Map(np.flipud(ch_mask_data), pre_processed_map.meta)
    
    contours = measure.find_contours(ch_mask_data)
    ch_boundary_data = np.zeros(ch_mask_data.shape)

    for contour in contours:
        contour = contour.astype(int)
        ch_boundary_data[contour[:,0], contour[:,1]] = 1
    
    # Compute outcomes -------------------------------------------------------
    area_percent, open_area = detect.get_open_area(ch_mask_map, 0)
    fractal_D = get_fractal_D(ch_boundary_data, scale_range=[10, 500])
    
    return area_percent, open_area, fractal_D


def get_outcomes_v0_5_1(percent_of_peak, morph_radius_dist):
    """Retrieve segmentation map outcomes with specified design
    variables over time.
    
    Args
        percent_of_peak: float percentage measured from the zero
            value up to or beyond the histogram value
        morph_radius_dist: float distances in Mm for radius of
            disk structuring element in morphological operations
    Returns
        Array of median area percentage and autocorrelation of the detected
            area time series.
    """
    print((percent_of_peak, morph_radius_dist), end='  ')
    outcomes_by_dates = [
        get_contour_outcomes_for_date_str(
            he_date_str, percent_of_peak, morph_radius_dist
        )
        for he_date_str in HE_DATE_LIST
    ]
    area_percent_list = [
        date_outcomes[0] for date_outcomes in outcomes_by_dates
    ]
    area_list = [
        date_outcomes[1] for date_outcomes in outcomes_by_dates
    ]
    fractal_D_list = [
        date_outcomes[2] for date_outcomes in outcomes_by_dates
    ]
    
    area_percent_median = np.median(area_percent_list)
    autocorr = pd.Series(area_list).autocorr()
    fractal_D_median = np.nanmedian(fractal_D_list)
    
    return np.array([area_percent_median, autocorr, fractal_D_median])

get_vect_outcomes_v0_5_1 = np.vectorize(get_outcomes_v0_5_1, signature='(),()->(3)')

In [ ]:
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

# Optionally overwrite existing files
if os.path.isfile(save_file) and not overwrite:
    sys.exit(f'{file_date_str} data already exists.')

num_nodes = len(PERCENTS_OF_PEAK)*len(PERCENTS_OF_PEAK)
print(f'Evaluating outcomes for {num_nodes} nodes and {num_maps} dates')

# ~1min/step
X, Y = np.meshgrid(PERCENTS_OF_PEAK, MORPH_RADII)
outcome_array = get_vect_outcomes_v0_5_1(X, Y)

np.save(save_file, outcome_array, allow_pickle=True)

### Area

In [ ]:
area_percent_median_array = np.load(save_file, allow_pickle=True)[:,:,0]
upper_level = np.ceil(np.max(area_percent_median_array))
# levels = [0, 0.1, 0.5]
# levels.extend(list(np.linspace()))
step = 2
levels = np.arange(0,upper_level + step,step)

plt.figure(figsize=(12,8))
plt.contourf(PERCENTS_OF_PEAK, MORPH_RADII,
             area_percent_median_array, levels, cmap='plasma')
plt.colorbar()

plt.title('Median Detected Area Percentage (%)')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
# plt.ylabel('SE Disk Radius (px)')
plt.ylabel('SE Disk Radius (Mm)')
plt.savefig(f'{output_dir}Area_fill_{file_date_str}.jpg')

In [ ]:
# v0.5.1 Design
percent_of_peak_design = [70, 70, 80, 90]
morph_radius_design = [   15, 17, 13, 13] # Mm

# # v0.5.1 KPVT Design
# percent_of_peak_design = [80, 80, 90, 100]
# morph_radius_design = [   15, 17, 13, 13] # Mm

# # vY Design 1
# percent_of_peak_design = [62, 68, 73, 80]
# morph_radius_design = [   11, 13,  8, 10]

# # vY Design 2
# percent_of_peak_design = [85, 73, 95, 85]
# morph_radius_design = [   10, 14, 10, 14]

area_percent_median_array = np.load(save_file, allow_pickle=True)[:,:,0]
upper_level = np.ceil(np.max(area_percent_median_array))
step = 1
levels_1 = np.arange(0,5 + step,step)
step = 2
levels_2 = np.arange(6,upper_level + step,step)
levels = np.hstack((levels_1, levels_2))

plt.figure(figsize=(10,8))
cp = plt.contour(PERCENTS_OF_PEAK, MORPH_RADII,
                 area_percent_median_array, levels, cmap='plasma')
plt.clabel(cp, fontsize=14)

plt.title('Median Detected Area Percentage (%)')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
# plt.ylabel('SE Disk Radius (px)')
plt.ylabel('SE Disk Radius (Mm)')
# plt.savefig(f'{output_dir}Area_{file_date_str}.jpg')

plt.scatter(percent_of_peak_design, morph_radius_design, color='k')
plt.savefig(f'{output_dir}Area_Point_{file_date_str}.jpg')

### Robustness

In [ ]:
area_percent_median_array = np.load(save_file, allow_pickle=True)[:,:,0]
d_area_d_thresh = np.diff(area_percent_median_array, axis=1)/p_step

plt.figure(figsize=(12,8))
plt.contourf(PERCENTS_OF_PEAK[:-1], MORPH_RADII,
             d_area_d_thresh, cmap='bone')
plt.colorbar()

plt.title('$\partial(Median\ Area)/\partial(Threshold)$ (%/%)')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
# plt.ylabel('SE Disk Radius (px)')
plt.ylabel('SE Disk Radius (Mm)')
plt.savefig(f'{output_dir}Partial_Thresh_{file_date_str}.jpg')

In [ ]:
area_percent_median_array = np.load(save_file, allow_pickle=True)[:,:,0]
d_area_d_radius = np.diff(area_percent_median_array, axis=0)/r_step

plt.figure(figsize=(12,8))
plt.contourf(PERCENTS_OF_PEAK, MORPH_RADII[:-1],
             d_area_d_radius, cmap='bone')
plt.colorbar()

plt.title('$\partial(Median\ Area)/\partial(Radius)$ (%/px)')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
# plt.ylabel('SE Disk Radius (px)')
plt.ylabel('SE Disk Radius (Mm)')
plt.savefig(f'{output_dir}Partial_Radius_{file_date_str}.jpg')

In [ ]:
d_area_d_radius[np.where(MORPH_RADII == 10), np.where(PERCENTS_OF_PEAK == 115)]

### Persistence

In [ ]:
# Not available in updated versions
norm_mad_array = np.load(save_file, allow_pickle=True)[:,:,2]
upper_level = np.ceil(np.max(norm_mad_array))
levels = [0, 1, 10]
levels.extend(list(np.linspace(15,upper_level,7)))

plt.figure(figsize=(12,8))
plt.contourf(PERCENTS_OF_PEAK, MORPH_RADII, norm_mad_array, levels)
plt.colorbar()

plt.title('Normalized MAD of Detected Area (%)')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
plt.ylabel('SE Disk Radius (px)')
plt.savefig(f'{output_dir}Norm_MAD_{file_date_str}.jpg')

In [ ]:
autocorr_array = np.load(save_file, allow_pickle=True)[:,:,1]

plt.figure(figsize=(12,8))
plt.contourf(PERCENTS_OF_PEAK, MORPH_RADII, -autocorr_array)
plt.colorbar()

plt.title('Negative Autocorrelation')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
# plt.ylabel('SE Disk Radius (px)')
plt.ylabel('SE Disk Radius (Mm)')
plt.savefig(f'{output_dir}Autocorr_{file_date_str}.jpg')

### Boundary Complexity

In [ ]:
fractal_D_bound_median_array = np.load(save_file, allow_pickle=True)[:,:,2]
# fractal_D_bound_median_array = np.load(save_file, allow_pickle=True)[:,:,3]
lower_level = np.floor(np.nanmin(fractal_D_bound_median_array)*10)/10
upper_level = np.ceil(np.nanmax(fractal_D_bound_median_array)*10)/10
step = 0.025
levels = np.arange(lower_level, upper_level + step, step)

plt.figure(figsize=(12,8))
plt.contourf(PERCENTS_OF_PEAK, MORPH_RADII, fractal_D_bound_median_array, levels)
plt.colorbar()

plt.title('Fractal Dimension')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
plt.ylabel('SE Disk Radius (Mm)')
plt.savefig(f'{output_dir}Fractal_Dim_{file_date_str}.jpg')

In [ ]:
f = interpolate.RegularGridInterpolator(
    (MORPH_RADII, PERCENTS_OF_PEAK), fractal_D_bound_median_array
)
f((8,110))

In [ ]:
# Needs verification
interp_points = 500
num_bins = 17

area_interp = interpolate.RegularGridInterpolator(
    (MORPH_RADII, PERCENTS_OF_PEAK), area_percent_median_array
)
fractal_D_interp = interpolate.RegularGridInterpolator(
    (MORPH_RADII, PERCENTS_OF_PEAK), fractal_D_bound_median_array
)

interp_percents_of_peak = np.linspace(np.min(PERCENTS_OF_PEAK), np.max(PERCENTS_OF_PEAK), interp_points)
interp_morph_radii = np.linspace(np.min(MORPH_RADII), np.max(MORPH_RADII), interp_points)
XX, YY = np.meshgrid(interp_percents_of_peak, interp_morph_radii)

interpolated_area = area_interp((YY, XX))
interpolated_fractal_D = fractal_D_interp((YY, XX))

bins = np.linspace(np.min(area_percent_median_array), np.max(area_percent_median_array), num_bins)
binned_idx_array = np.digitize(interpolated_area, bins) - 1

X = np.zeros_like(binned_idx_array, dtype=float)

for bin_num in range(num_bins):
    bin_idxs = np.where(binned_idx_array == bin_num)
    bin_fractal_D = interpolated_fractal_D[bin_idxs]
    
    if not np.any(bin_fractal_D):
        continue

    X[bin_idxs] = bin_fractal_D - np.mean(bin_fractal_D)
    
# X = X

bound = max([np.max(X), np.abs(np.min(X))])
levels = np.linspace(-bound, bound, 15)

plt.figure(figsize=(12,8))
plt.contourf(interp_percents_of_peak, interp_morph_radii, X,
             levels, cmap='RdBu_r')
plt.colorbar()

plt.title('Binned Difference Fractal Boundary Dimension')
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.xlabel('Threshold Relative to Mode (%)')
plt.ylabel('SE Disk Radius (Mm)')

### Objective Functions

v0.5.1

In [ ]:
def obj_func_v0_5_1(norm_mad_array, area_percent_median_array, mu, M):
    penalty = mu*(area_percent_median_array - M)**2 + 1/area_percent_median_array
    obj_func_array = norm_mad_array + penalty
    return obj_func_array

def get_optim_vars(mu, M):
    obj_func_array = obj_func(NORM_MAD_ARRAY, AREA_PERCENT_MEDIAN_ARRAY, mu, M)
    xi, yi = np.unravel_index(np.argmin(obj_func_array), obj_func_array.shape)
    return np.array([PERCENTS_OF_PEAK[yi], MORPH_RADII[xi]])

get_vect_optim_vars = np.vectorize(get_optim_vars, signature='(),()->(2)')

In [ ]:
# Area penalty weight
mu = 1

# Target median area
M = 3

area_percent_median_array = np.load(save_file, allow_pickle=True)[:,:,0]
norm_mad_array = np.load(save_file, allow_pickle=True)[:,:,2]
obj_func_array = obj_func_v0_5_1(norm_mad_array, area_percent_median_array, mu, M)

z = obj_func_array[~np.isinf(obj_func_array)]
lev_exp = np.linspace(
    np.log10(z.min()), np.log10(z.max()), 10
)
levels = np.power(10, lev_exp)

plt.figure(figsize=(12,8))
cs = plt.contourf(PERCENTS_OF_PEAK, MORPH_RADII, obj_func_array, levels, 
                  norm=colors.LogNorm(), cmap='gray')
plt.colorbar(cs, format=ticker.FuncFormatter(lambda x, pos: f'{x:.1f}'))

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title(f'Objective Function | $\mu$: {mu} $M$: {M}%')
plt.xlabel('Threshold Relative to Mode (%)')
plt.ylabel('SE Disk Radius (px)')
plt.savefig(f'{output_dir}Obj_Func_Mu{mu}_M{M}_{file_date_str}.jpg')

In [ ]:
obj_func_array[np.where(MORPH_RADII == 20), np.where(PERCENTS_OF_PEAK == 115)]

In [ ]:
mu_step = 5
M_step = 0.1
mu = np.arange(0, 500 + mu_step, mu_step)
M = np.arange(2, 6 + M_step, M_step)


X, Y = np.meshgrid(mu, M)
AREA_PERCENT_MEDIAN_ARRAY = np.load(save_file, allow_pickle=True)[:,:,0]
NORM_MAD_ARRAY = np.load(save_file, allow_pickle=True)[:,:,1]

optim_var_array = get_vect_optim_vars(X, Y)
optim_var_pairs = optim_var_array.reshape((np.prod(optim_var_array.shape[:2]), 2))
unique_optim_var_pairs, optim_counts = np.unique(
    optim_var_pairs,axis=0, return_counts=True
)

plt.figure(figsize=(12,8))
plt.scatter(unique_optim_var_pairs[:,0], unique_optim_var_pairs[:,1], s=250,
            c=optim_counts, cmap='turbo')
plt.colorbar()
plt.xticks(PERCENTS_OF_PEAK)
plt.yticks(MORPH_RADII)

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Optimal Design Variables with Varied $\mu,\ M$')
plt.xlabel('Threshold Relative to Mode (%)')
plt.ylabel('SE Disk Radius (px)')

In [ ]:
plt.figure(figsize=(12,8))
plt.contourf(mu, M, optim_var_array[:,:,0], cmap='gray')
plt.colorbar()

plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Optimal Threshold Relative to Mode (%)')
plt.xlabel('Penalty Weight $mu$')
plt.ylabel('Target Median Area Percentage $M$ (%)')

v0.5.2

In [ ]:
def obj_func_v0_5_2(autocorr_array, area_percent_median_array, mu, M):
    C = mu*(area_percent_median_array - M)**2
    obj_func_array = -autocorr_array + C
    return obj_func_array

In [ ]:
# Target median area
M = 3

output_dir = DETECTION_IMAGE_DIR + 'Outcome_Maps/Obj_Func_v0_5_2/'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

area_percent_median_array = np.load(save_file, allow_pickle=True)[:,:,0]
autocorr_array = np.load(save_file, allow_pickle=True)[:,:,1]

# Area penalty weight
for mu in np.logspace(-5,0,20):
    obj_func_array = obj_func_v0_5_2(autocorr_array, area_percent_median_array, mu, M)
    if np.min(obj_func_array) < 0:
        obj_func_array = obj_func_array + np.abs(np.min(obj_func_array)) + 0.1

    z = obj_func_array[~np.isinf(obj_func_array)]
    lev_exp = np.linspace(
        np.log10(z.min()), np.log10(z.max()), 10
    )
    levels = np.power(10, lev_exp)

    fig = plt.figure(figsize=(12,8))
    
    cs = plt.contourf(PERCENTS_OF_PEAK, MORPH_RADII, obj_func_array, levels, 
                    norm=colors.LogNorm(), cmap='gray')
    plt.colorbar(cs, format=ticker.FuncFormatter(lambda x, pos: f'{x:.1f}'))

    plt.suptitle(DATE_RANGE_SUPTITLE)
    plt.title(f'Objective Function | $\mu$: {mu:.2e} $M$: {M}%')
    plt.xlabel('Threshold Relative to Mode (%)')
    plt.ylabel('SE Disk Radius (px)')
    plt.savefig(f'{output_dir}Mu{mu:.6f}_M{M}_{file_date_str}'.replace('.','_') + '.jpg')
    plt.close(fig)
    
    print(f'Mu: {mu:.2e} saved.')

In [ ]:
detect.write_ensemble_video(output_dir, fps=8)

### v0.1 Heat Maps

#### Create CH Masks

In [ ]:
save_file = f'{DETECTION_IMAGE_DIR}Outcome_Maps/{he_date_str}_maps.npy'

Create CH Mask List (Expensive computation)

In [ ]:
# Lower threshold accepts more and lets morphology carry the load in selection and removal
thresh_step = 5
radius_step = 1
percent_of_peak_list = list(np.arange(70,106,thresh_step))
morph_radius_list = list(np.arange(6,21,radius_step))

# List of CHS masks for different files with varied parameters
all_ch_mask_list = [
    detect.get_ch_mask(pre_processed_map_data, percent_of_peak, morph_radius)
    for percent_of_peak, morph_radius
    in zip(percent_of_peak_list, morph_radius_list)
]

save_list = [he_date_str, percent_of_peak_list, morph_radius_list, all_ch_mask_list]
np.save(save_file, np.array(save_list, dtype=object), allow_pickle=True)

Load CH Mask List

In [ ]:
save_list = np.load(save_file, allow_pickle=True)
date_str = save_list[0]
percent_of_peak_list = save_list[1]
morph_radius_list = save_list[2]
all_ch_mask_list = save_list[3]

In [ ]:
title_list = [f'{percent_of_peak:d}% of Peak | {radius:d}px Radius'
              for percent_of_peak in percent_of_peak_list
              for radius in morph_radius_list]

In [ ]:
def plot_heat_map(outcome_list, title, percent_of_peak_list, morph_radius_list, 
                  color_scale='Magma'):
    # Reverse order to facilitate plotting
    y_axis_list = morph_radius_list.copy()
    y_axis_list.reverse()
    
    outcome_map = np.flipud(np.reshape(
        outcome_list, (len(percent_of_peak_list),len(morph_radius_list))).T)

    fig = px.imshow(outcome_map, 
                    labels=dict(x='Threshold Level as Percent of Peak (%)',
                                y='SE Disk Radius (px)'),
                    x=percent_of_peak_list, y=y_axis_list,
                    aspect='auto', color_continuous_scale=color_scale)
    fig.update_layout(title=title, width=700)
    fig.show()
    
    
def plot_heat_map_band(outcome_list, heat_map_title, lower_bound, upper_bound,
                       percent_of_peak_list, morph_radius_list, 
                       array, all_ch_masks_list, title_list, color_scale='Magma'):
    edit_outcome_list = outcome_list.copy()
    
    # Index list of outcomes within bounds 
    idx_list = [i for i in range(len(edit_outcome_list)) 
              if edit_outcome_list[i] >= lower_bound and edit_outcome_list[i] <= upper_bound]
    num_ch_masks = len(idx_list)
    
    if not num_ch_masks:
        print('No masks in outcome range')
        return
    
    max_outcome = max(edit_outcome_list)
    # Highlight outcomes within bounds
    for i in idx_list:
        edit_outcome_list[i] = 2*max_outcome

    plot_heat_map(edit_outcome_list, heat_map_title,
                  percent_of_peak_list, morph_radius_list, color_scale)
    
    if num_ch_masks == 1:
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot()
        
        outcome_idx = idx_list[0]
        ax.imshow(array, cmap=plt.cm.afmhot)
        ax.contour(all_ch_masks_list[outcome_idx], linewidths=0.5, cmap=plt.cm.gray)
        ax.set_title(title_list[outcome_idx], fontsize=18)
    else:
        fig, axes = plt.subplots(nrows=1, ncols=num_ch_masks, figsize=(6*num_ch_masks, 6))
        ax = axes.ravel()
    
        for i in range(num_ch_masks):
            outcome_idx = idx_list[i]
            ax[i].imshow(array, cmap=plt.cm.afmhot)
            ax[i].contour(all_ch_masks_list[outcome_idx], linewidths=0.5, cmap=plt.cm.gray)
            ax[i].set_title(title_list[outcome_idx], fontsize=18)

#### Pixel Percentage

In [ ]:
area_percent_list = detect.get_px_percent_list(all_ch_mask_list)

In [ ]:
heat_map_title = f'{date_str} Segmented Area Percentage'

plot_heat_map(
    area_percent_list, heat_map_title, percent_of_peak_list, morph_radius_list
)

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot()

ax.hist(area_percent_list[:-1], bins=30)
ax.set_title(f'{date_str} Segmented Area Bins', fontsize=20)
ax.set_xlabel('Area Percentage', fontsize=18)
ax.set_ylabel('Number of Masks in Area Bin', fontsize=18)

Heat Map Bands

In [ ]:
lower_bound = 16
upper_bound = 18

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list
)

In [ ]:
lower_bound = 6.8
upper_bound = 7.3

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list
)

In [ ]:
lower_bound = 5.4
upper_bound = 6

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list
)

In [ ]:
lower_bound = 3.2
upper_bound = 3.3

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list
)

#### CH Number

In [ ]:
num_ch_list = detect.get_num_CH_list(all_ch_mask_list)

In [ ]:
heat_map_title = f'{date_str} Segmented Hole Number'
color_scale = 'Aggrnyl'

plot_heat_map(
    num_ch_list, heat_map_title, percent_of_peak_list, morph_radius_list, color_scale
)

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot()

ax.hist(num_ch_list, bins=30, range=(0,25))
ax.set_title(f'{date_str} Segmented Hole Number Bins', fontsize=20)
ax.set_xlabel('Segemented Hole Number', fontsize=18)
ax.set_ylabel('Number of Masks in Hole Number Bin', fontsize=18)

Heat Map Bands

In [ ]:
lower_bound = 17
upper_bound = 17

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Segmented Holes'
color_scale = 'Aggrnyl'

plot_heat_map_band(
    num_ch_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list, color_scale
)

In [ ]:
lower_bound = 11
upper_bound = 11

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Segmented Holes'
color_scale = 'Aggrnyl'

plot_heat_map_band(
    num_ch_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list, color_scale
)

In [ ]:
lower_bound = 9
upper_bound = 9

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Segmented Holes'
color_scale = 'Aggrnyl'

plot_heat_map_band(
    num_ch_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list, color_scale
)

#### Lower Tail Width

In [ ]:
def get_ch_lower_tail_width_list(array, ch_mask_list): 
    """Retrieve the average of the histogram lower tail width across CHs
    for each segmentation in a list.
    
    Args
        array: image to process
        ch_mask_list: binary coronal holes mask list
    Returns
        List of mean histogram tail widths of CHs detected in segmentations.
    """
    labeled_ch_list = [ndimage.label(ch_mask)[0]
                          for ch_mask in ch_mask_list]
    num_ch_list = [get_num_ch(ch_mask) for ch_mask in ch_mask_list]
    
    # List of average lower tail widths across all CH's of each segmentaion
    lower_tail_width_list = []
    
    count = 0
    for labeled_ch_mask, num_ch in zip(labeled_ch_list, num_ch_list):

        map_data_by_ch = get_map_data_by_ch(array, labeled_ch_mask, num_ch)
    
        ch_mask_lower_tail_width_list = get_ch_lower_tail_widths(map_data_by_ch)
        
        mean_lower_tail_width = np.mean(ch_mask_lower_tail_width_list)
        
        lower_tail_width_list.append(mean_lower_tail_width)
        count = count + 1
        
    return lower_tail_width_list

In [ ]:
lower_tail_width_list = detect.get_ch_lower_tail_width_list(he, all_ch_mask_list)

In [ ]:
heat_map_title = f'{date_str} Mean CH Tail Width'
color_scale = 'ice'

plot_heat_map(
    lower_tail_width_list, heat_map_title, percent_of_peak_list, morph_radius_list, color_scale
)

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot()

ax.hist(lower_tail_width_list, bins=30, range=(10,40))
ax.set_title(f'{date_str} Mean CH Tail Width Bins', fontsize=20)
ax.set_xlabel('Mean CH Tail Width', fontsize=18)
ax.set_ylabel('Number of Masks in Tail Width Bin', fontsize=18)

Heat Map Bands

In [ ]:
lower_bound = 29
upper_bound = 29.5

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Mean CH Tail Width'
color_scale = 'ice'

plot_heat_map_band(
    lower_tail_width_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list, color_scale
)

In [ ]:
lower_bound = 24.5
upper_bound = 25

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Mean CH Tail Width'
color_scale = 'ice'

plot_heat_map_band(
    lower_tail_width_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    he, all_ch_mask_list, title_list, color_scale
)

#### Multiplied

In [ ]:
heat_map_title = f'{date_str} Multiplied Metrics'
color_scale = 'dense_r'

outcome_list = list(np.array(area_percent_list)*np.array(num_ch_list)*np.array(lower_tail_width_list))

plot_heat_map(
    outcome_list, heat_map_title, percent_of_peak_list, morph_radius_list, color_scale
)

## Confidence Histograms

In [ ]:
def get_outcomes_by_all_date_ch(cl_list):
    """Retrieve outcomes per CH in ensemble maps in all datetimes
    at specified confidence levels from ensemble maps.
    
    See get_outcomes for retrieved outcomes.
    
    Args
        cl_list: list of float confidence levels at which
            to threshold ensemble maps for computing outcomes
    Returns
        Dataframes of outcomes by confidence level over time.
    """
    # Dictionaries for outcomes of distinct CHs at varied confidence levels
    area_dict = {cl:[] for cl in cl_list}
    lat_dict = {cl:[] for cl in cl_list}
    lon_dict = {cl:[] for cl in cl_list}
    unsigned_flux_dict = {cl:[] for cl in cl_list}
    signed_flux_dict = {cl:[] for cl in cl_list}
    mag_skew_dict = {cl:[] for cl in cl_list}
    unipolarity_dict = {cl:[] for cl in cl_list}

    for he_date_str in HE_DATE_LIST:
        
        # Extract He I observation
        he_file = f'{HE_DIR}{he_date_str}.fts'
        he_map = prepare_data.get_nso_sunpy_map(he_file)
        if not he_map:
            print(f'{he_date_str} He I observation extraction failed.')
            continue
        
        # Extract saved ensemble map
        ensemble_file = f'{DETECTION_SAVE_DIR}{he_date_str}_ensemble_map.npy'
        ensemble_map_data = np.load(ensemble_file, allow_pickle=True)[-1]
        ensemble_map = sunpy.map.Map(np.flipud(ensemble_map_data), he_map.meta)
        
        # Extract saved processed magnetograms
        mag_date_str = prepare_data.get_nearest_date_str(
            MAG_DATE_LIST, selected_date_str=he_date_str
        )
        reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                                 + f'Mag{mag_date_str}_He{he_date_str}.fits')
        reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)
        
        # Outcomes per CH detected at the given or greater
        # confidence levels
        outcome_by_ch_dict_by_cl = [
            detect.get_outcomes_by_ch(ensemble_map, pre_processed_map,
                                      reprojected_mag_map, cl)
            for cl in cl_list
        ]
        area_single_date_dict = {
            cl:outcome_by_ch_dict['area'] for cl, outcome_by_ch_dict
            in zip(cl_list, outcome_by_ch_dict_by_cl)
        }
        lat_single_date_dict = {
            cl:outcome_by_ch_dict['cm_lat'] for cl, outcome_by_ch_dict
            in zip(cl_list, outcome_by_ch_dict_by_cl)
        }
        lon_single_date_dict = {
            cl:outcome_by_ch_dict['cm_lon'] for cl, outcome_by_ch_dict
            in zip(cl_list, outcome_by_ch_dict_by_cl)
        }
        unsigned_flux_single_date_dict = {
            cl:outcome_by_ch_dict['unsigned_flux'] for cl, outcome_by_ch_dict
            in zip(cl_list, outcome_by_ch_dict_by_cl)
        }
        signed_flux_single_date_dict = {
            cl:outcome_by_ch_dict['signed_flux'] for cl, outcome_by_ch_dict
            in zip(cl_list, outcome_by_ch_dict_by_cl)
        }
        mag_skew_single_date_dict = {
            cl:outcome_by_ch_dict['mag_skew'] for cl, outcome_by_ch_dict
            in zip(cl_list, outcome_by_ch_dict_by_cl)
        }
        unipolarity_single_date_dict = {
            cl:outcome_by_ch_dict['unipolarity'] for cl, outcome_by_ch_dict
            in zip(cl_list, outcome_by_ch_dict_by_cl)
        }
        
        # Extend outcomes per CH
        for cl in cl_list:
            area_dict[cl].extend(
                area_single_date_dict[cl]
            )
            lat_dict[cl].extend(
                lat_single_date_dict[cl]
            )
            lon_dict[cl].extend(
                lon_single_date_dict[cl]
            )
            unsigned_flux_dict[cl].extend(
                unsigned_flux_single_date_dict[cl]
            )
            signed_flux_dict[cl].extend(
                signed_flux_single_date_dict[cl]
            )
            mag_skew_dict[cl].extend(
                mag_skew_single_date_dict[cl]
            )
            unipolarity_dict[cl].extend(
                unipolarity_single_date_dict[cl]
            )
            
    return area_dict, lat_dict, lon_dict, \
        unsigned_flux_dict, signed_flux_dict, \
        mag_skew_dict, unipolarity_dict


In [ ]:
def get_outcomes_by_all_date_ch_v0_5_1(cl_list):
    """Retrieve outcomes per CH in ensemble maps in all datetimes
    at specified confidence levels from ensemble maps.
    
    See get_outcomes for retrieved outcomes.
    
    Args
        cl_list: list of float confidence levels at which
            to threshold ensemble maps for computing outcomes
    Returns
        Dataframes of outcomes by confidence level over time.
    """
    # Dictionaries for outcomes of distinct CHs at varied confidence levels
    outcomes_by_all_ch_dict = {}
    for outcome_key in detect.OUTCOME_KEY_LIST:
        outcomes_by_all_ch_dict[outcome_key] = {cl:[] for cl in cl_list}


    for he_date_str in HE_DATE_LIST:
        
        # Extract saved ensemble map array and convert to Sunpy map
        ensemble_file = f'{DETECTION_MAP_SAVE_DIR}{he_date_str}_ensemble_map.fits'
        ensemble_map = sunpy.map.Map(ensemble_file)
        
        # Extract saved He I observation
        he_fits_file = DATA_FITS_FORMAT.format(
            data_dir=HE_DIR, date_str=he_date_str
        )
        he_map = prepare_data.get_nso_sunpy_map(he_fits_file)
        he_map_data = np.flipud(he_map.data)
        
        # Extract saved processed magnetogram
        mag_date_str = prepare_data.get_nearest_date_str(
            MAG_DATE_LIST, selected_date_str=he_date_str
        )
        reprojected_fits_file = (f'{ROTATED_MAG_SAVE_DIR}'
                                 + f'Mag{mag_date_str}_He{he_date_str}.fits')
        reprojected_mag_map = sunpy.map.Map(reprojected_fits_file)
        
        # Extract single date outcomes by CH ---------------------------------
        # Applied at varied confidence levels, then extending a main
        # dictionary with outcomes by CH from all dates
        
        # List of varied confidence levels for outcomes per CH detected
        # at the given or greater confidence levels
        outcome_by_ch_dict_by_cl = [
            detect.get_outcomes_by_ch(ensemble_map, he_map_data,
                                      reprojected_mag_map, cl)
            for cl in cl_list
        ]
        
        # Dictionary of outcomes holding dictionaries of
        # confidence levels for outcomes per CH
        single_date_outcome_dict = {}
        for outcome_key in detect.OUTCOME_KEY_LIST:
            single_date_outcome_dict[outcome_key] = {
                cl:outcome_by_ch_dict[outcome_key] for cl, outcome_by_ch_dict
                in zip(cl_list, outcome_by_ch_dict_by_cl)
            }
        
        # Extend main outcomes per CH dictionary by confidence level
        for cl in cl_list:
            for outcome_key in detect.OUTCOME_KEY_LIST:
                outcomes_by_all_ch_dict[outcome_key][cl].extend(
                    single_date_outcome_dict[outcome_key][cl]
                )
            
    return outcomes_by_all_ch_dict

### Evaluate outcomes

In [ ]:
cl_list = [0, 35, 65, 95]
outcome_dicts = get_outcomes_by_all_date_ch(cl_list)
area_dict, lat_dict, lon_dict = outcome_dicts[:3]
unsigned_flux_dict, signed_flux_dict = outcome_dicts[3:5]
mag_skew_dict, unipolarity_dict = outcome_dicts[5:]

v0.5.1+

In [ ]:
# cl_list = [50, 75, 90]
cl_list = [0]
unipolarity_confidence = True
outcomes_by_all_ch_dict = get_outcomes_by_all_date_ch_v0_5_1(cl_list)

In [ ]:
# Number of candidate regions
# Requires setting DETECTION_VERSION_DIR = DETECT_DIR + 'v0_5_1_No_Thresh/'
len(outcomes_by_all_ch_dict[detect.OUTCOME_KEY_LIST[0]][0])

### Lat/Lon

In [ ]:
outcome_dict = outcomes_by_all_ch_dict['cm_lon']
plt.figure(figsize=(10,6))
plt.title('Longitude Histogram')
plt.ylabel('Number of CH Detections')
plt.xlabel('Longitude (deg)')
orientation = 'vertical'
plt.xlim([-90,90])
plt.ylim([0,50])

# plt.ylim([0,120])

# outcome_dict = outcomes_by_all_ch_dict['cm_lat']
# plt.figure(figsize=(7,6))
# plt.title('Latitude Histogram')
# plt.ylabel('Latitude (deg)')
# plt.xlabel('Number of CH Detections')
# orientation = 'horizontal'
# plt.ylim([-90,90])

cmap = colormaps['bone_r']
color_list = cmap(np.linspace(0.25, 1, len(cl_list)))

plt.suptitle(DATE_RANGE_SUPTITLE)

bins = np.arange(-90,90.1,10)
for cl, color in zip(cl_list, color_list):
    if unipolarity_confidence:
        label = f'>= {cl/100} Unipolarity'
    else:
        label = f'{cl}th % Smoothness'
    
    plt.hist(
        outcome_dict[cl], bins, color=color,
        orientation=orientation, label=label
    )
    
plt.legend()

Sine Lat/Lon

In [ ]:
sz_factor = 0.6

total_width = 13.5
lon_width = 8

outcome_dict = outcomes_by_all_ch_dict['cm_lon']
plt.figure(figsize=(lon_width*sz_factor,5*sz_factor), dpi=300)
# Prev 6
plt.title('Longitude Histogram')
plt.ylabel('Number of CH Detections')
plt.xlabel(r'sin($\phi$)')
orientation = 'vertical'
plt.xlim([-1,1])
plt.ylim([0,50])
plt.xticks([-1, -0.5, 0, 0.5, 1])
loc = 'upper left'

# outcome_dict = outcomes_by_all_ch_dict['cm_lat']
# plt.figure(figsize=((total_width - lon_width)*sz_factor,5*sz_factor), dpi=300)
# plt.title('Latitude Histogram')
# plt.ylabel(r'sin($\theta$)')
# plt.xlabel('Number of CH Detections')
# orientation = 'horizontal'
# plt.ylim([-1,1])
# plt.xlim([0,70])
# plt.yticks([-1, -0.5, 0, 0.5, 1])
# loc = 'upper right'

cmap = colormaps['bone_r']
color_list = cmap(np.linspace(0.25, 1, len(cl_list)))

# plt.suptitle(DATE_RANGE_SUPTITLE)

bins = np.arange(-1,1.01,0.125)
for cl, color, linestyle in zip(cl_list, color_list, ['-', '--', '-.']):
    sine_angle = np.sin(np.deg2rad(outcome_dict[cl]))
    
    if unipolarity_confidence:
        label = fr'$U \geq${cl/100}'
    else:
        label = f'{cl}th % Smoothness'
    
    # plt.hist(
    #     sine_angle, bins, color,
    #     orientation=orientation, label=label
    # )
    plt.hist(
        sine_angle, bins, histtype='step',
        color='white', edgecolor=color,
        linestyle=linestyle, linewidth=2,
        orientation=orientation
    )
    plt.plot([2,3], [2,3], color=color, linestyle=linestyle, label=label)
    
plt.legend(loc=loc)

### Unsigned Flux

In [ ]:
outcome_dict = outcomes_by_all_ch_dict['unsigned_flux']

bin_min = np.log10(np.min(outcome_dict[0]))
bin_max = np.ceil(np.log10(np.max(outcome_dict[0])))
bins = 10**(np.linspace(bin_min,bin_max,25))

cmap = colormaps['bone_r']
color_list = cmap(np.linspace(0.25, 1, len(cl_list)))

plt.figure(figsize=(10,6))
plt.xscale('log')
plt.title('Unsigned Flux Histogram')
plt.ylabel('Number of CH Candidates')
plt.xlabel('Unsigned Open Flux (Wb)')

for cl, color in zip(cl_list, color_list):
    if unipolarity_confidence:
        label = f'{cl/100} Unipolarity'
    else:
        label = f'{cl}th % Smoothness'
    
    plt.hist(
        outcome_dict[cl], bins, color=color,
        orientation=orientation, label=label
    )
    
plt.legend()

### Unipolarity

In [ ]:
unipolarity_threshold = 0.5
smooth_percentile_bounds = [0, 50, 80]

# Outcomes for all candidates (>=0% confidence)
cl = 0
unipolarity_list = outcomes_by_all_ch_dict['unipolarity'][cl]
grad_median_list = outcomes_by_all_ch_dict['grad_median'][cl]

# Smoothness percentile by candidate
# Mapped in [0,100) and reversed order from gradient median quanitfying roughness
smooth_percentiles = 100 - stats.rankdata(grad_median_list)/len(grad_median_list)*100

# Stratify candidate CH unipolarity by percentiles of smoothness
u_by_smooth_pct_dict = {}
    
for smooth_pct_bound in smooth_percentile_bounds:
    
    candidate_u_above_smooth_pct_list = [
        unipolarity for unipolarity, smooth_percentile
        in zip(unipolarity_list, smooth_percentiles)
        if smooth_percentile >= smooth_pct_bound
    ]
    u_by_smooth_pct_dict[smooth_pct_bound] = candidate_u_above_smooth_pct_list

bins = np.arange(0,1.01,0.05)

cmap = colormaps['bone_r']
color_list = cmap(np.linspace(0.25, 1, len(smooth_percentile_bounds)))

line_styles = ['-', '--', '-.']

plt.figure(figsize=(10,6))
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Unipolarity Histogram')
plt.ylabel('Number of CH Candidates')
plt.xlabel('Unipolarity')
plt.xticks([0,0.25,0.5,0.75,1])
plt.xlim([0,1])
plt.ylim([0,140])

for smooth_pct, color, linestyle in zip(
    smooth_percentile_bounds, color_list, line_styles):
    
    plt.hist(
        u_by_smooth_pct_dict[smooth_pct], bins, histtype='step',
        color='white', edgecolor=color,
        linestyle=linestyle, linewidth=3,
        label=f'{smooth_pct}th % Smoothness'
    )

plt.legend()
plt.vlines([unipolarity_threshold, unipolarity_threshold], ymin=-10, ymax=150,
           linestyles=['--'], color='k', linewidth=1)

In [ ]:
num_candidates = len(u_by_smooth_pct_dict[0])

u_among_all_candidates = np.array(u_by_smooth_pct_dict[0])
num_bipolar_candidates = np.count_nonzero(u_among_all_candidates < 0.5)
num_unipolar_candidates = np.count_nonzero(u_among_all_candidates >= 0.5)

f'Bipolar candidate fraction: {num_bipolar_candidates/num_candidates*100:.2f}%'

In [ ]:
u_among_smooth_candidates = np.array(u_by_smooth_pct_dict[50])

num_bipolar_smooth_candidates = np.count_nonzero(u_among_smooth_candidates < 0.5)
num_unipolar_smooth_candidates = np.count_nonzero(u_among_smooth_candidates >= 0.5)

('Bipolar, unsmooth candidate fraction: '
 f'{num_bipolar_smooth_candidates/num_bipolar_candidates*100:.2f}%')

In [ ]:
('Unpolar, smooth candidate fraction: '
 f'{num_unipolar_smooth_candidates/num_unipolar_candidates*100:.2f}%')

In [ ]:
len(u_among_all_candidates) - len(u_among_unsmooth_candidates)

In [ ]:
outcome_dict = unipolarity_dict

# bins = np.linspace(0,1,25)
bins = np.arange(0,1.01,0.05)

cmap = colormaps['bone_r']
color_list = cmap(np.linspace(0.25, 1, len(cl_list)))

plt.figure(figsize=(10,6))
plt.suptitle(DATE_RANGE_SUPTITLE)
plt.title('Unipolarity Histogram')
plt.ylabel('Number of CH Candidates')
plt.xlabel('Unipolarity')

for cl, color in zip(cl_list, color_list):
    if unipolarity_confidence:
        label = f'{cl/100} Unipolarity'
    else:
        label = f'{cl}th % Smoothness'
    
    plt.hist(
        outcome_dict[cl], bins, color=color, label=label
    )

plt.legend()
plt.xlim([0,1])

# COSPAR

In [ ]:
# ROC Curve

# plt.figure(figsize=(5,5), dpi=150)
plt.figure(figsize=(7,7), dpi=150)
plt.plot([0,1], [0,1], linestyle='--', color='k', label='Random Classifier')
plt.scatter(0,0.78,label='CHIMERA')
plt.scatter(0.04,0.84,label='SPOCA')
plt.scatter(0.07,0.83,label='ACWE03')
plt.scatter(0.13,0.85,label='CRONNOS')
plt.scatter(0.25,0.87,label='SPOCA-HEK')
plt.scatter(0.29,0.86,label='ACWE04',marker='^',color='green')
plt.scatter(0.38,0.98,label='WWWBCS')
plt.scatter(0.49,0.99,label='CHMAP')

plt.scatter(0.85,0.92,label='CHORTLE')
plt.scatter(0.35,0.82,label='CNN193')
plt.scatter(0.54,0.86,label='CHIPS',marker='^')
plt.scatter(0.62,0.87,label='SYNCH',marker='^')
plt.scatter(0.21,0.77,label='TH35',marker='^')
plt.scatter(0.21,0.79,label='CHARM',marker='v')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.xlim([0,1])
plt.ylim([0,1])
plt.legend(loc='lower right', reverse=True)

## Label CHs

Obtain preliminary segmentations, plot numbered CHs, specify labels, and save CH_LABEL_DICT for a single date

In [ ]:
he_date_str = HE_DATE_LIST[0]

# # v0.5.1 SOLIS Design
# percent_of_peak_list = [70, 70, 80, 90]
# morph_radius_list = [   15, 17, 13, 13] # Mm

# Aggressive/Conservative Masks from v0.5.1 SOLIS Design
percent_of_peak_list = [70, 90]
morph_radius_list = [   15, 13] # Mm

# # v0.5.1 KPVT Design
# percent_of_peak_list = [85, 105, 85, 95]
# morph_radius_list = [   17, 13, 15, 13] # Mm

# Extract FITS file pre-processed map
pre_process_fits_file = (PREPROCESS_MAP_SAVE_DIR + he_date_str
                        + '_pre_processed_map.fits')
pre_processed_map = sunpy.map.Map(pre_process_fits_file)
pre_processed_map_data = np.flipud(pre_processed_map.data)

ch_mask_list = detect.get_ch_mask_list_v0_5_1(
    pre_processed_map, percent_of_peak_list, morph_radius_list
)

ch_label_dict = {
    'label_list': {
        0: [],
        1: []
    },
    'labeled_data': {
        0: [],
        1: []
    }
}

In [ ]:
DETECTION_IMAGE_DIR

In [ ]:
DISK_VAL = -0.5
UNLABELED_VAL = 1.5

LABEL_CMAP = colors.ListedColormap(['black', 'indianred', 'gold', 'forestgreen', 'gray'])

cmap_object = colormaps['tab20']
black_start_cmap = np.vstack(([0,0,0,1], cmap_object(np.arange(0,20,1))))
NUMBERED_CMAP = colors.ListedColormap(black_start_cmap)


def save_ch_label_dict(pre_processed_map_data, he_date_str, ch_mask_list, ch_label_dict):
    # Array with number labels per distinct CH and number of labels
    for mask_idx, ch_mask in enumerate(ch_mask_list):
        # Array with number labels per distinct CH and number of labels
        numbered_mask, num_ch = ndimage.label(ch_mask)
        
        # Remove background to display CHs on disk
        numbered_mask_on_disk = np.where(
            np.isnan(pre_processed_map_data), np.nan, numbered_mask
        )
        
        # Initialize and iterate to assign labels to CHs
        labeled_mask_on_disk = np.where(
            numbered_mask_on_disk == 0, DISK_VAL, numbered_mask_on_disk
        )
        
        ch_label_list = ch_label_dict['label_list'][mask_idx]
        assert(len(ch_label_list) == num_ch,
               f'CHs in mask {mask_idx} have been left unlabeled.')
        
        for ch_idx in range(num_ch):
            labeled_mask_on_disk = np.where(
                labeled_mask_on_disk == ch_idx + 1,
                ch_label_list[ch_idx], labeled_mask_on_disk
            )
        ch_label_dict['labeled_data'][mask_idx] = labeled_mask_on_disk
        
    # Save dictionary
    out_dir = f'{DETECTION_IMAGE_DIR}Labels/'
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
    
    np.save(f'{out_dir}{he_date_str}.npy', ch_label_dict, allow_pickle=True)
    print(f'{he_date_str} Label Dictionary Saved')
        
    return ch_label_dict


def plot_labeled_chs(pre_processed_map_data, he_date_str, ch_label_dict,
                     percent_of_peak_list, morph_radius_list,
                     ch_mask_list, plot_numbered):
    image_size = 5
    num_cols = len(ch_mask_list)
    fig = plt.figure(figsize=((image_size+1.5)*num_cols, image_size))
    
    fig.suptitle(he_date_str, fontsize=16)

    for mask_idx, ch_mask in enumerate(ch_mask_list):
        # Array with number labels per distinct CH and number of labels
        numbered_mask, num_ch = ndimage.label(ch_mask)
        
        # Remove background to display CHs on disk
        numbered_mask_on_disk = np.where(
            np.isnan(pre_processed_map_data), np.nan, numbered_mask
        )

        ax = fig.add_subplot(1, num_cols, mask_idx + 1)
        if numbered_mode:
            im = ax.imshow(numbered_mask_on_disk, NUMBERED_CMAP,
                           interpolation='nearest', vmin=-0.5, vmax=20.5)
            fig.colorbar(im, ticks=np.arange(0,20.5,4))
        else:
            # Extend label list for unlabeled CHs
            ch_label_list = ch_label_dict['label_list'][mask_idx]
            ch_label_list = (
                ch_label_list[:num_ch] + [UNLABELED_VAL]*(num_ch - len(ch_label_list))
            )
            
            # Initialize and iterate to assign labels to CHs
            labeled_mask_on_disk = np.where(
                numbered_mask_on_disk == 0, DISK_VAL, numbered_mask_on_disk
            )
            for ch_idx in range(num_ch):
                labeled_mask_on_disk = np.where(
                    labeled_mask_on_disk == ch_idx + 1,
                    ch_label_list[ch_idx], labeled_mask_on_disk
                )
            
            im = ax.imshow(
                labeled_mask_on_disk, LABEL_CMAP, interpolation='nearest',
                vmin=DISK_VAL - 0.25, vmax=UNLABELED_VAL + 0.25
            )
            cb = fig.colorbar(
                im, fraction=0.05, ticks=np.linspace(DISK_VAL, UNLABELED_VAL, 5)
            )
            cb.ax.set_yticklabels(['Disk', '0: False', '0.5: Marginal', '1: True', 'Unlabeled'])
        
        ax.axes.xaxis.set_ticks([])
        ax.axes.yaxis.set_ticks([])
        ax.set_title((f'{percent_of_peak_list[mask_idx]:d}% of Mode Threshold | '
                      f'{morph_radius_list[mask_idx]:d}Mm SE Disk Radius'))

# NUMBERED_CMAP

In [ ]:
plot_numbered = False

ch_label_dict['label_list'][0] = [
    0, 0, 1, 0, # blue, orange
    1, 0, 0, 1, # green, red
    0, 0, 0, 0, # purple, brown
]
ch_label_dict['label_list'][1] = [
    0, 0, 0, 1, # blue, orange
    1, 0, 1, 0, # green, red
    1, 0, 0, 0, # purple, brown
]

plot_labeled_chs(pre_processed_map_data, he_date_str, ch_label_dict,
                 percent_of_peak_list, morph_radius_list,
                 ch_mask_list, plot_numbered)

Update labels and verify saving to CH_LABEL_DICT

In [ ]:
ch_label_dict = save_ch_label_dict(
    pre_processed_map_data, he_date_str, ch_mask_list, ch_label_dict
)

labeled_mask_on_disk = ch_label_dict['labeled_data'][1]

im = plt.imshow(
    labeled_mask_on_disk, LABEL_CMAP, interpolation='nearest',
    vmin=DISK_VAL - 0.25, vmax=UNLABELED_VAL + 0.25
)
cb = plt.colorbar(
    im, fraction=0.05, ticks=np.linspace(DISK_VAL, UNLABELED_VAL, 5)
)
cb.ax.set_yticklabels(['Disk', '0: False', '0.5: Marginal', '1: True', 'Unlabeled'])
print()

# Optimization

In [ ]:
# 4s/map * 30 maps/step = 150s/step

In [ ]:
percent_of_peak = 90
morph_radius = 10

he_map = prepare_data.get_nso_sunpy_map(HE_DIR + HE_DATE_LIST[0] + '.fts')
he = detect.pre_process_v0_4(he_map.data)
ch_mask_data = detect.get_ch_mask(
    he, percent_of_peak, morph_radius
)
plt.imshow(ch_mask_data)

Eval Functions

In [ ]:
import functools

class Optimizer:
    
    @functools.lru_cache(maxsize=None)
    def get_area_list(self, design_vars):
        """Retrieve detected area percentages for specied dates.
        """
        percent_of_peak, morph_radius = design_vars
        area_percent_list = []

        for he_date_str in HE_DATE_LIST:
            he_map = prepare_data.get_nso_sunpy_map(
                HE_DIR + he_date_str + '.fts'
            )
            he = detect.pre_process_v0_4(he_map.data)
            ch_mask_data = detect.get_ch_mask(
                he, percent_of_peak, morph_radius
            )
            ch_mask_map = sunpy.map.Map(np.flipud(ch_mask_data), he_map.meta)
            area_percent = detect.get_open_area(ch_mask_map, 0)[0]
            area_percent_list.append(area_percent)
            
        return area_percent_list

# Initiate optimizer object with cached area percent list
OPTIMIZER = Optimizer()

In [ ]:
percent_of_peak = 90
morph_radius = 10
MORPH_RADIUS = 10
MAX_AREA_PERCENT = 4
MIN_AREA_PERCENT = 1
design_vars = (percent_of_peak, morph_radius)


def obj_func():
    """Objective function for persistence optimization.
    Penalizes normalized MAD of detected area
    """
    pass

def constraints(design_vars):
    """Constraints for persistence optimization.
    """
    area_percent_list = [6, 5, 4, 9]
    # OPTIMIZER.get_area_list(design_vars)
    mean_area_percent = np.mean(area_percent_list)
    
    return (mean_area_percent - MIN_AREA_PERCENT,
            MAX_AREA_PERCENT - mean_area_percent)
    # return np.array([mean_area_percent - MIN_AREA_PERCENT],
    #                  MAX_AREA_PERCENT - mean_area_percent])

Test Functions

In [ ]:
optimizer.get_area_list(design_vars)

In [ ]:
get_area_list(design_vars)

In [ ]:
obj_func(90, 10)

In [ ]:
constraints(design_vars)

## Visual

## Minimize

### SLSQP

Scipy options

In [ ]:
optimize.show_options(solver='minimize', method='SLSQP')

In [ ]:
optimize.rosen([0.5, 0, 10])

Execute Minimize

In [ ]:
design_vars = (percent_of_peak, morph_radius)
ineq_cons = ({
    'type': 'ineq',
    'fun' : constraints,
    # 'args': (design_vars,)
})
# Unconstrained
res = optimize.minimize(
    obj_func, percent_of_peak, method='SLSQP',
    options={'disp': True, 'finite_diff_rel_step': [0.1]},
)
# Constrained
# res = optimize.minimize(
#     obj_func, design_vars, args=(optimizer,), method='SLSQP', constraints=ineq_cons,
#     options={'ftol': 1e-9, 'disp': True}
# )

### BFGS

In [ ]:
percent_of_peak = 100
design_vars = (percent_of_peak, morph_radius)
res = optimize.minimize(
    obj_func, percent_of_peak, method='BFGS',
    options={'disp': True, 'xrtol': 0.01, 'eps': 0.1},
)

In [ ]:
res1 = res
res1

# NSO CH Estimates

## Input Level 2 Products: **Stage/Level2/**

10830i equivalent width: **svsm_e3100_S2_yyyymmdd_hhmm.fts.gz**

10830i intensity: **svsm_i3000_S2_yyyymmdd_hhmm.fts.gz**

6302l magnetogram: **svsm_m1100_S2_yyyymmdd_hhmm.fts.gz**

6302l intensity: **svsm_i1000_S2_yyyymmdd_hhmm.fts.gz**

In [ ]:
raw_he_eqw_fits_path = NSO_INPUT_DIR + 'svsm_e3100_S2_20140626_1419.fts'

raw_he_intensity_fits_path = NSO_INPUT_DIR + 'svsm_i3000_S2_20140626_1419.fts'

raw_magnetogram_fits_path = NSO_INPUT_DIR + 'svsm_m1100_S2_20140626_1444.fts'

raw_mag_intensity_fits_path = NSO_INPUT_DIR + 'svsm_i1000_S2_20140626_1444.fts'

im_list = plot_detection.plot_raw_fits_content(
    raw_he_eqw_fits_path, header_list=['IMTYPE'],
    # print_header=True
)
plot_detection.plot_raw_fits_content(
    raw_he_intensity_fits_path, header_list=['IMTYPE'],
    # print_header=True
)
plot_detection.plot_raw_fits_content(
    raw_magnetogram_fits_path, header_list=['IMTYPE'],
    print_header=True
)
plot_detection.plot_raw_fits_content(
    raw_mag_intensity_fits_path, header_list=['IMTYPE'],
    # print_header=True
)

## Output Level 3 Products: **Stage/Level3/**
He I EqW Maps List: **hDataList.txt**
- 'Text list of available 10830 He EqW low-res sine-latlon heliographic maps'
- Produced by **mk_datalist**
  - Args: heliographic He I EqW maps

Magnetogram Maps List: **mDataList.txt**
- 'Text list of available 6301.5 low-res sine-latlon heliographic maps'
- Produced by **mk_datalist**
  - Args: heliographic magnetograms

CH Maps List: **oDataList.txt**
- 'Text list of available 10830 solar wind source sine-latlon heliographic maps'
- Produced by **mk_datalist**
  - Args: heliographic CH maps

Carrington Rotation CH Images
- '10830 solar wind sine-latlon daily synoptic map plots'
- High-Res: **chsh.jpg**
- Med-Res: **chsm.jpg**
- Low-Res: **chsl.jpg**
- Produced by **plot_lev3_map**
  - Args: 1 carrington rotation CH map, CH maps list

Unincluded He I Observation Image?: **svsm_o10mr_S3_{yyyymmdd}_{hhmm}.jpg**
- Produced by **mk_obsimg**
  - Args: 1 sky frame disk CH map, CH maps list

#### Single Maps: **../single/{yyyy}/**

Sine-LatLon He I EQW: **svsm_e31hr_B3_{yyyymmdd}_{hhmm}.fts.gz**
- '10830 He EqW high-res sine-latlon heliographic map'
- Produced by **mk_synimg**
  - Args: L2 He I EQW file, L2 He I continuum intensity file

LatLon He I EQW: **svsm_e31lr_L3_{yyyymmdd}_{hhmm}.fts.gz**
- '10830 He EqW low-res latlon heliographic map'
- Produced by **mk_synimg**
  - Args: L2 He I EQW file, L2 He I continuum intensity file

Sine-LatLon Magnetogram: **svsm_m11hr_B3_{yyyymmdd}_{hhmm}.fts.gz**
- '6301.5 high-res sine-latlon heliographic map'
- Produced by **mk_synimg**
  - Args: L2 magnetogram file, L2 6302l magnetogram intensity file

LatLon Magnetogram: **svsm_m11lr_L3_{yyyymmdd}_{hhmm}.fts.gz**
- '6301.5 low-res latlon heliographic map'
- Produced by **mk_synimg**
  - Args: L2 magnetogram file, L2 6302l magnetogram intensity file

Sine-LatLon CH Map: **svsm_o1083_B3_{yyyymmdd}_{hhmm}.fts.gz**
- '10830 solar wind source sine-latlon heliographic map'
- Produced by **mk_holeimg**
  - Args: processed He I EqW, He I EqW maps list, processed magnetogram, magnetogram maps list

Sky Frame Disk CH Map: **svsmgo1083_B3_{yyyymmdd}_{hhmm}.fts.gz**
- '10830 solar wind source sky frame heliocentric map'
- Produced by **mk_dchimg**
  - Args: 1 disk CH map, disk CH maps list

In [ ]:
# he_eqw_fits_path = NSO_SINGLE_DIR + 'svsm_e31hr_B3_20140606_1746.fts'
he_eqw_fits_path = NSO_SINGLE_DIR + 'svsm_e31hr_B3_20140626_1419.fts'
# he_eqw_fits_path = NSO_SINGLE_DIR + 'svsm_e31lr_L3_20140626_1419.fts'

# mag_fits_path = NSO_SINGLE_DIR + 'svsm_m11hr_B3_20140606_1605.fts'
mag_fits_path = NSO_SINGLE_DIR + 'svsm_m11hr_B3_20140626_1444.fts'

# E: Empty file
# E ch_fits_map = NSO_SINGLE_DIR + 'svsm_o1083_B3_20140601_1836.fts'
# E ch_fits_map = NSO_SINGLE_DIR + 'svsm_o1083_B3_20140603_1704.fts'
# E ch_fits_map = NSO_SINGLE_DIR + 'svsm_o1083_B3_20140606_1755.fts'
# E ch_fits_map = NSO_SINGLE_DIR + 'svsm_o1083_B3_20140612_1438.fts'
# E ch_fits_map = NSO_SINGLE_DIR + 'svsm_o1083_B3_20140620_1711.fts'
ch_fits_map = NSO_SINGLE_DIR + 'svsm_o1083_B3_20140626_1428.fts'

# E sky_ch_fits_path = NSO_SINGLE_DIR + 'svsmgo1083_B3_20140606_1755.fts'
sky_ch_fits_path = NSO_SINGLE_DIR + 'svsmgo1083_B3_20140626_1428.fts'

plot_detection.plot_raw_fits_content(
    he_eqw_fits_path, header_list=['IMTYPE', 'COMMENT2'],
    cmaps=[plt.cm.gray, plt.cm.gray],
    # print_header=True
)
plot_detection.plot_raw_fits_content(
    mag_fits_path, header_list=['IMTYPE', 'COMMENT1', 'COMMENT2'],
    cmaps=[plt.cm.gray, plt.cm.gray, plt.cm.gray],
    # print_header=True
)
plot_detection.plot_raw_fits_content(
    ch_fits_map, header_list=['IMGTYP01', 'IMGTYP02', 'IMGTYP03'],
    print_header=True
)
im_list = plot_detection.plot_raw_fits_content(
    sky_ch_fits_path, header_list=['COMMENT2'],
    # print_header=True
)

##### NSO Processed EQW

In [ ]:
fits = '~/Desktop/out_solarstrm_data/' + 'svsm_e31lr_L3_20140626_1419.fts'
im_list = plot_detection.plot_raw_fits_content(
    fits, header_list=['IMTYPE', 'COMMENT2'],
    cmaps=[plt.cm.gray, plt.cm.gray, plt.cm.gray],
    print_header=True
)

In [ ]:
date_str = '2014_06_26__00_00'

raw_nso_eqw = NSO_EQW_DICT[date_str]
nso_eqw_nan = detect.pre_process_eqw_v0_1(raw_nso_eqw)[2]

titles = ['EQW', 'EQW NaN']
plot_detection.plot_hists(
    [raw_nso_eqw, nso_eqw_nan], titles, semilogy=True
)

lower_bounds = [-1600,  0,  0]
upper_bounds = [0,      250, 500]
plot_detection.plot_thresholds(
    nso_eqw_nan, bounds=[lower_bounds, upper_bounds], 
    bounds_as_percent=False, threshold_type='band'
)
plot_detection.plot_thresholds(
    nso_eqw_nan, bounds=[75, 85, 100], bounds_as_percent=True
)

### Merged Maps: **../merged/carr-daily/**
Carrington Rotation CH Map: **svsm_o31hr_B3_cr{RRRR}_{DDD}.fts.gz**
- 'Solar wind source high-res sine-latlon daily synoptic map'
- Produced by **create_crmap**
  - Args: 27 most recent disk CH maps, disk CH maps list

In [ ]:
synoptic_ch_fits_path = NSO_MERGED_DIR + 'svsm_o31hr_B3_cr2152_275.fts'

plot_detection.plot_raw_fits_content(
    synoptic_ch_fits_path,
    # header_list=['DATE', 'CARR01', 'IMTYPE'],
    header_list=['IMGTYP01', 'IMGTYP02', 'IMGTYP03', 'IMGTYP04'],
    # print_header=True
)

## Algorithm on Pre-Processed EQW

In [ ]:
date_str = '2014_06_26__00_00'

raw_nso_eqw = NSO_EQW_DICT[date_str]
nso_eqw_nan = detect.pre_process_eqw_v0_1(raw_nso_eqw)[2]

percent_of_peak_list = [80,85,90]
radius_list = [6]

ensemble_map, holes_mask_list, confidence_list = detect.get_ensemble_v0_3(
    nso_eqw_nan, percent_of_peak_list, radius_list)

plot_detection.plot_ensemble(
    nso_eqw_nan, ensemble_map, confidence_list, holes_mask_list
)

## NSO Carrington Map Comparison

In [ ]:
# Does not work
percent_of_peak_list = [80,90,100]
radius_list = [11,13,15]

for he_date_str in list(reversed(HE_DATE_LIST)):
    raw_he = HE_FITS_DICT[date_str][0]
    he = detect.pre_process_v0_1(raw_he)[0]
    
    ensemble_map = detect.get_ensemble(
        he, percent_of_peak_list, radius_list
    )[0]

    euv = EUV_DICT[he_date_str]

    plot_ensemble_comparison(he, he_date_str, ensemble_map, euv)

In [ ]:
def plot_ch_map(date_str_list, cr_str, ch_map_dict):
    """Plot NSO detected CH Carrington map.
    """
    # Display selected column number corresponding to date list
    selected_datetime_list = [
        datetime.strptime(
            date_str, DICT_DATE_STR_FORMAT)
        for date_str in date_str_list
    ]
    selected_cr_list = [
        carrington_rotation_number(selected_datetime)
        for selected_datetime in selected_datetime_list
    ]
    
    cr_str_list = cr_str.split('_')
    cr_num_list = [float(cr_str) for cr_str in cr_str_list]
    
    cr_range = cr_num_list[-1] - cr_num_list[0]
    cr_percent_list = [
        (selected_cr - cr_num_list[0])/cr_range
        for selected_cr in selected_cr_list
    ]
    
    ch_map = ch_map_dict[cr_str]
    rows, cols = ch_map.shape
    
    selected_col_list = [
        cols - cr_percent*cols
        for cr_percent in cr_percent_list
    ]
    
    print('Selected Date Columns:')
    for date_str, selected_col in zip(
        date_str_list, selected_col_list):
        print(f'{date_str}: {selected_col:.1f}px \t', end='')

    # Prepare the figure and axes with map projection
    fig = plt.figure(figsize=(10, 10))

    ax = fig.add_subplot()
    ax.set_title(f'CR{cr_str}', fontsize=20)
    
    ax.imshow(ch_map, extent=[0,cols, rows, 0])
    ax.vlines(x=selected_col_list, ymin=rows, ymax=0, linestyles='dashed',
              colors='black')


def rename_all_gong(gong_dir):
    """Rename all GONG magnetogram FITS files to include observation date in title"""
    glob_pattern = gong_dir + '*.fits'
    
    fits_path_list = glob.glob(glob_pattern)
    
    for fits_path in fits_path_list:
        gong_fits = fits.open(fits_path)
        
        gong_fits_header_keys = list(gong_fits[0].header.keys())
                
        # Pass to next FITS file if header information is missing
        if 'CAR_ROT' not in gong_fits_header_keys:
            continue
        
        # Carrington Rotation
        CR_str = f'CR{gong_fits[0].header["CAR_ROT"]}'
        
        gong_fits.close()
            
        os.rename(fits_path, gong_dir + CR_str + '.fits')

In [ ]:
cr_str = '2151.0342_2152.1035'

plot_ch_map(list(reversed(HE_DATE_LIST)), cr_str, CH_MAP_DICT)